In [89]:
import os
from IPython.display import IFrame
import pandas as pd
import sys
import igv
import numpy as np
import pyBigWig
import sys
sys.path.insert(0, '../../')
from JKBio import TerraFunction as terra
from ccle_processing.src.CCLE_postp_function import *
from JKBio import Helper as h
from JKBio.epigenetics import ChIP_helper as chip
import dalmatian as dm
import itertools
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from bokeh.plotting import *
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
output_notebook()
%load_ext autoreload
%autoreload 2

Loading BokehJS ...

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# ChIP

In [ ]:
!gsutil ls gs://amlproject/Chip_AML/Chip/

In [ ]:
a = !gsutil ls gs://amlproject/Chip_AML/Chip/IRF2BP2/

In [ ]:
# install bwa
! mkdir ref
! curl ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/215/GCF_000001215.4_Release_6_plus_ISO1_MT/GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna.gz -o ref/reference_droso.fna.gz
! gunzip ref/reference_droso.fna.gz
! bwa index -a bwtsw ref/reference_droso.fna

## V1

### analysis

In [ ]:
rename1 = {
 'gs://transfer-amlproject/20191211_10_MP7613_S8_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp910_1.fastq.gz",
 'gs://transfer-amlproject/20191211_10_MP7613_S8_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp910_2.fastq.gz",
 'gs://transfer-amlproject/20191211_11_MP7613_S9_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp911_1.fastq.gz",
 'gs://transfer-amlproject/20191211_11_MP7613_S9_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp911_2.fastq.gz",
 'gs://transfer-amlproject/20191211_12_MP7613_S10_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp912_1.fastq.gz",
 'gs://transfer-amlproject/20191211_12_MP7613_S10_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp912_2.fastq.gz",
 'gs://transfer-amlproject/20191211_13_MP7613_S11_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp913_1.fastq.gz",
 'gs://transfer-amlproject/20191211_13_MP7613_S11_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp913_2.fastq.gz",
 'gs://transfer-amlproject/20191211_14_MP7613_S12_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp914_1.fastq.gz",
 'gs://transfer-amlproject/20191211_14_MP7613_S12_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp914_2.fastq.gz",
 'gs://transfer-amlproject/20191211_15_MP7613_S13_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp915_1.fastq.gz",
 'gs://transfer-amlproject/20191211_15_MP7613_S13_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp915_2.fastq.gz",
 'gs://transfer-amlproject/20191211_16_MP7613_S14_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp916_1.fastq.gz",
 'gs://transfer-amlproject/20191211_16_MP7613_S14_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp916_2.fastq.gz",
 'gs://transfer-amlproject/20191211_1_MP7613_S1_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp901_1.fastq.gz",
 'gs://transfer-amlproject/20191211_1_MP7613_S1_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp901_2.fastq.gz",
 'gs://transfer-amlproject/20191211_2_MP7613_S2_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp902_1.fastq.gz",
 'gs://transfer-amlproject/20191211_2_MP7613_S2_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp902_2.fastq.gz",
 'gs://transfer-amlproject/20191211_3_MP7613_S3_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp903_1.fastq.gz",
 'gs://transfer-amlproject/20191211_3_MP7613_S3_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp903_2.fastq.gz",
 'gs://transfer-amlproject/20191211_4_MP7613_S4_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp904_1.fastq.gz",
 'gs://transfer-amlproject/20191211_4_MP7613_S4_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp904_2.fastq.gz",
 'gs://transfer-amlproject/20191211_5R_MP7613_S15_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp905_1.fastq.gz",
 'gs://transfer-amlproject/20191211_5R_MP7613_S15_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp905_2.fastq.gz",
 'gs://transfer-amlproject/20191211_6R_MP7613_S16_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp906_1.fastq.gz",
 'gs://transfer-amlproject/20191211_6R_MP7613_S16_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp906_2.fastq.gz",
 'gs://transfer-amlproject/20191211_7_MP7613_S5_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp907_1.fastq.gz",
 'gs://transfer-amlproject/20191211_7_MP7613_S5_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp907_2.fastq.gz",
 'gs://transfer-amlproject/20191211_8_MP7613_S6_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp908_1.fastq.gz",
 'gs://transfer-amlproject/20191211_8_MP7613_S6_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp908_2.fastq.gz",
 'gs://transfer-amlproject/20191211_9_MP7613_S7_R1_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp909_1.fastq.gz",
 'gs://transfer-amlproject/20191211_9_MP7613_S7_R2_001.fastq.gz':"gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/mp909_2.fastq.gz"}

In [ ]:
for k, val in rename1.items():
    os.system('gsutil mv '+k+' '+val)

In [ ]:
! mkdir bams && gsutil -m cp "gs://amlproject/Chip_AML/IRF2BP2/IRF2BP2/*" bams

In [ ]:
inputfastq="gs://amlproject/Chip_AML/Chip/mp99.fastq.gz"
! gsutil cp $inputfastq bams
a = ! ls bams

In [ ]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
url="https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U"
gsheet = sheets.get(url).sheets[2].to_frame()

In [ ]:
gsheet

In [ ]:
df = {
"fastq1": [],
"fastq2": [],
"antibody": [],
"group": [],
"replicate": [],
"control": []
}
for val in helper.grouped(a[:-1],2):
    row = gsheet[gsheet.id==val[0].split('/')[-1].split('_')[0]]
    df['group'].append((row.id.values[0] + '_'+ row.name_replicate.values[0] +"_"+row.protein.values[0]))
    df['replicate'].append(1)
    df['fastq1'].append(val[0])
    #df['fastq2'].append(val[1])
    df['antibody'].append(row.protein.values[0])
    df['control'].append("INPUT")
df['group'].append('INPUT')
df['replicate'].append(1)
df['fastq1'].append(a[-1])
#df['fastq2'].append(val[1])
df['antibody'].append("INPUT")
df['control'].append("INPUT")
df = pd.DataFrame(df)

In [ ]:
df[df.columns[[3,4,0,1,2,5]]].to_csv('design.csv')

In [ ]:
#process chips
! sudo ./nextflow run nf-core/chipseq --single_end --seq_center 'DFCI' --email 'jkobject@gmail.com' --narrow_peak --input design.csv --genome GRCh38 --skip_preseq --max_cpus 24 -profile docker -w work

In [ ]:
#!gsutil cp results gs://amlproject/Chip_AML/Chip/IRF2BP2/ && mv results results1 && sudo rm -r work

In [ ]:
# get scaling values
mappedreads, umappedreads_norm, mapped = chip.getSpikeInControlScales(refgenome="ref/reference_droso.fna",FastQfolder='bams',pairedEnd=True, cores=8)
mappedreads, umappedreads_norm, mapped

computing scales from the excel sheet

In [20]:
scales = [[562285,1496707],
[1686168,7198567],
[3642441,2612624],
[3992589,3474812],
[3347901,3829477],
[6181136,989703],
[7523840,4173047],
[922482,1195857]]

In [135]:
scales = [[1.0, 0.3756814126], #IRF2BP2
[1.0, 0.2342366196], #MED1
[0.5546432407, 1.0], #MYB
[0.1601166841, 1.0], #MYC
[0.870315477, 1.0], #POL2S2
[1.0, 0.8742449687], #POL2S5
[0.7172728398, 1.0], #POL2TOT
[1.0, 0.7713982525]] #SPI1

### on scalled data


In [3]:
!gsutil -m cp gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/*.bam ../../data/diffBinding_rep1/

Copying gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/mp784-MV411_IRF2BP2_DMSO_MYC-MYC-r1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/mp785-MV411_IRF2BP2_DMSO_MYB-MYB-r1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/MV411_IRF2BP2_DMSO_R1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/mp780-MV411_IRF2BP2_DMSO_MED1-MED1-r1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/MV411_IRF2BP2_VHL_R1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/mp779-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/mp782-MV411_IRF2BP2_DMSO_POLII_S2-POLII_S2-r1.mLb.clN.sorted.bam...
Copying gs

In [136]:
bams = !ls ../../data/diffBinding_rep1/diffData/*treat_pileup.bdg
bams = ['_'.join(i.split('/')[-1].split('_')[:-2]) for i in bams]
bams

['MV411_IRF2BP2_DMSO_FLAG_IRF2BP2_R1',
 'MV411_IRF2BP2_DMSO_MED1_R1',
 'MV411_IRF2BP2_DMSO_MYB_R1',
 'MV411_IRF2BP2_DMSO_MYC_R1',
 'MV411_IRF2BP2_DMSO_POLII_S2_R1',
 'MV411_IRF2BP2_DMSO_POLII_S5_R1',
 'MV411_IRF2BP2_DMSO_POLII_total_R1',
 'MV411_IRF2BP2_DMSO_SPI1_R1',
 'MV411_IRF2BP2_VHL_FLAG_IRF2BP2_R1',
 'MV411_IRF2BP2_VHL_MED1_R1',
 'MV411_IRF2BP2_VHL_MYB_R1',
 'MV411_IRF2BP2_VHL_MYC_R1',
 'MV411_IRF2BP2_VHL_POLII_S2_R1',
 'MV411_IRF2BP2_VHL_POLII_S5_R1',
 'MV411_IRF2BP2_VHL_POLII_total_R1',
 'MV411_IRF2BP2_VHL_SPI1_R1']

In [138]:
# diffPeak on scalled data
sizes = [220, 191, 211, 204, 285, 222, 228, 194]
for i in range(int(len(bams)/2)):
    name1 = bams[i]
    name2 = bams[i+8]
    print(name1,name2)
    print(chip.diffPeak(name1,name2, directory= "../../data/diffBinding_rep1/diffData/", res_directory='../../data/diffBinding_rep1/diffPeaks/', scaling1=scales[i][1], scaling2=scales[i][0], size=size[i]))

MV411_IRF2BP2_DMSO_FLAG_IRF2BP2_R1 MV411_IRF2BP2_VHL_FLAG_IRF2BP2_R1
doing differential peak binding
CompletedProcess(args='macs2 bdgdiff --t1 ../../data/diffBinding_rep1/diffData/MV411_IRF2BP2_DMSO_FLAG_IRF2BP2_R1_treat_pileup.bdg --c1 ../../data/diffBinding_rep1/diffData/MV411_IRF2BP2_DMSO_FLAG_IRF2BP2_R1_control_lambda.bdg --t2 ../../data/diffBinding_rep1/diffData/MV411_IRF2BP2_VHL_FLAG_IRF2BP2_R1_treat_pileup.bdg --c2 ../../data/diffBinding_rep1/diffData/MV411_IRF2BP2_VHL_FLAG_IRF2BP2_R1_control_lambda.bdg --d1 0.3756814126 --d2 1.0 -g 60 -l 206 --o-prefix MV411_IRF2BP2_DMSO_FLAG_IRF2BP2_R1_vs_MV411_IRF2BP2_VHL_FLAG_IRF2BP2_R1 --outdir ../../data/diffBinding_rep1/diffPeaks/', returncode=0, stdout=b'', stderr=b'INFO  @ Wed, 29 Apr 2020 15:50:06: Read and build treatment 1 bedGraph... \nINFO  @ Wed, 29 Apr 2020 15:50:58: Read and build control 1 bedGraph... \nINFO  @ Wed, 29 Apr 2020 15:52:33: Read and build treatment 2 bedGraph... \nINFO  @ Wed, 29 Apr 2020 15:53:23: Read and build 

In [22]:
# diffPeak on scalled data
for i in range(int(len(bams[1:])/2)-1):
    name1 = bams[1+i]
    name2 = bams[9+i]
    chip.fullDiffPeak(name1,name2, bams[0], scaling = scales[i],compute_size=True)

doing diff from ../../data/diffBinding_rep1/mp779-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.mLb.clN.sorted.bam and ../../data/diffBinding_rep1/mp787-MV411_IRF2BP2_VHL_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.mLb.clN.sorted.bam
computing the fragment avg size
220
562285 1496707
doing differential peak binding
doing diff from ../../data/diffBinding_rep1/mp780-MV411_IRF2BP2_DMSO_MED1-MED1-r1.mLb.clN.sorted.bam and ../../data/diffBinding_rep1/mp788-MV411_IRF2BP2_VHL_MED1-MED1-r1.mLb.clN.sorted.bam
computing the fragment avg size
191
1686168 7198567
doing differential peak binding
doing diff from ../../data/diffBinding_rep1/mp781-MV411_IRF2BP2_DMSO_POLII_total-POLII_total-r1.mLb.clN.sorted.bam and ../../data/diffBinding_rep1/mp789-MV411_IRF2BP2_VHL_POLII_total-POLII_total-r1.mLb.clN.sorted.bam
computing the fragment avg size
228
3642441 2612624
doing differential peak binding
doing diff from ../../data/diffBinding_rep1/mp782-MV411_IRF2BP2_DMSO_POLII_S2-POLII_S2-r1.mLb.clN.sorted.bam and ../../dat

In [ ]:
scales = [1.0,
1.0,
0.5546432407,
0.1601166841,
0.870315477,
1.0,
0.7172728398,
1.0,
0.3756814126,
0.2342366196,
1.0,
1.0,
1.0,
0.8742449687,
1.0,
0.7713982525]

In [ ]:
chip.bigWigFrom(bams[1:],genome='GRCh38',scaling=scales)

In [139]:
bw = ! ls ../../data/recalib_bigwig_rep1/*.bw
bw

['../../data/recalib_bigwig_rep1/MV411_IRF2BP2_DMSO_FLAG_IRF2BP2_R1.bw',
 '../../data/recalib_bigwig_rep1/MV411_IRF2BP2_DMSO_MED1_R1.bw',
 '../../data/recalib_bigwig_rep1/MV411_IRF2BP2_DMSO_MYB_R1.bw',
 '../../data/recalib_bigwig_rep1/MV411_IRF2BP2_DMSO_MYC_R1.bw',
 '../../data/recalib_bigwig_rep1/MV411_IRF2BP2_DMSO_POLII_S2_R1.bw',
 '../../data/recalib_bigwig_rep1/MV411_IRF2BP2_DMSO_POLII_S5_R1.bw',
 '../../data/recalib_bigwig_rep1/MV411_IRF2BP2_DMSO_POLII_total_R1.bw',
 '../../data/recalib_bigwig_rep1/MV411_IRF2BP2_DMSO_SPI1_R1.bw',
 '../../data/recalib_bigwig_rep1/MV411_IRF2BP2_VHL_FLAG_IRF2BP2_R1.bw',
 '../../data/recalib_bigwig_rep1/MV411_IRF2BP2_VHL_MED1_R1.bw',
 '../../data/recalib_bigwig_rep1/MV411_IRF2BP2_VHL_MYB_R1.bw',
 '../../data/recalib_bigwig_rep1/MV411_IRF2BP2_VHL_MYC_R1.bw',
 '../../data/recalib_bigwig_rep1/MV411_IRF2BP2_VHL_POLII_S2_R1.bw',
 '../../data/recalib_bigwig_rep1/MV411_IRF2BP2_VHL_POLII_S5_R1.bw',
 '../../data/recalib_bigwig_rep1/MV411_IRF2BP2_VHL_POLII_tota

In [140]:
cond1peak = ! ls ../../data/diffBinding_rep1/diffPeaks/*cond1.bed
cond2peak = ! ls ../../data/diffBinding_rep1/diffPeaks/*cond2.bed
commonpeak = ! ls ../../data/diffBinding_rep1/diffPeaks/*common.bed
commonpeak

['../../data/diffBinding_rep1/diffPeaks/MV411_IRF2BP2_DMSO_FLAG_IRF2BP2_R1_vs_MV411_IRF2BP2_VHL_FLAG_IRF2BP2_R1_c3.0_common.bed',
 '../../data/diffBinding_rep1/diffPeaks/MV411_IRF2BP2_DMSO_MED1_R1_vs_MV411_IRF2BP2_VHL_MED1_R1_c3.0_common.bed',
 '../../data/diffBinding_rep1/diffPeaks/MV411_IRF2BP2_DMSO_MYB_R1_vs_MV411_IRF2BP2_VHL_MYB_R1_c3.0_common.bed',
 '../../data/diffBinding_rep1/diffPeaks/MV411_IRF2BP2_DMSO_MYC_R1_vs_MV411_IRF2BP2_VHL_MYC_R1_c3.0_common.bed',
 '../../data/diffBinding_rep1/diffPeaks/MV411_IRF2BP2_DMSO_POLII_S2_R1_vs_MV411_IRF2BP2_VHL_POLII_S2_R1_c3.0_common.bed',
 '../../data/diffBinding_rep1/diffPeaks/MV411_IRF2BP2_DMSO_POLII_S5_R1_vs_MV411_IRF2BP2_VHL_POLII_S5_R1_c3.0_common.bed',
 '../../data/diffBinding_rep1/diffPeaks/MV411_IRF2BP2_DMSO_POLII_total_R1_vs_MV411_IRF2BP2_VHL_POLII_total_R1_c3.0_common.bed',
 '../../data/diffBinding_rep1/diffPeaks/MV411_IRF2BP2_DMSO_SPI1_R1_vs_MV411_IRF2BP2_VHL_SPI1_R1_c3.0_common.bed']

In [141]:
names = ["FLAG_IRF2BP2","MED1","MYB","MYC","POLII_S2","POLII_S5","POLII_total","SPI1"]

In [142]:
for i in range(int(len(bw)/2)):
    if i<0:
        continue
    name1 = bw[i]
    name2 = bw[8+i]
    peak = [cond1peak[i], commonpeak[i], cond2peak[i]]
    for val in peak:
        chip.dropWeirdChromosomes(val)
    name = names[i]
    print(name1,name2)
    chip.computePeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], window=3000, refpoint='center', folder="", title=name, numthreads=7, torecompute=True, name='../../data/diffBinding_rep1/diffPeaks/'+name+'_mat.pdf', withDeeptools=True)

found 1 header line... removing
removed 8 lines
found 1 header line... removing
found 0 header line... removing
removed 1 lines
../../data/recalib_bigwig_rep1/MV411_IRF2BP2_DMSO_FLAG_IRF2BP2_R1.bw ../../data/recalib_bigwig_rep1/MV411_IRF2BP2_VHL_FLAG_IRF2BP2_R1.bw
CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_rep1/MV411_IRF2BP2_DMSO_FLAG_IRF2BP2_R1.bw ../../data/recalib_bigwig_rep1/MV411_IRF2BP2_VHL_FLAG_IRF2BP2_R1.bw  --referencePoint center --regionsFileName ../../data/diffBinding_rep1/diffPeaks/MV411_IRF2BP2_DMSO_FLAG_IRF2BP2_R1_vs_MV411_IRF2BP2_VHL_FLAG_IRF2BP2_R1_c3.0_cond1.bed ../../data/diffBinding_rep1/diffPeaks/MV411_IRF2BP2_DMSO_FLAG_IRF2BP2_R1_vs_MV411_IRF2BP2_VHL_FLAG_IRF2BP2_R1_c3.0_common.bed  --missingDataAsZero --outFileName ../../data/diffBinding_rep1/diffPeaks/FLAG_IRF2BP2_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotHeatmap --matrixFile ../../data/diffBinding_rep1/diffPeaks/FLAG_IRF2BP2_mat.gz --outFileNam

CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_rep1/MV411_IRF2BP2_DMSO_POLII_total_R1.bw ../../data/recalib_bigwig_rep1/MV411_IRF2BP2_VHL_POLII_total_R1.bw  --referencePoint center --regionsFileName ../../data/diffBinding_rep1/diffPeaks/MV411_IRF2BP2_DMSO_POLII_total_R1_vs_MV411_IRF2BP2_VHL_POLII_total_R1_c3.0_cond1.bed ../../data/diffBinding_rep1/diffPeaks/MV411_IRF2BP2_DMSO_POLII_total_R1_vs_MV411_IRF2BP2_VHL_POLII_total_R1_c3.0_common.bed ../../data/diffBinding_rep1/diffPeaks/MV411_IRF2BP2_DMSO_POLII_total_R1_vs_MV411_IRF2BP2_VHL_POLII_total_R1_c3.0_cond2.bed  --missingDataAsZero --outFileName ../../data/diffBinding_rep1/diffPeaks/POLII_total_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotHeatmap --matrixFile ../../data/diffBinding_rep1/diffPeaks/POLII_total_mat.gz --outFileName ../../data/diffBinding_rep1/diffPeaks/POLII_total_mat.pdf --refPointLabel center --regionsLabel DMSO_peaks common VHL_peaks  --samplesLabel DMSO VHL

In [107]:
mkdir ../../data/peaks1

In [112]:
! gsutil cp gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/macs/narrowPeak/*.narrowPeak ../../data/peaks1

Copying gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/macs/narrowPeak/MV411_IRF2BP2_DMSO_R1_peaks.narrowPeak...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/macs/narrowPeak/MV411_IRF2BP2_VHL_R1_peaks.narrowPeak...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/macs/narrowPeak/mp779-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.narrowPeak...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/macs/narrowPeak/mp780-MV411_IRF2BP2_DMSO_MED1-MED1-r1.narrowPeak...
\ [4 files][ 17.2 MiB/ 17.2 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/macs/narrowPeak/mp781-MV411_IRF2BP2_DMSO_POLII_total-PO

In [6]:
peaks = ! ls ../../data/peaks1/*
peaks

['../../data/peaks1/mp779-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.narrowPeak',
 '../../data/peaks1/mp780-MV411_IRF2BP2_DMSO_MED1-MED1-r1.narrowPeak',
 '../../data/peaks1/mp781-MV411_IRF2BP2_DMSO_POLII_total-POLII_total-r1.narrowPeak',
 '../../data/peaks1/mp782-MV411_IRF2BP2_DMSO_POLII_S2-POLII_S2-r1.narrowPeak',
 '../../data/peaks1/mp783-MV411_IRF2BP2_DMSO_POLII_S5-POLII_S5-r1.narrowPeak',
 '../../data/peaks1/mp784-MV411_IRF2BP2_DMSO_MYC-MYC-r1.narrowPeak',
 '../../data/peaks1/mp785-MV411_IRF2BP2_DMSO_MYB-MYB-r1.narrowPeak',
 '../../data/peaks1/mp786-MV411_IRF2BP2_DMSO_SPI1-SPI1-r1.narrowPeak',
 '../../data/peaks1/mp787-MV411_IRF2BP2_VHL_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.narrowPeak',
 '../../data/peaks1/mp788-MV411_IRF2BP2_VHL_MED1-MED1-r1.narrowPeak',
 '../../data/peaks1/mp789-MV411_IRF2BP2_VHL_POLII_total-POLII_total-r1.narrowPeak',
 '../../data/peaks1/mp790-MV411_IRF2BP2_VHL_POLII_S2-POLII_S2-r1.narrowPeak',
 '../../data/peaks1/mp791-MV411_IRF2BP2_VHL_POLII_S5-POLII_S5-r1.narrowP

In [7]:
names = ["FLAG_IRF2BP2","MED1","MYB","MYC","POLII_S2","POLII_S5","POLII_total","SPI1"]
names.extend([i+'_VHL' for i in names])
names

['FLAG_IRF2BP2',
 'MED1',
 'MYB',
 'MYC',
 'POLII_S2',
 'POLII_S5',
 'POLII_total',
 'SPI1',
 'FLAG_IRF2BP2_VHL',
 'MED1_VHL',
 'MYB_VHL',
 'MYC_VHL',
 'POLII_S2_VHL',
 'POLII_S5_VHL',
 'POLII_total_VHL',
 'SPI1_VHL']

In [8]:
for i in range(len(bw)):
    chip.computePeaksAt(peaks[i], bw[i], window=3000, folder="", title=names[i], numthreads=7, torecompute=True, onlyProfile=True, refpoint='center', name='../../data/diffBinding_rep1/diffPeaks/'+names[i]+'_mat_profile.pdf', withDeeptools=True)
    chip.computePeaksAt(peaks[i], bw[i], window=3000, folder="", title=names[i], numthreads=7, torecompute=True, refpoint='center',onlyProfile=True,name='../../data/diffBinding_rep1/diffPeaks/'+names[i]+'_mat_profile_clust3.pdf', withDeeptools=True, cluster=3)

CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_rep1/MV411_IRF2BP2_DMSO_FLAG_IRF2BP2_R1.bw  --referencePoint center --regionsFileName ../../data/peaks1/mp779-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.narrowPeak  --missingDataAsZero --outFileName ../../data/diffBinding_rep1/diffPeaks/FLAG_IRF2BP2_mat_profile.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotProfile --matrixFile ../../data/diffBinding_rep1/diffPeaks/FLAG_IRF2BP2_mat_profile.gz --outFileName ../../data/diffBinding_rep1/diffPeaks/FLAG_IRF2BP2_mat_profile.pdf --refPointLabel center --plotTitle FLAG_IRF2BP2', returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_rep1/MV411_IRF2BP2_DMSO_FLAG_IRF2BP2_R1.bw  --referencePoint center --regionsFileName ../../data/peaks1/mp779-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.narrowPeak  --missingDataAsZero --outFileName ../../data/diffBinding_rep1/diffPeaks/FLAG

CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_rep1/MV411_IRF2BP2_DMSO_POLII_total_R1.bw  --referencePoint center --regionsFileName ../../data/peaks1/mp785-MV411_IRF2BP2_DMSO_MYB-MYB-r1.narrowPeak  --missingDataAsZero --outFileName ../../data/diffBinding_rep1/diffPeaks/POLII_total_mat_profile_clust3.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotProfile --matrixFile ../../data/diffBinding_rep1/diffPeaks/POLII_total_mat_profile_clust3.gz --outFileName ../../data/diffBinding_rep1/diffPeaks/POLII_total_mat_profile_clust3.pdf --refPointLabel center --perGroup --kmeans 3 --plotTitle POLII_total', returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_rep1/MV411_IRF2BP2_DMSO_SPI1_R1.bw  --referencePoint center --regionsFileName ../../data/peaks1/mp786-MV411_IRF2BP2_DMSO_SPI1-SPI1-r1.narrowPeak  --missingDataAsZero --outFileName ../../data/diffBinding_rep1/diffPeaks/SPI1_mat

CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_rep1/MV411_IRF2BP2_VHL_POLII_S5_R1.bw  --referencePoint center --regionsFileName ../../data/peaks1/mp792-MV411_IRF2BP2_VHL_MYC-MYC-r1.narrowPeak  --missingDataAsZero --outFileName ../../data/diffBinding_rep1/diffPeaks/POLII_S5_VHL_mat_profile.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotProfile --matrixFile ../../data/diffBinding_rep1/diffPeaks/POLII_S5_VHL_mat_profile.gz --outFileName ../../data/diffBinding_rep1/diffPeaks/POLII_S5_VHL_mat_profile.pdf --refPointLabel center --plotTitle POLII_S5_VHL', returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_rep1/MV411_IRF2BP2_VHL_POLII_S5_R1.bw  --referencePoint center --regionsFileName ../../data/peaks1/mp792-MV411_IRF2BP2_VHL_MYC-MYC-r1.narrowPeak  --missingDataAsZero --outFileName ../../data/diffBinding_rep1/diffPeaks/POLII_S5_VHL_mat_profile_clust3.gz --upstream 3000 -

### on unscalled data

In [62]:
bams = !ls ../../data/diffBinding_rep1/mp*.bam
bams

['../../data/diffBinding_rep1/mp779-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_rep1/mp780-MV411_IRF2BP2_DMSO_MED1-MED1-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_rep1/mp781-MV411_IRF2BP2_DMSO_POLII_total-POLII_total-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_rep1/mp782-MV411_IRF2BP2_DMSO_POLII_S2-POLII_S2-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_rep1/mp783-MV411_IRF2BP2_DMSO_POLII_S5-POLII_S5-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_rep1/mp784-MV411_IRF2BP2_DMSO_MYC-MYC-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_rep1/mp785-MV411_IRF2BP2_DMSO_MYB-MYB-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_rep1/mp786-MV411_IRF2BP2_DMSO_SPI1-SPI1-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_rep1/mp787-MV411_IRF2BP2_VHL_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_rep1/mp788-MV411_IRF2BP2_VHL_MED1-MED1-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_rep1/mp789-MV411_IRF2BP2_VHL_

In [3]:
! mkdir ../../data/diffBinding_rep1/diffPeaks_unscaled

/bin/sh: 1: mkdir../../data/diffBinding_rep1/diffPeaks_unscaled: not found


In [31]:
bams= list(bams)

In [33]:
bams

['../../data/diffBinding_rep1/INPUT_R1.mLb.clN.sorted.bam',
 '../../data/diffBinding_rep1/mp779-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_rep1/mp780-MV411_IRF2BP2_DMSO_MED1-MED1-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_rep1/mp781-MV411_IRF2BP2_DMSO_POLII_total-POLII_total-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_rep1/mp782-MV411_IRF2BP2_DMSO_POLII_S2-POLII_S2-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_rep1/mp783-MV411_IRF2BP2_DMSO_POLII_S5-POLII_S5-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_rep1/mp784-MV411_IRF2BP2_DMSO_MYC-MYC-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_rep1/mp785-MV411_IRF2BP2_DMSO_MYB-MYB-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_rep1/mp786-MV411_IRF2BP2_DMSO_SPI1-SPI1-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_rep1/mp787-MV411_IRF2BP2_VHL_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_rep1/mp788-MV411_IRF2BP2_VHL_MED1-MED1-r1.mLb.clN.sorted.

In [36]:
size

[220, 191, 211, 204, 285, 222, 228, 194]

In [38]:
# on unscalled data 
for i in range(int((len(bams)-1)/2)-1):
    if i < 0:
        continue
    name1 = bams[1+i]
    name2 = bams[9+i]
    chip.fullDiffPeak(name1,name2, control1=bams[0], size=size[i], compute_size=False, directory = "../../data/diffBinding_rep1/diffData_unscaled/", res_directory = "../../data/diffBinding_rep1/diffPeaks_unscaled/",pairedend=False)

doing diff from ../../data/diffBinding_rep1/mp779-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.mLb.clN.sorted.bam and ../../data/diffBinding_rep1/mp787-MV411_IRF2BP2_VHL_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.mLb.clN.sorted.bam
using default size
computing the scaling values
b'INFO  @ Mon, 20 Apr 2020 16:15:53: \n# Command line: callpeak -B -t ../../data/diffBinding_rep1/mp779-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.mLb.clN.sorted.bam -c ../../data/diffBinding_rep1/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 220 -n mp779-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r1 --outdir ../../data/diffBinding_rep1/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = mp779-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r1\n# format = BAM\n# ChIP-seq file = [\'../../data/diffBinding_rep1/mp779-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffBinding_rep1/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 

doing diff from ../../data/diffBinding_rep1/mp780-MV411_IRF2BP2_DMSO_MED1-MED1-r1.mLb.clN.sorted.bam and ../../data/diffBinding_rep1/mp788-MV411_IRF2BP2_VHL_MED1-MED1-r1.mLb.clN.sorted.bam
using default size
computing the scaling values
b'INFO  @ Mon, 20 Apr 2020 16:40:46: \n# Command line: callpeak -B -t ../../data/diffBinding_rep1/mp780-MV411_IRF2BP2_DMSO_MED1-MED1-r1.mLb.clN.sorted.bam -c ../../data/diffBinding_rep1/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 191 -n mp780-MV411_IRF2BP2_DMSO_MED1-MED1-r1 --outdir ../../data/diffBinding_rep1/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = mp780-MV411_IRF2BP2_DMSO_MED1-MED1-r1\n# format = BAM\n# ChIP-seq file = [\'../../data/diffBinding_rep1/mp780-MV411_IRF2BP2_DMSO_MED1-MED1-r1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffBinding_rep1/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant s

doing diff from ../../data/diffBinding_rep1/mp781-MV411_IRF2BP2_DMSO_POLII_total-POLII_total-r1.mLb.clN.sorted.bam and ../../data/diffBinding_rep1/mp789-MV411_IRF2BP2_VHL_POLII_total-POLII_total-r1.mLb.clN.sorted.bam
using default size
computing the scaling values
b'INFO  @ Mon, 20 Apr 2020 17:02:43: \n# Command line: callpeak -B -t ../../data/diffBinding_rep1/mp781-MV411_IRF2BP2_DMSO_POLII_total-POLII_total-r1.mLb.clN.sorted.bam -c ../../data/diffBinding_rep1/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 211 -n mp781-MV411_IRF2BP2_DMSO_POLII_total-POLII_total-r1 --outdir ../../data/diffBinding_rep1/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = mp781-MV411_IRF2BP2_DMSO_POLII_total-POLII_total-r1\n# format = BAM\n# ChIP-seq file = [\'../../data/diffBinding_rep1/mp781-MV411_IRF2BP2_DMSO_POLII_total-POLII_total-r1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffBinding_rep1/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model

doing diff from ../../data/diffBinding_rep1/mp782-MV411_IRF2BP2_DMSO_POLII_S2-POLII_S2-r1.mLb.clN.sorted.bam and ../../data/diffBinding_rep1/mp790-MV411_IRF2BP2_VHL_POLII_S2-POLII_S2-r1.mLb.clN.sorted.bam
using default size
computing the scaling values
b'INFO  @ Mon, 20 Apr 2020 17:27:57: \n# Command line: callpeak -B -t ../../data/diffBinding_rep1/mp782-MV411_IRF2BP2_DMSO_POLII_S2-POLII_S2-r1.mLb.clN.sorted.bam -c ../../data/diffBinding_rep1/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 204 -n mp782-MV411_IRF2BP2_DMSO_POLII_S2-POLII_S2-r1 --outdir ../../data/diffBinding_rep1/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = mp782-MV411_IRF2BP2_DMSO_POLII_S2-POLII_S2-r1\n# format = BAM\n# ChIP-seq file = [\'../../data/diffBinding_rep1/mp782-MV411_IRF2BP2_DMSO_POLII_S2-POLII_S2-r1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffBinding_rep1/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5

doing diff from ../../data/diffBinding_rep1/mp783-MV411_IRF2BP2_DMSO_POLII_S5-POLII_S5-r1.mLb.clN.sorted.bam and ../../data/diffBinding_rep1/mp791-MV411_IRF2BP2_VHL_POLII_S5-POLII_S5-r1.mLb.clN.sorted.bam
using default size
computing the scaling values
b'INFO  @ Mon, 20 Apr 2020 17:48:53: \n# Command line: callpeak -B -t ../../data/diffBinding_rep1/mp783-MV411_IRF2BP2_DMSO_POLII_S5-POLII_S5-r1.mLb.clN.sorted.bam -c ../../data/diffBinding_rep1/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 285 -n mp783-MV411_IRF2BP2_DMSO_POLII_S5-POLII_S5-r1 --outdir ../../data/diffBinding_rep1/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = mp783-MV411_IRF2BP2_DMSO_POLII_S5-POLII_S5-r1\n# format = BAM\n# ChIP-seq file = [\'../../data/diffBinding_rep1/mp783-MV411_IRF2BP2_DMSO_POLII_S5-POLII_S5-r1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffBinding_rep1/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5

doing diff from ../../data/diffBinding_rep1/mp784-MV411_IRF2BP2_DMSO_MYC-MYC-r1.mLb.clN.sorted.bam and ../../data/diffBinding_rep1/mp792-MV411_IRF2BP2_VHL_MYC-MYC-r1.mLb.clN.sorted.bam
using default size
computing the scaling values
b'INFO  @ Mon, 20 Apr 2020 18:10:18: \n# Command line: callpeak -B -t ../../data/diffBinding_rep1/mp784-MV411_IRF2BP2_DMSO_MYC-MYC-r1.mLb.clN.sorted.bam -c ../../data/diffBinding_rep1/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 222 -n mp784-MV411_IRF2BP2_DMSO_MYC-MYC-r1 --outdir ../../data/diffBinding_rep1/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = mp784-MV411_IRF2BP2_DMSO_MYC-MYC-r1\n# format = BAM\n# ChIP-seq file = [\'../../data/diffBinding_rep1/mp784-MV411_IRF2BP2_DMSO_MYC-MYC-r1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffBinding_rep1/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assi

doing diff from ../../data/diffBinding_rep1/mp785-MV411_IRF2BP2_DMSO_MYB-MYB-r1.mLb.clN.sorted.bam and ../../data/diffBinding_rep1/mp793-MV411_IRF2BP2_VHL_MYB-MYB-r1.mLb.clN.sorted.bam
using default size
computing the scaling values
b'INFO  @ Mon, 20 Apr 2020 18:32:36: \n# Command line: callpeak -B -t ../../data/diffBinding_rep1/mp785-MV411_IRF2BP2_DMSO_MYB-MYB-r1.mLb.clN.sorted.bam -c ../../data/diffBinding_rep1/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 228 -n mp785-MV411_IRF2BP2_DMSO_MYB-MYB-r1 --outdir ../../data/diffBinding_rep1/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = mp785-MV411_IRF2BP2_DMSO_MYB-MYB-r1\n# format = BAM\n# ChIP-seq file = [\'../../data/diffBinding_rep1/mp785-MV411_IRF2BP2_DMSO_MYB-MYB-r1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffBinding_rep1/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assi

doing diff from ../../data/diffBinding_rep1/mp786-MV411_IRF2BP2_DMSO_SPI1-SPI1-r1.mLb.clN.sorted.bam and ../../data/diffBinding_rep1/mp794-MV411_IRF2BP2_VHL_SPI1-SPI1-r1.mLb.clN.sorted.bam
using default size
computing the scaling values
b'INFO  @ Mon, 20 Apr 2020 18:55:42: \n# Command line: callpeak -B -t ../../data/diffBinding_rep1/mp786-MV411_IRF2BP2_DMSO_SPI1-SPI1-r1.mLb.clN.sorted.bam -c ../../data/diffBinding_rep1/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 194 -n mp786-MV411_IRF2BP2_DMSO_SPI1-SPI1-r1 --outdir ../../data/diffBinding_rep1/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = mp786-MV411_IRF2BP2_DMSO_SPI1-SPI1-r1\n# format = BAM\n# ChIP-seq file = [\'../../data/diffBinding_rep1/mp786-MV411_IRF2BP2_DMSO_SPI1-SPI1-r1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffBinding_rep1/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant s

In [13]:
!mkdir ../../data/bigwigrep1unscaled

In [14]:
! gsutil cp gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/bigwig/*.bigWig ../../data/bigwigrep1unscaled/

Copying gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/bigwig/INPUT_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_DMSO_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/bigwig/MV411_IRF2BP2_VHL_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/bigwig/mp779-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.mLb.clN.bigWig...
\ [4 files][  3.3 GiB/  3.3 GiB]  206.0 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://amlproject/Chip/IRF2BP2_degraded_rep1/bwa/mergedLibrary/bigwig/mp780-MV411_IRF2BP2_DMSO_MED1-MED1-r1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep1/bw

In [102]:
bw = ! ls ../../data/bigwigrep1unscaled/
bw

['INPUT_R1.mLb.clN.bigWig',
 'mp779-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.mLb.clN.bigWig',
 'mp780-MV411_IRF2BP2_DMSO_MED1-MED1-r1.mLb.clN.bigWig',
 'mp781-MV411_IRF2BP2_DMSO_POLII_total-POLII_total-r1.mLb.clN.bigWig',
 'mp782-MV411_IRF2BP2_DMSO_POLII_S2-POLII_S2-r1.mLb.clN.bigWig',
 'mp783-MV411_IRF2BP2_DMSO_POLII_S5-POLII_S5-r1.mLb.clN.bigWig',
 'mp784-MV411_IRF2BP2_DMSO_MYC-MYC-r1.mLb.clN.bigWig',
 'mp785-MV411_IRF2BP2_DMSO_MYB-MYB-r1.mLb.clN.bigWig',
 'mp786-MV411_IRF2BP2_DMSO_SPI1-SPI1-r1.mLb.clN.bigWig',
 'mp787-MV411_IRF2BP2_VHL_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.mLb.clN.bigWig',
 'mp788-MV411_IRF2BP2_VHL_MED1-MED1-r1.mLb.clN.bigWig',
 'mp789-MV411_IRF2BP2_VHL_POLII_total-POLII_total-r1.mLb.clN.bigWig',
 'mp790-MV411_IRF2BP2_VHL_POLII_S2-POLII_S2-r1.mLb.clN.bigWig',
 'mp791-MV411_IRF2BP2_VHL_POLII_S5-POLII_S5-r1.mLb.clN.bigWig',
 'mp792-MV411_IRF2BP2_VHL_MYC-MYC-r1.mLb.clN.bigWig',
 'mp793-MV411_IRF2BP2_VHL_MYB-MYB-r1.mLb.clN.bigWig',
 'mp794-MV411_IRF2BP2_VHL_SPI1-SPI1-r1.m

In [103]:
commonpeak = ! ls ../../data/diffBinding_rep1/diffPeaks_unscaled/*common.bed
cond1peak = ! ls ../../data/diffBinding_rep1/diffPeaks_unscaled/*cond1.bed
cond2peak = ! ls ../../data/diffBinding_rep1/diffPeaks_unscaled/*cond2.bed
cond2peak

['../../data/diffBinding_rep1/diffPeaks_unscaled/mp779-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r1_vs_mp787-MV411_IRF2BP2_VHL_FLAG_IRF2BP2-FLAG_IRF2BP2-r1_c3.0_cond2.bed',
 '../../data/diffBinding_rep1/diffPeaks_unscaled/mp780-MV411_IRF2BP2_DMSO_MED1-MED1-r1_vs_mp788-MV411_IRF2BP2_VHL_MED1-MED1-r1_c3.0_cond2.bed',
 '../../data/diffBinding_rep1/diffPeaks_unscaled/mp781-MV411_IRF2BP2_DMSO_POLII_total-POLII_total-r1_vs_mp789-MV411_IRF2BP2_VHL_POLII_total-POLII_total-r1_c3.0_cond2.bed',
 '../../data/diffBinding_rep1/diffPeaks_unscaled/mp782-MV411_IRF2BP2_DMSO_POLII_S2-POLII_S2-r1_vs_mp790-MV411_IRF2BP2_VHL_POLII_S2-POLII_S2-r1_c3.0_cond2.bed',
 '../../data/diffBinding_rep1/diffPeaks_unscaled/mp783-MV411_IRF2BP2_DMSO_POLII_S5-POLII_S5-r1_vs_mp791-MV411_IRF2BP2_VHL_POLII_S5-POLII_S5-r1_c3.0_cond2.bed',
 '../../data/diffBinding_rep1/diffPeaks_unscaled/mp784-MV411_IRF2BP2_DMSO_MYC-MYC-r1_vs_mp792-MV411_IRF2BP2_VHL_MYC-MYC-r1_c3.0_cond2.bed',
 '../../data/diffBinding_rep1/diffPeaks_unscaled

In [104]:
names = ["FLAG_IRF2BP2","MED1","POLII_total","POLII_S2","POLII_S5","MYC","MYB","SPI1"]

In [106]:
for i in range(int(len(bw[1:])/2)-1):
    name1 = bw[1+i]
    name2 = bw[9+i]
    peak = [cond1peak[i], commonpeak[i], cond2peak[i]]
    for val in peak:
        chip.dropWeirdChromosomes(val)
    name = names[i]
    chip.computePeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], window=3000, title=name, numthreads=7, refpoint='center', folder="../../data/bigwigrep1unscaled/", torecompute=True, name='../../data/diffBinding_rep1/diffPeaks_unscaled/'+name+'_mat.pdf', withDeeptools=True)

found 0 header line... removing
found 0 header line... removing
found 0 header line... removing
CompletedProcess(args='computeMatrix reference-point -S ../../data/bigwigrep1unscaled/mp779-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.mLb.clN.bigWig ../../data/bigwigrep1unscaled/mp787-MV411_IRF2BP2_VHL_FLAG_IRF2BP2-FLAG_IRF2BP2-r1.mLb.clN.bigWig  --referencePoint center --regionsFileName ../../data/diffBinding_rep1/diffPeaks_unscaled/mp779-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r1_vs_mp787-MV411_IRF2BP2_VHL_FLAG_IRF2BP2-FLAG_IRF2BP2-r1_c3.0_cond1.bed ../../data/diffBinding_rep1/diffPeaks_unscaled/mp779-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r1_vs_mp787-MV411_IRF2BP2_VHL_FLAG_IRF2BP2-FLAG_IRF2BP2-r1_c3.0_common.bed ../../data/diffBinding_rep1/diffPeaks_unscaled/mp779-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r1_vs_mp787-MV411_IRF2BP2_VHL_FLAG_IRF2BP2-FLAG_IRF2BP2-r1_c3.0_cond2.bed  --missingDataAsZero --outFileName ../../data/diffBinding_rep1/diffPeaks_unscaled/FLAG_IRF2

found 0 header line... removing
found 0 header line... removing
CompletedProcess(args='computeMatrix reference-point -S ../../data/bigwigrep1unscaled/mp784-MV411_IRF2BP2_DMSO_MYC-MYC-r1.mLb.clN.bigWig ../../data/bigwigrep1unscaled/mp792-MV411_IRF2BP2_VHL_MYC-MYC-r1.mLb.clN.bigWig  --referencePoint center --regionsFileName ../../data/diffBinding_rep1/diffPeaks_unscaled/mp784-MV411_IRF2BP2_DMSO_MYC-MYC-r1_vs_mp792-MV411_IRF2BP2_VHL_MYC-MYC-r1_c3.0_cond1.bed ../../data/diffBinding_rep1/diffPeaks_unscaled/mp784-MV411_IRF2BP2_DMSO_MYC-MYC-r1_vs_mp792-MV411_IRF2BP2_VHL_MYC-MYC-r1_c3.0_common.bed ../../data/diffBinding_rep1/diffPeaks_unscaled/mp784-MV411_IRF2BP2_DMSO_MYC-MYC-r1_vs_mp792-MV411_IRF2BP2_VHL_MYC-MYC-r1_c3.0_cond2.bed  --missingDataAsZero --outFileName ../../data/diffBinding_rep1/diffPeaks_unscaled/MYC_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotHeatmap --matrixFile ../../data/diffBinding_rep1/diffPeaks_unscaled/MYC_mat.gz --outFileName ../../data/diffBi

In [ ]:
! gsutil -m cp -r "diffPeaks" gs://amlproject/Chip/IRF2BP2/ &&\
    gsutil -m cp -r "diffData" gs://amlproject/Chip/IRF2BP2/ &&\
        gsutil -m cp -r "bigwigs" gs://amlproject/Chip/IRF2BP2/

## v2

In [ ]:
#! rm -r bams && mkdir bams && gsutil -m cp "gs://transfer-amlproject/20191219_MP7659*" bams/

### analysis

In [ ]:
rename = {
"20191219_MP7659_1_S1_R1_001.fastq.gz":"mp795_R1.fastq.gz",
"20191219_MP7659_1_S1_R2_001.fastq.gz":"mp795_R2.fastq.gz",
"20191219_MP7659_2_S2_R1_001.fastq.gz":"mp796_R1.fastq.gz",
"20191219_MP7659_2_S2_R2_001.fastq.gz":"mp796_R2.fastq.gz",
"20191219_MP7659_3_S3_R1_001.fastq.gz":"mp797_R1.fastq.gz",
"20191219_MP7659_3_S3_R2_001.fastq.gz":"mp797_R2.fastq.gz",
"20191219_MP7659_4_S4_R1_001.fastq.gz":"mp798_R1.fastq.gz",
"20191219_MP7659_4_S4_R2_001.fastq.gz":"mp798_R2.fastq.gz",
"20191219_MP7659_5_S5_R1_001.fastq.gz":"mp799_R1.fastq.gz",
"20191219_MP7659_5_S5_R2_001.fastq.gz":"mp799_R2.fastq.gz",
"20191219_MP7659_6_S6_R1_001.fastq.gz":"mp800_R1.fastq.gz",
"20191219_MP7659_6_S6_R2_001.fastq.gz":"mp800_R2.fastq.gz",
"20191219_MP7659_7_S7_R1_001.fastq.gz":"mp801_R1.fastq.gz",
"20191219_MP7659_7_S7_R2_001.fastq.gz":"mp801_R2.fastq.gz",
"20191219_MP7659_8_S8_R1_001.fastq.gz":"mp802_R1.fastq.gz",
"20191219_MP7659_8_S8_R2_001.fastq.gz":"mp802_R2.fastq.gz",
"20191219_MP7659_9_S9_R1_001.fastq.gz":"mp803_R1.fastq.gz",
"20191219_MP7659_9_S9_R2_001.fastq.gz":"mp803_R2.fastq.gz",
"20191219_MP7659_10_S10_R1_001.fastq.gz":"mp804_R1.fastq.gz",
"20191219_MP7659_10_S10_R2_001.fastq.gz":"mp804_R2.fastq.gz",
"20191219_MP7659_11_S11_R1_001.fastq.gz":"mp805_R1.fastq.gz",
"20191219_MP7659_11_S11_R2_001.fastq.gz":"mp805_R2.fastq.gz",
"20191219_MP7659_12_S12_R1_001.fastq.gz":"mp806_R1.fastq.gz",
"20191219_MP7659_12_S12_R2_001.fastq.gz":"mp806_R2.fastq.gz",
"20191219_MP7659_13_S13_R1_001.fastq.gz":"mp807_R1.fastq.gz",
"20191219_MP7659_13_S13_R2_001.fastq.gz":"mp807_R2.fastq.gz",
"20191219_MP7659_14_S14_R1_001.fastq.gz":"mp808_R1.fastq.gz",
"20191219_MP7659_14_S14_R2_001.fastq.gz":"mp808_R2.fastq.gz",
"20191219_MP7659_15_S15_R1_001.fastq.gz":"mp809_R1.fastq.gz",
"20191219_MP7659_15_S15_R2_001.fastq.gz":"mp809_R2.fastq.gz",
"20191219_MP7659_16_S16_R1_001.fastq.gz":"mp810_R1.fastq.gz",
"20191219_MP7659_16_S16_R2_001.fastq.gz":"mp810_R2.fastq.gz"
}

In [ ]:
for k,v in rename.items():
    ! mv $k $v
! gsutil -m cp 'bams/*' gs://amlproject/Chip_AML/IRF2BP2/results_rep2/    

In [ ]:
! gsutil ls gs://amlproject/Chip_AML/IRF2BP2/

In [ ]:
inputfastq="gs://amlproject/Chip_AML/Chip/mp99.fastq.gz"
! gsutil cp $inputfastq bams
a = ! ls bams

In [ ]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
url="https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U"
gsheet = sheets.get(url).sheets[2].to_frame()

In [ ]:
df = {
"fastq1": [],
"fastq2": [],
"antibody": [],
"group": [],
"replicate": [],
"control": []
}
for val in helper.grouped(a[:-1],2):
    row = gsheet[gsheet.id==val[0].split('/')[-1].split('_')[0]]
    df['group'].append((row.id.values[0] + '_'+ row.name_replicate.values[0] +"_"+row.protein.values[0]))
    df['replicate'].append(1)
    df['fastq1'].append(val[0])
    #df['fastq2'].append(val[1])
    df['antibody'].append(row.protein.values[0])
    df['control'].append("INPUT")
df['group'].append('INPUT')
df['replicate'].append(1)
df['fastq1'].append(a[-1])
#df['fastq2'].append(val[1])
df['antibody'].append("INPUT")
df['control'].append("INPUT")
df = pd.DataFrame(df)

In [ ]:
df[df.columns[[3,4,0,1,2,5]]].to_csv('design.csv')

In [ ]:
#process chips
! sudo ./nextflow run nf-core/chipseq --single_end --seq_center 'DFCI' --email 'jkobject@gmail.com' --narrow_peak --input design.csv --genome GRCh38 --skip_preseq --max_cpus 24 -profile docker -w work

In [ ]:
#!gsutil cp results gs://amlproject/Chip_AML/IRF2BP2/ && sudo rm -r work && mv results results2

In [ ]:
# get scaling values
mappedreads, umappedreads_norm, mapped = chip.getSpikeInControlScales(refgenome="ref/reference_droso.fna",FastQfolder='bams',pairedEnd=True, cores=8)
mappedreads, umappedreads_norm, mapped

computing scales from the excel sheet

In [ ]:
scales = [[508878,637972],
[1929129,11595],
[669536,429562],
[1272730,810802],
[743859,893304],
[312888,1154119],
[1086031,880901],
[850181,1019640]]

In [51]:
scales = [[1.0, 0.7976494266],
[0.006010484524, 1.0],
[0.6415816327, 1.0],
[0.6370573492, 1.0],
[1.0, 0.8327053276],
[1.0, 0.2711054926],
[0.8111195721, 1.0],
[1.0, 0.8338050685]]

### on scalled data

In [52]:
# on scaled data
bams = ! ls results2/bwa/mergedLibrary/*.bam
for i in range(int(len(bams[1:])/2)-1):
    name1 = bams[1+val]
    name2 = bams[9+val]
    chip.fullDiffPeak(name1,name2,control1='../../data/results3/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam', directory = "../../data/diffBinding_rep2/diffData_unscaled/", res_directory = "../../data/diffBinding_rep2/diffPeaks_unscaled/", scaling = scales[val][::-1])

In [ ]:
bams = ! ls results2/bwa/mergedLibrary/*.bam
bams = [i.split('/')[-1].split('.')[0] for i in bams]
bams

In [ ]:
# diffPeak on scalled data
sizes = [220, 191, 228, 285, 222, 204, 211, 194]
for i in range(int(len(bams[1:])/2)-1):
    name1 = bams[1+i]
    name2 = bams[9+i]
    chip.diffPeak(name1,name2, directory= "../../data/diffBinding_rep1/diffData/", res_directory='../../data/diffBinding_rep1/diffPeaks/', scaling1=scales[i][1], scaling2=scales[i][0], size=size[i])

In [ ]:
scales = [1.0,
0.006010484524,
0.6415816327,
0.6370573492,
1.0,
1.0,
0.8111195721,
1.0,
0.7976494266,
1.0,
1.0,
1.0,
0.8327053276,
0.2711054926,
1.0,
0.8338050685]

In [ ]:
chip.bigWigFrom(bams[1:],genome='GRCh38',scaling=scales)

In [107]:
bw = ! ls ../../data/recalib_bigwig_rep2/*.bw
bw

['../../data/recalib_bigwig_rep2/mp795_R1.bw',
 '../../data/recalib_bigwig_rep2/mp796_R1.bw',
 '../../data/recalib_bigwig_rep2/mp797_R1.bw',
 '../../data/recalib_bigwig_rep2/mp798_R1.bw',
 '../../data/recalib_bigwig_rep2/mp799_R1.bw',
 '../../data/recalib_bigwig_rep2/mp800_R1.bw',
 '../../data/recalib_bigwig_rep2/mp801_R1.bw',
 '../../data/recalib_bigwig_rep2/mp802_R1.bw',
 '../../data/recalib_bigwig_rep2/mp803_R1.bw',
 '../../data/recalib_bigwig_rep2/mp804_R1.bw',
 '../../data/recalib_bigwig_rep2/mp805_R1.bw',
 '../../data/recalib_bigwig_rep2/mp806_R1.bw',
 '../../data/recalib_bigwig_rep2/mp807_R1.bw',
 '../../data/recalib_bigwig_rep2/mp808_R1.bw',
 '../../data/recalib_bigwig_rep2/mp809_R1.bw',
 '../../data/recalib_bigwig_rep2/mp810_R1.bw']

In [108]:
cond1peak = ! ls ../../data/diffBinding_rep2/diffPeaks/*cond1.bed
cond2peak = ! ls ../../data/diffBinding_rep2/diffPeaks/*cond2.bed
commonpeak = ! ls ../../data/diffBinding_rep2/diffPeaks/*common.bed
commonpeak

['../../data/diffBinding_rep2/diffPeaks/mp795_R1_vs_mp803_R1_c3.0_common.bed',
 '../../data/diffBinding_rep2/diffPeaks/mp796_R1_vs_mp804_R1_c3.0_common.bed',
 '../../data/diffBinding_rep2/diffPeaks/mp797_R1_vs_mp805_R1_c3.0_common.bed',
 '../../data/diffBinding_rep2/diffPeaks/mp798_R1_vs_mp806_R1_c3.0_common.bed',
 '../../data/diffBinding_rep2/diffPeaks/mp799_R1_vs_mp807_R1_c3.0_common.bed',
 '../../data/diffBinding_rep2/diffPeaks/mp800_R1_vs_mp808_R1_c3.0_common.bed',
 '../../data/diffBinding_rep2/diffPeaks/mp801_R1_vs_mp809_R1_c3.0_common.bed',
 '../../data/diffBinding_rep2/diffPeaks/mp802_R1_vs_mp810_R1_c3.0_common.bed']

In [109]:
names = ["FLAG_IRF2BP2","MED1","POLII_total","POLII_S2","POLII_S5","MYC","MYB","SPI1"]

In [110]:
for i in range(int(len(bw)/2)):
    name1 = bw[i]
    name2 = bw[8+i]
    peak = [cond1peak[i], commonpeak[i], cond2peak[i]]
    for val in peak:
        chip.dropWeirdChromosomes(val)
    name = names[i]
    chip.computePeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], window=3000, folder="", title=name, numthreads=20, name='../../data/diffBinding_rep2/diffPeaks/'+name+'_mat.pdf', refpoint='center', withDeeptools=True, torecompute= True,)

found 0 header line... removing
found 0 header line... removing
found 0 header line... removing
CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_rep2/mp795_R1.bw ../../data/recalib_bigwig_rep2/mp803_R1.bw  --referencePoint center --regionsFileName ../../data/diffBinding_rep2/diffPeaks/mp795_R1_vs_mp803_R1_c3.0_cond1.bed ../../data/diffBinding_rep2/diffPeaks/mp795_R1_vs_mp803_R1_c3.0_common.bed ../../data/diffBinding_rep2/diffPeaks/mp795_R1_vs_mp803_R1_c3.0_cond2.bed  --missingDataAsZero --outFileName ../../data/diffBinding_rep2/diffPeaks/FLAG_IRF2BP2_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 20 && plotHeatmap --matrixFile ../../data/diffBinding_rep2/diffPeaks/FLAG_IRF2BP2_mat.gz --outFileName ../../data/diffBinding_rep2/diffPeaks/FLAG_IRF2BP2_mat.pdf --refPointLabel center --regionsLabel DMSO_peaks common VHL_peaks  --samplesLabel DMSO VHL  --plotTitle FLAG_IRF2BP2', returncode=0, stdout=b'', stderr=b'')
empty bed
empty bed
found 0 h

In [119]:
mkdir ../../data/peaks2

In [120]:
! gsutil cp gs://amlproject/Chip/IRF2BP2_degraded_rep2/bwa/mergedLibrary/macs/narrowPeak/*.narrowPeak ../../data/peaks2

Copying gs://amlproject/Chip/IRF2BP2_degraded_rep2/bwa/mergedLibrary/macs/narrowPeak/mp795-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r2.narrowPeak...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep2/bwa/mergedLibrary/macs/narrowPeak/mp796-MV411_IRF2BP2_DMSO_MED1-MED1-r2.narrowPeak...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep2/bwa/mergedLibrary/macs/narrowPeak/mp797-MV411_IRF2BP2_DMSO_POLII_total-POLII_total-r2.narrowPeak...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep2/bwa/mergedLibrary/macs/narrowPeak/mp798-MV411_IRF2BP2_DMSO_POLII_S2-POLII_S2-r2.narrowPeak...
- [4 files][  7.3 MiB/  7.3 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://amlproject/Chip/IRF2BP2_degraded_rep2/bwa/mergedLibrary/macs/narrowP

In [119]:
peaks = ! ls ../../data/peaks2/*
peaks

['../../data/peaks2/mp795-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r2.narrowPeak',
 '../../data/peaks2/mp796-MV411_IRF2BP2_DMSO_MED1-MED1-r2.narrowPeak',
 '../../data/peaks2/mp797-MV411_IRF2BP2_DMSO_POLII_total-POLII_total-r2.narrowPeak',
 '../../data/peaks2/mp798-MV411_IRF2BP2_DMSO_POLII_S2-POLII_S2-r2.narrowPeak',
 '../../data/peaks2/mp799-MV411_IRF2BP2_DMSO_POLII_S5-POLII_S5-r2.narrowPeak',
 '../../data/peaks2/mp800-MV411_IRF2BP2_DMSO_MYC-MYC-r2.narrowPeak',
 '../../data/peaks2/mp801-MV411_IRF2BP2_DMSO_MYB-MYB-r2.narrowPeak',
 '../../data/peaks2/mp802-MV411_IRF2BP2_DMSO_SPI1-SPI1-r2.narrowPeak',
 '../../data/peaks2/mp803-MV411_IRF2BP2_VHL_FLAG_IRF2BP2-FLAG_IRF2BP2-r2.narrowPeak',
 '../../data/peaks2/mp804-MV411_IRF2BP2_VHL_MED1-MED1-r2.narrowPeak',
 '../../data/peaks2/mp805-MV411_IRF2BP2_VHL_POLII_total-POLII_total-r2.narrowPeak',
 '../../data/peaks2/mp806-MV411_IRF2BP2_VHL_POLII_S2-POLII_S2-r2.narrowPeak',
 '../../data/peaks2/mp807-MV411_IRF2BP2_VHL_POLII_S5-POLII_S5-r2.narrowP

In [120]:
names = ["FLAG_IRF2BP2","MED1","POLII_total","POLII_S2","POLII_S5","MYC","MYB","SPI1"]
names.extend([i+'_VHL' for i in names])
names

['FLAG_IRF2BP2',
 'MED1',
 'POLII_total',
 'POLII_S2',
 'POLII_S5',
 'MYC',
 'MYB',
 'SPI1',
 'FLAG_IRF2BP2_VHL',
 'MED1_VHL',
 'POLII_total_VHL',
 'POLII_S2_VHL',
 'POLII_S5_VHL',
 'MYC_VHL',
 'MYB_VHL',
 'SPI1_VHL']

In [128]:
bw

['../../data/bigwigrep2unscaled/INPUT_R1.mLb.clN.bigWig',
 '../../data/bigwigrep2unscaled/mp795-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r2.mLb.clN.bigWig',
 '../../data/bigwigrep2unscaled/mp796-MV411_IRF2BP2_DMSO_MED1-MED1-r2.mLb.clN.bigWig',
 '../../data/bigwigrep2unscaled/mp797-MV411_IRF2BP2_DMSO_POLII_total-POLII_total-r2.mLb.clN.bigWig',
 '../../data/bigwigrep2unscaled/mp798-MV411_IRF2BP2_DMSO_POLII_S2-POLII_S2-r2.mLb.clN.bigWig',
 '../../data/bigwigrep2unscaled/mp799-MV411_IRF2BP2_DMSO_POLII_S5-POLII_S5-r2.mLb.clN.bigWig',
 '../../data/bigwigrep2unscaled/mp800-MV411_IRF2BP2_DMSO_MYC-MYC-r2.mLb.clN.bigWig',
 '../../data/bigwigrep2unscaled/mp801-MV411_IRF2BP2_DMSO_MYB-MYB-r2.mLb.clN.bigWig',
 '../../data/bigwigrep2unscaled/mp802-MV411_IRF2BP2_DMSO_SPI1-SPI1-r2.mLb.clN.bigWig',
 '../../data/bigwigrep2unscaled/mp803-MV411_IRF2BP2_VHL_FLAG_IRF2BP2-FLAG_IRF2BP2-r2.mLb.clN.bigWig',
 '../../data/bigwigrep2unscaled/mp804-MV411_IRF2BP2_VHL_MED1-MED1-r2.mLb.clN.bigWig',
 '../../data/big

In [130]:
for i in range(len(bw)-1):
    chip.computePeaksAt(peaks[i], bw[1+i], window=3000, folder="", title=names[i], numthreads=7, torecompute=True, onlyProfile=True, name='../../data/diffBinding_rep2/diffPeaks/'+names[i]+'_mat_profile.pdf', refpoint='center', withDeeptools=True)
    chip.computePeaksAt(peaks[i], bw[1+i], window=3000, folder="", title=names[i], numthreads=7, torecompute=True, refpoint='center', onlyProfile=True, name='../../data/diffBinding_rep2/diffPeaks/'+names[i]+'_mat_profile_clust3.pdf', withDeeptools=True, cluster=3)

CompletedProcess(args='computeMatrix reference-point -S ../../data/bigwigrep2unscaled/mp795-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r2.mLb.clN.bigWig  --referencePoint center --regionsFileName ../../data/peaks2/mp795-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r2.narrowPeak  --missingDataAsZero --outFileName ../../data/diffBinding_rep2/diffPeaks/FLAG_IRF2BP2_mat_profile.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotProfile --matrixFile ../../data/diffBinding_rep2/diffPeaks/FLAG_IRF2BP2_mat_profile.gz --outFileName ../../data/diffBinding_rep2/diffPeaks/FLAG_IRF2BP2_mat_profile.pdf --refPointLabel center --plotTitle FLAG_IRF2BP2', returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args='computeMatrix reference-point -S ../../data/bigwigrep2unscaled/mp795-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r2.mLb.clN.bigWig  --referencePoint center --regionsFileName ../../data/peaks2/mp795-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r2.narrowPeak  --missingDataAsZ

CompletedProcess(args='computeMatrix reference-point -S ../../data/bigwigrep2unscaled/mp801-MV411_IRF2BP2_DMSO_MYB-MYB-r2.mLb.clN.bigWig  --referencePoint center --regionsFileName ../../data/peaks2/mp801-MV411_IRF2BP2_DMSO_MYB-MYB-r2.narrowPeak  --missingDataAsZero --outFileName ../../data/diffBinding_rep2/diffPeaks/MYB_mat_profile.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotProfile --matrixFile ../../data/diffBinding_rep2/diffPeaks/MYB_mat_profile.gz --outFileName ../../data/diffBinding_rep2/diffPeaks/MYB_mat_profile.pdf --refPointLabel center --plotTitle MYB', returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args='computeMatrix reference-point -S ../../data/bigwigrep2unscaled/mp801-MV411_IRF2BP2_DMSO_MYB-MYB-r2.mLb.clN.bigWig  --referencePoint center --regionsFileName ../../data/peaks2/mp801-MV411_IRF2BP2_DMSO_MYB-MYB-r2.narrowPeak  --missingDataAsZero --outFileName ../../data/diffBinding_rep2/diffPeaks/MYB_mat_profile_clust3.gz --upstream 3000 --downstre

CompletedProcess(args='computeMatrix reference-point -S ../../data/bigwigrep2unscaled/mp807-MV411_IRF2BP2_VHL_POLII_S5-POLII_S5-r2.mLb.clN.bigWig  --referencePoint center --regionsFileName ../../data/peaks2/mp807-MV411_IRF2BP2_VHL_POLII_S5-POLII_S5-r2.narrowPeak  --missingDataAsZero --outFileName ../../data/diffBinding_rep2/diffPeaks/POLII_S5_VHL_mat_profile.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotProfile --matrixFile ../../data/diffBinding_rep2/diffPeaks/POLII_S5_VHL_mat_profile.gz --outFileName ../../data/diffBinding_rep2/diffPeaks/POLII_S5_VHL_mat_profile.pdf --refPointLabel center --plotTitle POLII_S5_VHL', returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args='computeMatrix reference-point -S ../../data/bigwigrep2unscaled/mp807-MV411_IRF2BP2_VHL_POLII_S5-POLII_S5-r2.mLb.clN.bigWig  --referencePoint center --regionsFileName ../../data/peaks2/mp807-MV411_IRF2BP2_VHL_POLII_S5-POLII_S5-r2.narrowPeak  --missingDataAsZero --outFileName ../../data/diffBin

### unscalled

In [73]:
bams = !ls ../../data/diffBinding_rep2/mp*.bam
bams

["ls: cannot access '../../data/diffBinding_rep2/mp*.bam': No such file or directory"]

In [77]:
!mkdir  ../../data/diffBinding_rep2/diffPeaks_unscaled
!mkdir ../../data/bigwigrep2unscaled/
!gsutil -m cp gs://amlproject/Chip/IRF2BP2_degraded_rep2/bwa/mergedLibrary/bigwig/*.bigWig "../../data/bigwigrep2unscaled/"

/bin/sh: 1: mkdir../../data/diffBinding_rep2/diffPeaks_unscaled: not found
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep2/bwa/mergedLibrary/bigwig/INPUT_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep2/bwa/mergedLibrary/bigwig/mp795-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r2.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep2/bwa/mergedLibrary/bigwig/mp796-MV411_IRF2BP2_DMSO_MED1-MED1-r2.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep2/bwa/mergedLibrary/bigwig/mp797-MV411_IRF2BP2_DMSO_POLII_total-POLII_total-r2.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep2/bwa/mergedLibrary/bigwig/mp798-MV411_IRF2BP2_DMSO_POLII_S2-POLII_S2-r2.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep2/bwa/mergedLibrary/bigwig/mp801-MV411_IRF2BP2_DMSO_MYB-MYB-r2.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep2/bwa/mergedLibrary/bigwig/mp799-MV411_IRF2BP2_DMSO_POLII_S5-POLII_S5-r2.mLb.

In [111]:
bw = ! ls ../../data/bigwigrep2unscaled/*
bw

['../../data/bigwigrep2unscaled/INPUT_R1.mLb.clN.bigWig',
 '../../data/bigwigrep2unscaled/mp795-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r2.mLb.clN.bigWig',
 '../../data/bigwigrep2unscaled/mp796-MV411_IRF2BP2_DMSO_MED1-MED1-r2.mLb.clN.bigWig',
 '../../data/bigwigrep2unscaled/mp797-MV411_IRF2BP2_DMSO_POLII_total-POLII_total-r2.mLb.clN.bigWig',
 '../../data/bigwigrep2unscaled/mp798-MV411_IRF2BP2_DMSO_POLII_S2-POLII_S2-r2.mLb.clN.bigWig',
 '../../data/bigwigrep2unscaled/mp799-MV411_IRF2BP2_DMSO_POLII_S5-POLII_S5-r2.mLb.clN.bigWig',
 '../../data/bigwigrep2unscaled/mp800-MV411_IRF2BP2_DMSO_MYC-MYC-r2.mLb.clN.bigWig',
 '../../data/bigwigrep2unscaled/mp801-MV411_IRF2BP2_DMSO_MYB-MYB-r2.mLb.clN.bigWig',
 '../../data/bigwigrep2unscaled/mp802-MV411_IRF2BP2_DMSO_SPI1-SPI1-r2.mLb.clN.bigWig',
 '../../data/bigwigrep2unscaled/mp803-MV411_IRF2BP2_VHL_FLAG_IRF2BP2-FLAG_IRF2BP2-r2.mLb.clN.bigWig',
 '../../data/bigwigrep2unscaled/mp804-MV411_IRF2BP2_VHL_MED1-MED1-r2.mLb.clN.bigWig',
 '../../data/big

In [ ]:
# on unscalled data 
for i in range(int(len(bams)/2)):
    if i <0:
        continue
    name1 = bams[i]
    name2 = bams[8+i]
    chip.fullDiffPeak(name1,name2, control1='../../data/results3/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam', directory = "../../data/diffBinding_rep2/diffData_unscaled/", res_directory = "../../data/diffBinding_rep2/diffPeaks_unscaled/", pairedend=False)

In [112]:
commonpeak = ! ls ../../data/diffBinding_rep2/diffPeaks_unscaled/*common.bed
cond1peak = ! ls ../../data/diffBinding_rep2/diffPeaks_unscaled/*cond1.bed
cond2peak = ! ls ../../data/diffBinding_rep2/diffPeaks_unscaled/*cond2.bed
cond2peak

['../../data/diffBinding_rep2/diffPeaks_unscaled/mp795-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r2_vs_mp803-MV411_IRF2BP2_VHL_FLAG_IRF2BP2-FLAG_IRF2BP2-r2_c3.0_cond2.bed',
 '../../data/diffBinding_rep2/diffPeaks_unscaled/mp796-MV411_IRF2BP2_DMSO_MED1-MED1-r2_vs_mp804-MV411_IRF2BP2_VHL_MED1-MED1-r2_c3.0_cond2.bed',
 '../../data/diffBinding_rep2/diffPeaks_unscaled/mp797-MV411_IRF2BP2_DMSO_POLII_total-POLII_total-r2_vs_mp805-MV411_IRF2BP2_VHL_POLII_total-POLII_total-r2_c3.0_cond2.bed',
 '../../data/diffBinding_rep2/diffPeaks_unscaled/mp798-MV411_IRF2BP2_DMSO_POLII_S2-POLII_S2-r2_vs_mp806-MV411_IRF2BP2_VHL_POLII_S2-POLII_S2-r2_c3.0_cond2.bed',
 '../../data/diffBinding_rep2/diffPeaks_unscaled/mp799-MV411_IRF2BP2_DMSO_POLII_S5-POLII_S5-r2_vs_mp807-MV411_IRF2BP2_VHL_POLII_S5-POLII_S5-r2_c3.0_cond2.bed',
 '../../data/diffBinding_rep2/diffPeaks_unscaled/mp800-MV411_IRF2BP2_DMSO_MYC-MYC-r2_vs_mp808-MV411_IRF2BP2_VHL_MYC-MYC-r2_c3.0_cond2.bed',
 '../../data/diffBinding_rep2/diffPeaks_unscaled

In [113]:
names = ["FLAG_IRF2BP2","MED1","POLII_total","POLII_S2","POLII_S5","MYC","MYB","SPI1"]

In [114]:
for i in range(int(len(bw[1:])/2)):
    name1 = bw[1+i]
    name2 = bw[9+i]
    peak = [cond1peak[i], commonpeak[i], cond2peak[i]]
    for val in peak:
        chip.dropWeirdChromosomes(val)
    name = names[i]
    chip.computePeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], window=3000, title=name, numthreads=7, torecompute=True, refpoint='center', folder="", name='../../data/diffBinding_rep2/diffPeaks_unscaled/'+name+'_mat.pdf', withDeeptools=True)

found 0 header line... removing
found 1 header line... removing
removed 3 lines
found 1 header line... removing
removed 15 lines
CompletedProcess(args='computeMatrix reference-point -S ../../data/bigwigrep2unscaled/mp795-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r2.mLb.clN.bigWig ../../data/bigwigrep2unscaled/mp803-MV411_IRF2BP2_VHL_FLAG_IRF2BP2-FLAG_IRF2BP2-r2.mLb.clN.bigWig  --referencePoint center --regionsFileName ../../data/diffBinding_rep2/diffPeaks_unscaled/mp795-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r2_vs_mp803-MV411_IRF2BP2_VHL_FLAG_IRF2BP2-FLAG_IRF2BP2-r2_c3.0_cond1.bed ../../data/diffBinding_rep2/diffPeaks_unscaled/mp795-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r2_vs_mp803-MV411_IRF2BP2_VHL_FLAG_IRF2BP2-FLAG_IRF2BP2-r2_c3.0_common.bed ../../data/diffBinding_rep2/diffPeaks_unscaled/mp795-MV411_IRF2BP2_DMSO_FLAG_IRF2BP2-FLAG_IRF2BP2-r2_vs_mp803-MV411_IRF2BP2_VHL_FLAG_IRF2BP2-FLAG_IRF2BP2-r2_c3.0_cond2.bed  --missingDataAsZero --outFileName ../../data/diffBinding_

found 1 header line... removing
removed 22 lines
found 1 header line... removing
removed 3 lines
CompletedProcess(args='computeMatrix reference-point -S ../../data/bigwigrep2unscaled/mp800-MV411_IRF2BP2_DMSO_MYC-MYC-r2.mLb.clN.bigWig ../../data/bigwigrep2unscaled/mp808-MV411_IRF2BP2_VHL_MYC-MYC-r2.mLb.clN.bigWig  --referencePoint center --regionsFileName ../../data/diffBinding_rep2/diffPeaks_unscaled/mp800-MV411_IRF2BP2_DMSO_MYC-MYC-r2_vs_mp808-MV411_IRF2BP2_VHL_MYC-MYC-r2_c3.0_common.bed ../../data/diffBinding_rep2/diffPeaks_unscaled/mp800-MV411_IRF2BP2_DMSO_MYC-MYC-r2_vs_mp808-MV411_IRF2BP2_VHL_MYC-MYC-r2_c3.0_cond2.bed  --missingDataAsZero --outFileName ../../data/diffBinding_rep2/diffPeaks_unscaled/MYC_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotHeatmap --matrixFile ../../data/diffBinding_rep2/diffPeaks_unscaled/MYC_mat.gz --outFileName ../../data/diffBinding_rep2/diffPeaks_unscaled/MYC_mat.pdf --refPointLabel center --regionsLabel common VHL_peaks  --sam

In [ ]:
! gsutil -m cp -r "diffPeaks" gs://amlproject/Chip_AML/IRF2BP2/ &&\
    gsutil -m cp -r "diffData" gs://amlproject/Chip_AML/IRF2BP2/ &&\
        gsutil -m cp "bigwigs/mp*" gs://amlproject/Chip_AML/IRF2BP2/bigwigs

## histones V1&2

In [ ]:
bamfolder="histone_bams/"

In [ ]:
! mkdir histone_bams && gsutil cp "gs://transfer-amlproject/*.fastq.gz" $bamfolder

In [ ]:
! gsutil -m cp gs://transfer-amlproject/*MP7693* gs://transfer-amlproject/IRF2BP2_hist/ && gsutil -m rm gs://transfer-amlproject/*MP7693*


### analysis

In [ ]:
rename = {
    "20200103_1_MP7693_S6_R1_001.fastq.gz":"mp811_R1.fastq.gz",
    "20200103_1_MP7693_S6_R2_001.fastq.gz":"mp811_R2.fastq.gz",
    "20200103_2_MP7693_S7_R1_001.fastq.gz":"mp812_R1.fastq.gz",
    "20200103_2_MP7693_S7_R2_001.fastq.gz":"mp812_R2.fastq.gz",
    "20200103_3_MP7693_S8_R1_001.fastq.gz":"mp813_R1.fastq.gz",
    "20200103_3_MP7693_S8_R2_001.fastq.gz":"mp813_R2.fastq.gz",
    "20200103_4_MP7693_S9_R1_001.fastq.gz":"mp814_R1.fastq.gz",
    "20200103_4_MP7693_S9_R2_001.fastq.gz":"mp814_R2.fastq.gz",
    "20200103_5_MP7693_S10_R1_001.fastq.gz":"mp815_R1.fastq.gz",
    "20200103_5_MP7693_S10_R2_001.fastq.gz":"mp815_R2.fastq.gz",
    "20200103_6_MP7693_S11_R1_001.fastq.gz":"mp816_R1.fastq.gz",
    "20200103_6_MP7693_S11_R2_001.fastq.gz":"mp816_R2.fastq.gz",
    "20200103_7_MP7693_S12_R1_001.fastq.gz":"mp817_R1.fastq.gz",
    "20200103_7_MP7693_S12_R2_001.fastq.gz":"mp817_R2.fastq.gz",
    "20200103_8_MP7693_S13_R1_001.fastq.gz":"mp818_R1.fastq.gz",
    "20200103_8_MP7693_S13_R2_001.fastq.gz":"mp818_R2.fastq.gz",
    "20200103_9_MP7693_S14_R1_001.fastq.gz":"mp819_R1.fastq.gz",
    "20200103_9_MP7693_S14_R2_001.fastq.gz":"mp819_R2.fastq.gz",
    "20200103_10_MP7693_S15_R1_001.fastq.gz":"mp820_R1.fastq.gz",
    "20200103_10_MP7693_S15_R2_001.fastq.gz":"mp820_R2.fastq.gz",
    "20200103_11_MP7693_S16_R1_001.fastq.gz":"mp821_R1.fastq.gz",
    "20200103_11_MP7693_S16_R2_001.fastq.gz":"mp821_R2.fastq.gz",
    "20200103_12_MP7693_S17_R1_001.fastq.gz":"mp822_R1.fastq.gz",
    "20200103_12_MP7693_S17_R2_001.fastq.gz":"mp822_R2.fastq.gz",
    "20200103_13_MP7693_S18_R1_001.fastq.gz":"mp823_R1.fastq.gz",
    "20200103_13_MP7693_S18_R2_001.fastq.gz":"mp823_R2.fastq.gz",
    "20200103_14_MP7693_S19_R1_001.fastq.gz":"mp824_R1.fastq.gz",
    "20200103_14_MP7693_S19_R2_001.fastq.gz":"mp824_R2.fastq.gz",
    "20200103_15_MP7693_S20_R1_001.fastq.gz":"mp825_R1.fastq.gz",
    "20200103_15_MP7693_S20_R2_001.fastq.gz":"mp825_R2.fastq.gz",
    "20200103_16_MP7693_S21_R1_001.fastq.gz":"mp826_R1.fastq.gz",
    "20200103_16_MP7693_S21_R2_001.fastq.gz":"mp826_R2.fastq.gz",
    "20200103_17_MP7693_S22_R1_001.fastq.gz":"mp827_R1.fastq.gz",
    "20200103_17_MP7693_S22_R2_001.fastq.gz":"mp827_R2.fastq.gz",
    "20200103_18_MP7693_S23_R1_001.fastq.gz":"mp828_R1.fastq.gz",
    "20200103_18_MP7693_S23_R2_001.fastq.gz":"mp828_R2.fastq.gz",
    "20200103_19_MP7693_S24_R1_001.fastq.gz":"mp829_R1.fastq.gz",
    "20200103_19_MP7693_S24_R2_001.fastq.gz":"mp829_R2.fastq.gz",
    "20200103_20_MP7693_S25_R1_001.fastq.gz":"mp830_R1.fastq.gz",
    "20200103_20_MP7693_S25_R2_001.fastq.gz":"mp830_R2.fastq.gz"
}

In [ ]:
for k,v in rename.items():
    ! mv $bamfolder$k $bamfolder$v

In [ ]:
inputfastq="gs://amlproject/Chip_AML/Chip/mp99.fastq.gz"
! gsutil cp $inputfastq $bamfolder

In [ ]:
a = ! ls $bamfolder

In [ ]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
url="https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U"
gsheet = sheets.get(url).sheets[2].to_frame()

In [ ]:
df = {
"fastq1": [],
"fastq2": [],
"antibody": [],
"group": [],
"replicate": [],
"control": []
}
for val in helper.grouped(a[:-1],2):
    row = gsheet[gsheet.id==val[0].split('/')[-1].split('_')[0]]
    df['group'].append((row.id.values[0] + '_'+ row.name_replicate.values[0] +"_"+row.protein.values[0]))
    df['replicate'].append(1)
    df['fastq1'].append(val[0])
    #df['fastq2'].append(val[1])
    df['antibody'].append(row.protein.values[0])
    df['control'].append("INPUT")
df['group'].append('INPUT')
df['replicate'].append(1)
df['fastq1'].append(a[-1])
#df['fastq2'].append(val[1])
df['antibody'].append("INPUT")
df['control'].append("INPUT")
df = pd.DataFrame(df)

In [ ]:
df[df.columns[[3,4,0,1,2,5]]].to_csv('design.csv')

In [ ]:
#process chips
 ! sudo ./nextflow run nf-core/chipseq --single_end --seq_center 'DFCI' --email 'jkobject@gmail.com' --narrow_peak --input design.csv --genome GRCh38 --skip_preseq --max_cpus 24 -profile docker -w work

In [ ]:
! gsutil -m cp -r results/* gs://amlproject/Chip/IRF2BP2_degraded_histones/
! mv results results_hist

In [171]:
! gsutil -m cp gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/*.bam ../../data/diffBinding_hist/

Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/mp812-MV411_IRF2BP2_DMSO_6h-H3K27ac-r2.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/mp813-MV411_IRF2BP2_DMSO_6h-H3K27me3-r1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/mp814-MV411_IRF2BP2_DMSO_6h-H3K27me3-r2.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/mp815-MV411_IRF2BP2_DMSO_6h-H3K4me1-r1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/mp816-MV411_IRF2BP2_DMSO_6h-H3K4me1-r2.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/mp817-MV411_IRF2B

['../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp812-MV411_IRF2BP2_DMSO_6h-H3K27ac-r2.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp813-MV411_IRF2BP2_DMSO_6h-H3K27me3-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp814-MV411_IRF2BP2_DMSO_6h-H3K27me3-r2.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp815-MV411_IRF2BP2_DMSO_6h-H3K4me1-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp816-MV411_IRF2BP2_DMSO_6h-H3K4me1-r2.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp817-MV411_IRF2BP2_DMSO_6h-H3K4me3-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp818-MV411_IRF2BP2_DMSO_6h-H3K4me3-r2.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp819-MV411_IRF2BP2_DMSO_6h-H3K79me2-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp820-MV411_IRF2BP2_DMSO_6h-H3K79me2-r2.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp821-

In [38]:
bams = ! ls ../../data/diffBinding_hist/*.bam
bams = [i.split('/')[-1].split('.')[0] for i in bams]
bams

['INPUT_R1',
 'mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1',
 'mp812-MV411_IRF2BP2_DMSO_6h-H3K27ac-r2',
 'mp813-MV411_IRF2BP2_DMSO_6h-H3K27me3-r1',
 'mp814-MV411_IRF2BP2_DMSO_6h-H3K27me3-r2',
 'mp815-MV411_IRF2BP2_DMSO_6h-H3K4me1-r1',
 'mp816-MV411_IRF2BP2_DMSO_6h-H3K4me1-r2',
 'mp817-MV411_IRF2BP2_DMSO_6h-H3K4me3-r1',
 'mp818-MV411_IRF2BP2_DMSO_6h-H3K4me3-r2',
 'mp819-MV411_IRF2BP2_DMSO_6h-H3K79me2-r1',
 'mp820-MV411_IRF2BP2_DMSO_6h-H3K79me2-r2',
 'mp821-MV411_IRF2BP2_VHL_6h-H3K27ac-r1',
 'mp822-MV411_IRF2BP2_VHL_6h-H3K27ac-r2',
 'mp823-MV411_IRF2BP2_VHL_6h-H3K27me3-r1',
 'mp824-MV411_IRF2BP2_VHL_6h-H3K27me3-r2',
 'mp825-MV411_IRF2BP2_VHL_6h-H3K4me1-r1',
 'mp826-MV411_IRF2BP2_VHL_6h-H3K4me1-r2',
 'mp827-MV411_IRF2BP2_VHL_6h-H3K4me3-r1',
 'mp828-MV411_IRF2BP2_VHL_6h-H3K4me3-r2',
 'mp829-MV411_IRF2BP2_VHL_6h-H3K79me2-r1',
 'mp830-MV411_IRF2BP2_VHL_6h-H3K79me2-r2']

In [286]:
scales = [[191079,278272],
[274625,494562],
[1094016,2067804],
[1328914,969565],
[237779,132422],
[162971,174092],
[115788,187078],
[134269,495924],
[171890,225315],
[188370,199911]]

In [39]:
scales = [[1.0,0.6866626897],
[1.0,0.5552893267],
[1.0,0.5290714207],
[0.7295919826,1.0],
[0.5569120906,1.0],
[1.0,0.9361199825],
[1.0,0.6189290029],
[1.0,0.2707451142],
[1.0,0.762887513],
[1.0,0.9422693098]]

### on scalled data

In [40]:
# diffPeak on scaled data
size=[206, 213, 47, 272, 229, 235, 190, 196, 287, 288]
for i in range(int(len(bams)/2)):
    name1 = bams[1+i]
    name2 = bams[11+i]
    print(name1,name2)
    chip.diffPeak(name1, name2, directory= "../../data/diffBinding_hist/diffData/", res_directory='../../data/diffBinding_hist/diffPeaks/', scaling1=scales[i][1], scaling2=scales[i][0], size=size[i])

mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1 mp821-MV411_IRF2BP2_VHL_6h-H3K27ac-r1
doing differential peak binding
mp812-MV411_IRF2BP2_DMSO_6h-H3K27ac-r2 mp822-MV411_IRF2BP2_VHL_6h-H3K27ac-r2
doing differential peak binding
mp813-MV411_IRF2BP2_DMSO_6h-H3K27me3-r1 mp823-MV411_IRF2BP2_VHL_6h-H3K27me3-r1
doing differential peak binding
mp814-MV411_IRF2BP2_DMSO_6h-H3K27me3-r2 mp824-MV411_IRF2BP2_VHL_6h-H3K27me3-r2
doing differential peak binding
mp815-MV411_IRF2BP2_DMSO_6h-H3K4me1-r1 mp825-MV411_IRF2BP2_VHL_6h-H3K4me1-r1
doing differential peak binding
mp816-MV411_IRF2BP2_DMSO_6h-H3K4me1-r2 mp826-MV411_IRF2BP2_VHL_6h-H3K4me1-r2
doing differential peak binding
mp817-MV411_IRF2BP2_DMSO_6h-H3K4me3-r1 mp827-MV411_IRF2BP2_VHL_6h-H3K4me3-r1
doing differential peak binding
mp818-MV411_IRF2BP2_DMSO_6h-H3K4me3-r2 mp828-MV411_IRF2BP2_VHL_6h-H3K4me3-r2
doing differential peak binding
mp819-MV411_IRF2BP2_DMSO_6h-H3K79me2-r1 mp829-MV411_IRF2BP2_VHL_6h-H3K79me2-r1
doing differential peak binding
mp820-MV411_I

In [277]:
# diffPeak on scaled data
for i in range(int(len(bams)/2)):
    name1 = bams[1+i]
    name2 = bams[11+i]
    print(name1,name2)
    chip.fullDiffPeak(name1,name2, bams[0], scaling = scales[i], directory='../../data/diffBinding_hist/diffData/',
res_directory="../../data/diffBinding_hist/diffPeaks/", isTF=False, compute_size=True, pairedend=False)

../../data/diffBinding_hist/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1.mLb.clN.sorted.bam ../../data/diffBinding_hist/mp821-MV411_IRF2BP2_VHL_6h-H3K27ac-r1.mLb.clN.sorted.bam
doing diff from ../../data/diffBinding_hist/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1.mLb.clN.sorted.bam and ../../data/diffBinding_hist/mp821-MV411_IRF2BP2_VHL_6h-H3K27ac-r1.mLb.clN.sorted.bam
computing the fragment avg size


KeyboardInterrupt: 

In [ ]:
scales = [1.0,
1.0,
1.0,
0.7295919826,
0.5569120906,
1.0,
1.0,
1.0,
1.0,
1.0,
0.6866626897,
0.5552893267,
0.5290714207,
1.0,
1.0,
0.9361199825,
0.6189290029,
0.2707451142,
0.762887513,
0.9422693098,]

In [ ]:
chip.bigWigFrom(bams[1:],genome='GRCh38',scaling=scales)

In [58]:
bw = ! ls ../../data/recalib_bigwig_hist/*.bw
bw

['../../data/recalib_bigwig_hist/mp811_R1.bw',
 '../../data/recalib_bigwig_hist/mp812_R1.bw',
 '../../data/recalib_bigwig_hist/mp813_R1.bw',
 '../../data/recalib_bigwig_hist/mp814_R1.bw',
 '../../data/recalib_bigwig_hist/mp815_R1.bw',
 '../../data/recalib_bigwig_hist/mp816_R1.bw',
 '../../data/recalib_bigwig_hist/mp817_R1.bw',
 '../../data/recalib_bigwig_hist/mp818_R1.bw',
 '../../data/recalib_bigwig_hist/mp819_R1.bw',
 '../../data/recalib_bigwig_hist/mp820_R1.bw',
 '../../data/recalib_bigwig_hist/mp821_R1.bw',
 '../../data/recalib_bigwig_hist/mp822_R1.bw',
 '../../data/recalib_bigwig_hist/mp823_R1.bw',
 '../../data/recalib_bigwig_hist/mp824_R1.bw',
 '../../data/recalib_bigwig_hist/mp825_R1.bw',
 '../../data/recalib_bigwig_hist/mp826_R1.bw',
 '../../data/recalib_bigwig_hist/mp827_R1.bw',
 '../../data/recalib_bigwig_hist/mp828_R1.bw',
 '../../data/recalib_bigwig_hist/mp829_R1.bw',
 '../../data/recalib_bigwig_hist/mp830_R1.bw']

In [179]:
! rm ../../data/diffBinding_hist/diffData/mp*_R1*

In [59]:
cond1peak = ! ls ../../data/diffBinding_hist/diffPeaks/*cond1.bed
cond2peak = ! ls ../../data/diffBinding_hist/diffPeaks/*cond2.bed
commonpeak = ! ls ../../data/diffBinding_hist/diffPeaks/*common.bed
cond1peak

['../../data/diffBinding_hist/diffPeaks/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1_vs_mp821-MV411_IRF2BP2_VHL_6h-H3K27ac-r1_c3.0_cond1.bed',
 '../../data/diffBinding_hist/diffPeaks/mp812-MV411_IRF2BP2_DMSO_6h-H3K27ac-r2_vs_mp822-MV411_IRF2BP2_VHL_6h-H3K27ac-r2_c3.0_cond1.bed',
 '../../data/diffBinding_hist/diffPeaks/mp813-MV411_IRF2BP2_DMSO_6h-H3K27me3-r1_vs_mp823-MV411_IRF2BP2_VHL_6h-H3K27me3-r1_c3.0_cond1.bed',
 '../../data/diffBinding_hist/diffPeaks/mp814-MV411_IRF2BP2_DMSO_6h-H3K27me3-r2_vs_mp824-MV411_IRF2BP2_VHL_6h-H3K27me3-r2_c3.0_cond1.bed',
 '../../data/diffBinding_hist/diffPeaks/mp815-MV411_IRF2BP2_DMSO_6h-H3K4me1-r1_vs_mp825-MV411_IRF2BP2_VHL_6h-H3K4me1-r1_c3.0_cond1.bed',
 '../../data/diffBinding_hist/diffPeaks/mp816-MV411_IRF2BP2_DMSO_6h-H3K4me1-r2_vs_mp826-MV411_IRF2BP2_VHL_6h-H3K4me1-r2_c3.0_cond1.bed',
 '../../data/diffBinding_hist/diffPeaks/mp817-MV411_IRF2BP2_DMSO_6h-H3K4me3-r1_vs_mp827-MV411_IRF2BP2_VHL_6h-H3K4me3-r1_c3.0_cond1.bed',
 '../../data/diffBinding_hist/diffPea

In [60]:
names = ["H3K27ac", "H3K27ac_v2","H3K27me3","H3K27me3_v2","H3K4me1","H3K4me1_v2", "H3K4me3", "H3K4me3_v2", "H3K79me2", "H3K79me2_v2"]

In [61]:
for i in range(int(len(bw)/2)):
    if i < 0 : 
        continue
    print(i)
    name1 = bw[i]
    name2 = bw[10+i]
    peak = [cond1peak[i], commonpeak[i], cond2peak[i]]
    #for val in peak:
        #chip.dropWeirdChromosomes(val)
    name = names[i]
    print(name1,name2)
    chip.computePeaksAt(peak, [name1, name2], torecompute= True, bigwignames=['DMSO', 'VHL'], peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], window=3000, folder="", title=name, numthreads=7, refpoint='center', name='../../data/diffBinding_hist/diffPeaks/'+name+'_mat.pdf', withDeeptools=True)

0
../../data/recalib_bigwig_hist/mp811_R1.bw ../../data/recalib_bigwig_hist/mp821_R1.bw
CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_hist/mp811_R1.bw ../../data/recalib_bigwig_hist/mp821_R1.bw  --referencePoint center --regionsFileName ../../data/diffBinding_hist/diffPeaks/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1_vs_mp821-MV411_IRF2BP2_VHL_6h-H3K27ac-r1_c3.0_cond1.bed ../../data/diffBinding_hist/diffPeaks/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1_vs_mp821-MV411_IRF2BP2_VHL_6h-H3K27ac-r1_c3.0_common.bed ../../data/diffBinding_hist/diffPeaks/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1_vs_mp821-MV411_IRF2BP2_VHL_6h-H3K27ac-r1_c3.0_cond2.bed  --missingDataAsZero --outFileName ../../data/diffBinding_hist/diffPeaks/H3K27ac_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotHeatmap --matrixFile ../../data/diffBinding_hist/diffPeaks/H3K27ac_mat.gz --outFileName ../../data/diffBinding_hist/diffPeaks/H3K27ac_mat.pdf --refPointLabel center --regionsL

CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_hist/mp818_R1.bw ../../data/recalib_bigwig_hist/mp828_R1.bw  --referencePoint center --regionsFileName ../../data/diffBinding_hist/diffPeaks/mp818-MV411_IRF2BP2_DMSO_6h-H3K4me3-r2_vs_mp828-MV411_IRF2BP2_VHL_6h-H3K4me3-r2_c3.0_cond1.bed ../../data/diffBinding_hist/diffPeaks/mp818-MV411_IRF2BP2_DMSO_6h-H3K4me3-r2_vs_mp828-MV411_IRF2BP2_VHL_6h-H3K4me3-r2_c3.0_common.bed  --missingDataAsZero --outFileName ../../data/diffBinding_hist/diffPeaks/H3K4me3_v2_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotHeatmap --matrixFile ../../data/diffBinding_hist/diffPeaks/H3K4me3_v2_mat.gz --outFileName ../../data/diffBinding_hist/diffPeaks/H3K4me3_v2_mat.pdf --refPointLabel center --regionsLabel DMSO_peaks common  --samplesLabel DMSO VHL  --plotTitle H3K4me3_v2', returncode=0, stdout=b'', stderr=b'')
8
../../data/recalib_bigwig_hist/mp819_R1.bw ../../data/recalib_bigwig_hist/mp829_R1.bw
CompletedPro

In [225]:
names = ["H3K27ac", "H3K27ac_v2","H3K27me3","H3K27me3_v2","H3K4me1","H3K4me1_v2", "H3K4me3", "H3K4me3_v2", "H3K79me2", "H3K79me2_v2"]
names.extend([i+'_VHL' for i in names])
names

['H3K27ac',
 'H3K27ac_v2',
 'H3K27me3',
 'H3K27me3_v2',
 'H3K4me1',
 'H3K4me1_v2',
 'H3K4me3',
 'H3K4me3_v2',
 'H3K79me2',
 'H3K79me2_v2',
 'H3K27ac_VHL',
 'H3K27ac_v2_VHL',
 'H3K27me3_VHL',
 'H3K27me3_v2_VHL',
 'H3K4me1_VHL',
 'H3K4me1_v2_VHL',
 'H3K4me3_VHL',
 'H3K4me3_v2_VHL',
 'H3K79me2_VHL',
 'H3K79me2_v2_VHL']

In [129]:
mkdir ../../data/peakshist

In [148]:
! gsutil cp gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/macs/broadPeak/*.broadPeak ../../data/peakshist

Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/macs/broadPeak/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1.broadPeak...
Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/macs/broadPeak/mp812-MV411_IRF2BP2_DMSO_6h-H3K27ac-r2.broadPeak...
Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/macs/broadPeak/mp813-MV411_IRF2BP2_DMSO_6h-H3K27me3-r1.broadPeak...
Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/macs/broadPeak/mp814-MV411_IRF2BP2_DMSO_6h-H3K27me3-r2.broadPeak...
\ [4 files][ 17.4 MiB/ 17.4 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/macs/broadPeak/mp815-MV411_IRF2B

In [222]:
peaks = ! ls ../../data/peakshist/*
peaks

['../../data/peakshist/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1.broadPeak',
 '../../data/peakshist/mp812-MV411_IRF2BP2_DMSO_6h-H3K27ac-r2.broadPeak',
 '../../data/peakshist/mp813-MV411_IRF2BP2_DMSO_6h-H3K27me3-r1.broadPeak',
 '../../data/peakshist/mp814-MV411_IRF2BP2_DMSO_6h-H3K27me3-r2.broadPeak',
 '../../data/peakshist/mp815-MV411_IRF2BP2_DMSO_6h-H3K4me1-r1.broadPeak',
 '../../data/peakshist/mp816-MV411_IRF2BP2_DMSO_6h-H3K4me1-r2.broadPeak',
 '../../data/peakshist/mp817-MV411_IRF2BP2_DMSO_6h-H3K4me3-r1.broadPeak',
 '../../data/peakshist/mp818-MV411_IRF2BP2_DMSO_6h-H3K4me3-r2.broadPeak',
 '../../data/peakshist/mp819-MV411_IRF2BP2_DMSO_6h-H3K79me2-r1.broadPeak',
 '../../data/peakshist/mp820-MV411_IRF2BP2_DMSO_6h-H3K79me2-r2.broadPeak',
 '../../data/peakshist/mp821-MV411_IRF2BP2_VHL_6h-H3K27ac-r1.broadPeak',
 '../../data/peakshist/mp822-MV411_IRF2BP2_VHL_6h-H3K27ac-r2.broadPeak',
 '../../data/peakshist/mp823-MV411_IRF2BP2_VHL_6h-H3K27me3-r1.broadPeak',
 '../../data/peakshist/mp824-MV411_I

In [226]:
for i in range(len(bw)):
    chip.computePeaksAt(peaks[i], bw[i], window=3000, folder="", title=names[i], numthreads=7, torecompute=True, onlyProfile=True, refpoint='center', name='../../data/diffBinding_hist/diffPeaks/'+names[i]+'_mat_profile.pdf', withDeeptools=True)
    chip.computePeaksAt(peaks[i], bw[i], window=3000, folder="", title=names[i], numthreads=7, torecompute=True, refpoint='center', onlyProfile=True,name='../../data/diffBinding_hist/diffPeaks/'+names[i]+'_mat_profile_clust3.pdf', withDeeptools=True, cluster=3)

CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_hist/mp811_R1.bw  --referencePoint center --regionsFileName ../../data/peakshist/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1.broadPeak  --missingDataAsZero --outFileName ../../data/diffBinding_hist/diffPeaks/H3K27ac_mat_profile.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotProfile --matrixFile ../../data/diffBinding_hist/diffPeaks/H3K27ac_mat_profile.gz --outFileName ../../data/diffBinding_hist/diffPeaks/H3K27ac_mat_profile.pdf --refPointLabel center --plotTitle H3K27ac', returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_hist/mp811_R1.bw  --referencePoint center --regionsFileName ../../data/peakshist/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1.broadPeak  --missingDataAsZero --outFileName ../../data/diffBinding_hist/diffPeaks/H3K27ac_mat_profile_clust3.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotProfile 

CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_hist/mp817_R1.bw  --referencePoint center --regionsFileName ../../data/peakshist/mp817-MV411_IRF2BP2_DMSO_6h-H3K4me3-r1.broadPeak  --missingDataAsZero --outFileName ../../data/diffBinding_hist/diffPeaks/H3K4me3_mat_profile_clust3.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotProfile --matrixFile ../../data/diffBinding_hist/diffPeaks/H3K4me3_mat_profile_clust3.gz --outFileName ../../data/diffBinding_hist/diffPeaks/H3K4me3_mat_profile_clust3.pdf --refPointLabel center --perGroup --kmeans 3 --plotTitle H3K4me3', returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_hist/mp818_R1.bw  --referencePoint center --regionsFileName ../../data/peakshist/mp818-MV411_IRF2BP2_DMSO_6h-H3K4me3-r2.broadPeak  --missingDataAsZero --outFileName ../../data/diffBinding_hist/diffPeaks/H3K4me3_v2_mat_profile.gz --upstream 3000 --downstream 3000

CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_hist/mp824_R1.bw  --referencePoint center --regionsFileName ../../data/peakshist/mp824-MV411_IRF2BP2_VHL_6h-H3K27me3-r2.broadPeak  --missingDataAsZero --outFileName ../../data/diffBinding_hist/diffPeaks/H3K27me3_v2_VHL_mat_profile.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotProfile --matrixFile ../../data/diffBinding_hist/diffPeaks/H3K27me3_v2_VHL_mat_profile.gz --outFileName ../../data/diffBinding_hist/diffPeaks/H3K27me3_v2_VHL_mat_profile.pdf --refPointLabel center --plotTitle H3K27me3_v2_VHL', returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_hist/mp824_R1.bw  --referencePoint center --regionsFileName ../../data/peakshist/mp824-MV411_IRF2BP2_VHL_6h-H3K27me3-r2.broadPeak  --missingDataAsZero --outFileName ../../data/diffBinding_hist/diffPeaks/H3K27me3_v2_VHL_mat_profile_clust3.gz --upstream 3000 --downstream 300

CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_hist/mp830_R1.bw  --referencePoint center --regionsFileName ../../data/peakshist/mp830-MV411_IRF2BP2_VHL_6h-H3K79me2-r2.broadPeak  --missingDataAsZero --outFileName ../../data/diffBinding_hist/diffPeaks/H3K79me2_v2_VHL_mat_profile_clust3.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotProfile --matrixFile ../../data/diffBinding_hist/diffPeaks/H3K79me2_v2_VHL_mat_profile_clust3.gz --outFileName ../../data/diffBinding_hist/diffPeaks/H3K79me2_v2_VHL_mat_profile_clust3.pdf --refPointLabel center --perGroup --kmeans 3 --plotTitle H3K79me2_v2_VHL', returncode=0, stdout=b'', stderr=b'')


### on unscalled data

In [86]:
bams = !ls ../../data/diffBinding_hist/mp*.bam
bams

['../../data/diffBinding_hist/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp812-MV411_IRF2BP2_DMSO_6h-H3K27ac-r2.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp813-MV411_IRF2BP2_DMSO_6h-H3K27me3-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp814-MV411_IRF2BP2_DMSO_6h-H3K27me3-r2.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp815-MV411_IRF2BP2_DMSO_6h-H3K4me1-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp816-MV411_IRF2BP2_DMSO_6h-H3K4me1-r2.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp817-MV411_IRF2BP2_DMSO_6h-H3K4me3-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp818-MV411_IRF2BP2_DMSO_6h-H3K4me3-r2.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp819-MV411_IRF2BP2_DMSO_6h-H3K79me2-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp820-MV411_IRF2BP2_DMSO_6h-H3K79me2-r2.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp821-MV411_IRF2BP2_VHL_6h-H3K27ac-r1.mLb.clN.sorted.bam',
 '../..

In [209]:
! mkdir../../data/diffBinding_hist/diffPeaks_unscaled
!mkdir ../../data/bigwigrephistunscaled/
!gsutil -m cp gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/bigwig/*.bigWig ../../data/bigwigrephistunscaled/

Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/bigwig/INPUT_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/bigwig/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/bigwig/mp812-MV411_IRF2BP2_DMSO_6h-H3K27ac-r2.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/bigwig/mp813-MV411_IRF2BP2_DMSO_6h-H3K27me3-r1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/bigwig/mp814-MV411_IRF2BP2_DMSO_6h-H3K27me3-r2.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/bigwig/mp815-MV411_IRF2BP2_DMSO_6h-H3K4me1-r1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibrary/bigwig/mp816-MV411_IRF2BP2_DMSO_6h-H3K4me1-r2.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_histones/bwa/mergedLibr

In [191]:
# on unscalled data 
for i in range(int((len(bams)-1)/2)):
    name1 = bams[1+i]
    name2 = bams[11+i]
    chip.fullDiffPeak(name1,name2, control1='../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam', directory = "../../data/diffBinding_hist/diffData_unscaled/", res_directory = "../../data/diffBinding_hist/diffPeaks_unscaled/",isTF=False, compute_size=True, pairedend=False)

doing diff from ../../data/diffBinding_hist/mp812-MV411_IRF2BP2_DMSO_6h-H3K27ac-r2.mLb.clN.sorted.bam and ../../data/diffBinding_hist/mp822-MV411_IRF2BP2_VHL_6h-H3K27ac-r2.mLb.clN.sorted.bam
computing the fragment avg size
213
computing the scaling values
b'INFO  @ Tue, 24 Mar 2020 18:33:07: \n# Command line: callpeak -B -t ../../data/diffBinding_hist/mp812-MV411_IRF2BP2_DMSO_6h-H3K27ac-r2.mLb.clN.sorted.bam -c ../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 213 -n mp812-MV411_IRF2BP2_DMSO_6h-H3K27ac-r2 --outdir ../../data/diffBinding_hist/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = mp812-MV411_IRF2BP2_DMSO_6h-H3K27ac-r2\n# format = BAM\n# ChIP-seq file = [\'../../data/diffBinding_hist/mp812-MV411_IRF2BP2_DMSO_6h-H3K27ac-r2.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum ga

doing diff from ../../data/diffBinding_hist/mp813-MV411_IRF2BP2_DMSO_6h-H3K27me3-r1.mLb.clN.sorted.bam and ../../data/diffBinding_hist/mp823-MV411_IRF2BP2_VHL_6h-H3K27me3-r1.mLb.clN.sorted.bam
computing the fragment avg size
47
computing the scaling values
b'INFO  @ Tue, 24 Mar 2020 19:00:06: \n# Command line: callpeak -B -t ../../data/diffBinding_hist/mp813-MV411_IRF2BP2_DMSO_6h-H3K27me3-r1.mLb.clN.sorted.bam -c ../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 47 -n mp813-MV411_IRF2BP2_DMSO_6h-H3K27me3-r1 --outdir ../../data/diffBinding_hist/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = mp813-MV411_IRF2BP2_DMSO_6h-H3K27me3-r1\n# format = BAM\n# ChIP-seq file = [\'../../data/diffBinding_hist/mp813-MV411_IRF2BP2_DMSO_6h-H3K27me3-r1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximu

doing diff from ../../data/diffBinding_hist/mp814-MV411_IRF2BP2_DMSO_6h-H3K27me3-r2.mLb.clN.sorted.bam and ../../data/diffBinding_hist/mp824-MV411_IRF2BP2_VHL_6h-H3K27me3-r2.mLb.clN.sorted.bam
computing the fragment avg size
272
computing the scaling values
b'INFO  @ Tue, 24 Mar 2020 19:25:04: \n# Command line: callpeak -B -t ../../data/diffBinding_hist/mp814-MV411_IRF2BP2_DMSO_6h-H3K27me3-r2.mLb.clN.sorted.bam -c ../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 272 -n mp814-MV411_IRF2BP2_DMSO_6h-H3K27me3-r2 --outdir ../../data/diffBinding_hist/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = mp814-MV411_IRF2BP2_DMSO_6h-H3K27me3-r2\n# format = BAM\n# ChIP-seq file = [\'../../data/diffBinding_hist/mp814-MV411_IRF2BP2_DMSO_6h-H3K27me3-r2.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maxi

doing diff from ../../data/diffBinding_hist/mp815-MV411_IRF2BP2_DMSO_6h-H3K4me1-r1.mLb.clN.sorted.bam and ../../data/diffBinding_hist/mp825-MV411_IRF2BP2_VHL_6h-H3K4me1-r1.mLb.clN.sorted.bam
computing the fragment avg size
229
computing the scaling values
b'INFO  @ Tue, 24 Mar 2020 19:48:40: \n# Command line: callpeak -B -t ../../data/diffBinding_hist/mp815-MV411_IRF2BP2_DMSO_6h-H3K4me1-r1.mLb.clN.sorted.bam -c ../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 229 -n mp815-MV411_IRF2BP2_DMSO_6h-H3K4me1-r1 --outdir ../../data/diffBinding_hist/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = mp815-MV411_IRF2BP2_DMSO_6h-H3K4me1-r1\n# format = BAM\n# ChIP-seq file = [\'../../data/diffBinding_hist/mp815-MV411_IRF2BP2_DMSO_6h-H3K4me1-r1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum ga

doing diff from ../../data/diffBinding_hist/mp816-MV411_IRF2BP2_DMSO_6h-H3K4me1-r2.mLb.clN.sorted.bam and ../../data/diffBinding_hist/mp826-MV411_IRF2BP2_VHL_6h-H3K4me1-r2.mLb.clN.sorted.bam
computing the fragment avg size
235
computing the scaling values
b'INFO  @ Tue, 24 Mar 2020 20:23:27: \n# Command line: callpeak -B -t ../../data/diffBinding_hist/mp816-MV411_IRF2BP2_DMSO_6h-H3K4me1-r2.mLb.clN.sorted.bam -c ../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 235 -n mp816-MV411_IRF2BP2_DMSO_6h-H3K4me1-r2 --outdir ../../data/diffBinding_hist/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = mp816-MV411_IRF2BP2_DMSO_6h-H3K4me1-r2\n# format = BAM\n# ChIP-seq file = [\'../../data/diffBinding_hist/mp816-MV411_IRF2BP2_DMSO_6h-H3K4me1-r2.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum ga

doing diff from ../../data/diffBinding_hist/mp817-MV411_IRF2BP2_DMSO_6h-H3K4me3-r1.mLb.clN.sorted.bam and ../../data/diffBinding_hist/mp827-MV411_IRF2BP2_VHL_6h-H3K4me3-r1.mLb.clN.sorted.bam
computing the fragment avg size
190
computing the scaling values
b'INFO  @ Tue, 24 Mar 2020 20:55:24: \n# Command line: callpeak -B -t ../../data/diffBinding_hist/mp817-MV411_IRF2BP2_DMSO_6h-H3K4me3-r1.mLb.clN.sorted.bam -c ../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 190 -n mp817-MV411_IRF2BP2_DMSO_6h-H3K4me3-r1 --outdir ../../data/diffBinding_hist/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = mp817-MV411_IRF2BP2_DMSO_6h-H3K4me3-r1\n# format = BAM\n# ChIP-seq file = [\'../../data/diffBinding_hist/mp817-MV411_IRF2BP2_DMSO_6h-H3K4me3-r1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum ga

doing diff from ../../data/diffBinding_hist/mp818-MV411_IRF2BP2_DMSO_6h-H3K4me3-r2.mLb.clN.sorted.bam and ../../data/diffBinding_hist/mp828-MV411_IRF2BP2_VHL_6h-H3K4me3-r2.mLb.clN.sorted.bam
computing the fragment avg size
196
computing the scaling values
b'INFO  @ Tue, 24 Mar 2020 21:18:13: \n# Command line: callpeak -B -t ../../data/diffBinding_hist/mp818-MV411_IRF2BP2_DMSO_6h-H3K4me3-r2.mLb.clN.sorted.bam -c ../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 196 -n mp818-MV411_IRF2BP2_DMSO_6h-H3K4me3-r2 --outdir ../../data/diffBinding_hist/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = mp818-MV411_IRF2BP2_DMSO_6h-H3K4me3-r2\n# format = BAM\n# ChIP-seq file = [\'../../data/diffBinding_hist/mp818-MV411_IRF2BP2_DMSO_6h-H3K4me3-r2.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum ga

doing diff from ../../data/diffBinding_hist/mp819-MV411_IRF2BP2_DMSO_6h-H3K79me2-r1.mLb.clN.sorted.bam and ../../data/diffBinding_hist/mp829-MV411_IRF2BP2_VHL_6h-H3K79me2-r1.mLb.clN.sorted.bam
computing the fragment avg size
287
computing the scaling values
b'INFO  @ Tue, 24 Mar 2020 21:40:17: \n# Command line: callpeak -B -t ../../data/diffBinding_hist/mp819-MV411_IRF2BP2_DMSO_6h-H3K79me2-r1.mLb.clN.sorted.bam -c ../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 287 -n mp819-MV411_IRF2BP2_DMSO_6h-H3K79me2-r1 --outdir ../../data/diffBinding_hist/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = mp819-MV411_IRF2BP2_DMSO_6h-H3K79me2-r1\n# format = BAM\n# ChIP-seq file = [\'../../data/diffBinding_hist/mp819-MV411_IRF2BP2_DMSO_6h-H3K79me2-r1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maxi

doing diff from ../../data/diffBinding_hist/mp820-MV411_IRF2BP2_DMSO_6h-H3K79me2-r2.mLb.clN.sorted.bam and ../../data/diffBinding_hist/mp830-MV411_IRF2BP2_VHL_6h-H3K79me2-r2.mLb.clN.sorted.bam
computing the fragment avg size
288
computing the scaling values
b'INFO  @ Tue, 24 Mar 2020 22:15:01: \n# Command line: callpeak -B -t ../../data/diffBinding_hist/mp820-MV411_IRF2BP2_DMSO_6h-H3K79me2-r2.mLb.clN.sorted.bam -c ../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 288 -n mp820-MV411_IRF2BP2_DMSO_6h-H3K79me2-r2 --outdir ../../data/diffBinding_hist/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = mp820-MV411_IRF2BP2_DMSO_6h-H3K79me2-r2\n# format = BAM\n# ChIP-seq file = [\'../../data/diffBinding_hist/mp820-MV411_IRF2BP2_DMSO_6h-H3K79me2-r2.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maxi

In [131]:
bw = ! ls ../../data/bigwigrephistunscaled/*.bigWig
bw

['../../data/bigwigrephistunscaled/INPUT_R1.mLb.clN.bigWig',
 '../../data/bigwigrephistunscaled/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1.mLb.clN.bigWig',
 '../../data/bigwigrephistunscaled/mp812-MV411_IRF2BP2_DMSO_6h-H3K27ac-r2.mLb.clN.bigWig',
 '../../data/bigwigrephistunscaled/mp813-MV411_IRF2BP2_DMSO_6h-H3K27me3-r1.mLb.clN.bigWig',
 '../../data/bigwigrephistunscaled/mp814-MV411_IRF2BP2_DMSO_6h-H3K27me3-r2.mLb.clN.bigWig',
 '../../data/bigwigrephistunscaled/mp815-MV411_IRF2BP2_DMSO_6h-H3K4me1-r1.mLb.clN.bigWig',
 '../../data/bigwigrephistunscaled/mp816-MV411_IRF2BP2_DMSO_6h-H3K4me1-r2.mLb.clN.bigWig',
 '../../data/bigwigrephistunscaled/mp817-MV411_IRF2BP2_DMSO_6h-H3K4me3-r1.mLb.clN.bigWig',
 '../../data/bigwigrephistunscaled/mp818-MV411_IRF2BP2_DMSO_6h-H3K4me3-r2.mLb.clN.bigWig',
 '../../data/bigwigrephistunscaled/mp819-MV411_IRF2BP2_DMSO_6h-H3K79me2-r1.mLb.clN.bigWig',
 '../../data/bigwigrephistunscaled/mp820-MV411_IRF2BP2_DMSO_6h-H3K79me2-r2.mLb.clN.bigWig',
 '../../data/bigwigrephis

In [132]:
cond1peak = ! ls ../../data/diffBinding_hist/diffPeaks_unscaled/*cond1.bed
cond2peak = ! ls ../../data/diffBinding_hist/diffPeaks_unscaled/*cond2.bed
commonpeak = ! ls ../../data/diffBinding_hist/diffPeaks_unscaled/*common.bed
cond2peak

['../../data/diffBinding_hist/diffPeaks_unscaled/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1_vs_mp821-MV411_IRF2BP2_VHL_6h-H3K27ac-r1_c3.0_cond2.bed',
 '../../data/diffBinding_hist/diffPeaks_unscaled/mp812-MV411_IRF2BP2_DMSO_6h-H3K27ac-r2_vs_mp822-MV411_IRF2BP2_VHL_6h-H3K27ac-r2_c3.0_cond2.bed',
 '../../data/diffBinding_hist/diffPeaks_unscaled/mp813-MV411_IRF2BP2_DMSO_6h-H3K27me3-r1_vs_mp823-MV411_IRF2BP2_VHL_6h-H3K27me3-r1_c3.0_cond2.bed',
 '../../data/diffBinding_hist/diffPeaks_unscaled/mp814-MV411_IRF2BP2_DMSO_6h-H3K27me3-r2_vs_mp824-MV411_IRF2BP2_VHL_6h-H3K27me3-r2_c3.0_cond2.bed',
 '../../data/diffBinding_hist/diffPeaks_unscaled/mp815-MV411_IRF2BP2_DMSO_6h-H3K4me1-r1_vs_mp825-MV411_IRF2BP2_VHL_6h-H3K4me1-r1_c3.0_cond2.bed',
 '../../data/diffBinding_hist/diffPeaks_unscaled/mp816-MV411_IRF2BP2_DMSO_6h-H3K4me1-r2_vs_mp826-MV411_IRF2BP2_VHL_6h-H3K4me1-r2_c3.0_cond2.bed',
 '../../data/diffBinding_hist/diffPeaks_unscaled/mp817-MV411_IRF2BP2_DMSO_6h-H3K4me3-r1_vs_mp827-MV411_IRF2BP2_VHL_6h-H3

In [133]:
names = ["H3K27ac", "H3K27ac_v2","H3K27me3","H3K27me3_v2","H3K4me1","H3K4me1_v2", "H3K4me3", "H3K4me3_v2", "H3K79me2", "H3K79me2_v2"]

In [134]:
for i in range(int(len(bw)/2)):
    if i < 0:
        continue
    name1 = bw[1+i]
    name2 = bw[11+i]
    peak = [cond1peak[i], commonpeak[i], cond2peak[i]]
    #for val in peak:
        #chip.dropWeirdChromosomes(val)
    name = names[i]
    print(name1,name2)
    chip.computePeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], window=3000, folder="", title=name, numthreads=10, refpoint='center', name='../../data/diffBinding_hist/diffPeaks_unscaled/'+name+'_mat.pdf', withDeeptools=True, torecompute=True)

../../data/bigwigrephistunscaled/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1.mLb.clN.bigWig ../../data/bigwigrephistunscaled/mp821-MV411_IRF2BP2_VHL_6h-H3K27ac-r1.mLb.clN.bigWig
CompletedProcess(args='computeMatrix reference-point -S ../../data/bigwigrephistunscaled/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1.mLb.clN.bigWig ../../data/bigwigrephistunscaled/mp821-MV411_IRF2BP2_VHL_6h-H3K27ac-r1.mLb.clN.bigWig  --referencePoint center --regionsFileName ../../data/diffBinding_hist/diffPeaks_unscaled/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1_vs_mp821-MV411_IRF2BP2_VHL_6h-H3K27ac-r1_c3.0_cond1.bed ../../data/diffBinding_hist/diffPeaks_unscaled/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1_vs_mp821-MV411_IRF2BP2_VHL_6h-H3K27ac-r1_c3.0_common.bed ../../data/diffBinding_hist/diffPeaks_unscaled/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1_vs_mp821-MV411_IRF2BP2_VHL_6h-H3K27ac-r1_c3.0_cond2.bed  --missingDataAsZero --outFileName ../../data/diffBinding_hist/diffPeaks_unscaled/H3K27ac_mat.gz --upstream 3000 --downstream 30

CompletedProcess(args='computeMatrix reference-point -S ../../data/bigwigrephistunscaled/mp816-MV411_IRF2BP2_DMSO_6h-H3K4me1-r2.mLb.clN.bigWig ../../data/bigwigrephistunscaled/mp826-MV411_IRF2BP2_VHL_6h-H3K4me1-r2.mLb.clN.bigWig  --referencePoint center --regionsFileName ../../data/diffBinding_hist/diffPeaks_unscaled/mp816-MV411_IRF2BP2_DMSO_6h-H3K4me1-r2_vs_mp826-MV411_IRF2BP2_VHL_6h-H3K4me1-r2_c3.0_cond1.bed ../../data/diffBinding_hist/diffPeaks_unscaled/mp816-MV411_IRF2BP2_DMSO_6h-H3K4me1-r2_vs_mp826-MV411_IRF2BP2_VHL_6h-H3K4me1-r2_c3.0_common.bed ../../data/diffBinding_hist/diffPeaks_unscaled/mp816-MV411_IRF2BP2_DMSO_6h-H3K4me1-r2_vs_mp826-MV411_IRF2BP2_VHL_6h-H3K4me1-r2_c3.0_cond2.bed  --missingDataAsZero --outFileName ../../data/diffBinding_hist/diffPeaks_unscaled/H3K4me1_v2_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 10 && plotHeatmap --matrixFile ../../data/diffBinding_hist/diffPeaks_unscaled/H3K4me1_v2_mat.gz --outFileName ../../data/diffBinding_hist/diffPeak

CompletedProcess(args='computeMatrix reference-point -S ../../data/bigwigrephistunscaled/mp820-MV411_IRF2BP2_DMSO_6h-H3K79me2-r2.mLb.clN.bigWig ../../data/bigwigrephistunscaled/mp830-MV411_IRF2BP2_VHL_6h-H3K79me2-r2.mLb.clN.bigWig  --referencePoint center --regionsFileName ../../data/diffBinding_hist/diffPeaks_unscaled/mp820-MV411_IRF2BP2_DMSO_6h-H3K79me2-r2_vs_mp830-MV411_IRF2BP2_VHL_6h-H3K79me2-r2_c3.0_cond1.bed ../../data/diffBinding_hist/diffPeaks_unscaled/mp820-MV411_IRF2BP2_DMSO_6h-H3K79me2-r2_vs_mp830-MV411_IRF2BP2_VHL_6h-H3K79me2-r2_c3.0_common.bed ../../data/diffBinding_hist/diffPeaks_unscaled/mp820-MV411_IRF2BP2_DMSO_6h-H3K79me2-r2_vs_mp830-MV411_IRF2BP2_VHL_6h-H3K79me2-r2_c3.0_cond2.bed  --missingDataAsZero --outFileName ../../data/diffBinding_hist/diffPeaks_unscaled/H3K79me2_v2_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 10 && plotHeatmap --matrixFile ../../data/diffBinding_hist/diffPeaks_unscaled/H3K79me2_v2_mat.gz --outFileName ../../data/diffBinding_his

In [ ]:
! gsutil -m cp -r "diffPeaks" gs://amlproject/Chip_AML/IRF2BP2/ &&\
    gsutil -m cp -r "diffData" gs://amlproject/Chip_AML/IRF2BP2/ &&\
        gsutil -m cp "bigwigs/mp*" gs://amlproject/Chip_AML/IRF2BP2/bigwigs

## v4

In [142]:
! gsutil -m mv gs://transfer-amlproject/*MP7868*  gs://transfer-amlproject/IRF2BP2_v4/

Copying gs://transfer-amlproject/20200302_10_MP7868_S60_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200302_10_MP7868_S60_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200302_11_MP7868_S61_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200302_11_MP7868_S61_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200302_12_MP7868_S62_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200302_12_MP7868_S62_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200302_1_MP7868_S51_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200302_1_MP7868_S51_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200302_2_MP7868_S52_R1_001.fastq.gz [Content-Ty

### analysis

In [148]:
! mkdir ../../data/bams4 && gsutil -m cp gs://transfer-amlproject/IRF2BP2_v4/* ../../data/bams4/

Copying gs://transfer-amlproject/IRF2BP2_v4/20200302_10_MP7868_S60_R1_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v4/20200302_10_MP7868_S60_R2_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v4/20200302_11_MP7868_S61_R1_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v4/20200302_11_MP7868_S61_R2_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v4/20200302_12_MP7868_S62_R1_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v4/20200302_12_MP7868_S62_R2_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v4/20200302_1_MP7868_S51_R1_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v4/20200302_1_MP7868_S51_R2_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v4/20200302_2_MP7868_S52_R1_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v4/20200302_2_MP7868_S52_R2_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v4/20200302_3_MP7868_S53_R1_001.fastq.gz...
Copying gs://transfer-amlproject/IRF2BP2_v4/20200302_3_MP78

In [149]:
ls ../../data/bams4

20200302_10_MP7868_S60_R1_001.fastq.gz  20200302_4_MP7868_S54_R1_001.fastq.gz
20200302_10_MP7868_S60_R2_001.fastq.gz  20200302_4_MP7868_S54_R2_001.fastq.gz
20200302_11_MP7868_S61_R1_001.fastq.gz  20200302_5_MP7868_S55_R1_001.fastq.gz
20200302_11_MP7868_S61_R2_001.fastq.gz  20200302_5_MP7868_S55_R2_001.fastq.gz
20200302_12_MP7868_S62_R1_001.fastq.gz  20200302_6_MP7868_S56_R1_001.fastq.gz
20200302_12_MP7868_S62_R2_001.fastq.gz  20200302_6_MP7868_S56_R2_001.fastq.gz
20200302_1_MP7868_S51_R1_001.fastq.gz   20200302_7_MP7868_S57_R1_001.fastq.gz
20200302_1_MP7868_S51_R2_001.fastq.gz   20200302_7_MP7868_S57_R2_001.fastq.gz
20200302_2_MP7868_S52_R1_001.fastq.gz   20200302_8_MP7868_S58_R1_001.fastq.gz
20200302_2_MP7868_S52_R2_001.fastq.gz   20200302_8_MP7868_S58_R2_001.fastq.gz
20200302_3_MP7868_S53_R1_001.fastq.gz   20200302_9_MP7868_S59_R1_001.fastq.gz
20200302_3_MP7868_S53_R2_001.fastq.gz   20200302_9_MP7868_S59_R2_001.fastq.gz


In [150]:
rename = {
"20200302_1_MP7868_S51_R1_001.fastq.gz":"mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1.fastq.gz", 
"20200302_1_MP7868_S51_R2_001.fastq.gz":"mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R2.fastq.gz", 
"20200302_2_MP7868_S52_R1_001.fastq.gz":"mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1.fastq.gz", 
"20200302_2_MP7868_S52_R2_001.fastq.gz":"mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R2.fastq.gz", 
"20200302_3_MP7868_S53_R1_001.fastq.gz":"mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R1.fastq.gz", 
"20200302_3_MP7868_S53_R2_001.fastq.gz":"mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R2.fastq.gz",
"20200302_4_MP7868_S54_R1_001.fastq.gz":"mp849-MV411_IRF2BP_DMSO_6h-POLII_total-r3_R1.fastq.gz",
"20200302_4_MP7868_S54_R2_001.fastq.gz":"mp849-MV411_IRF2BP_DMSO_6h-POLII_total-r3_R2.fastq.gz",
"20200302_5_MP7868_S55_R1_001.fastq.gz":"mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R1.fastq.gz",
"20200302_5_MP7868_S55_R2_001.fastq.gz":"mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R2.fastq.gz",
"20200302_6_MP7868_S56_R1_001.fastq.gz":"mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1.fastq.gz",
"20200302_6_MP7868_S56_R2_001.fastq.gz":"mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R2.fastq.gz",
"20200302_7_MP7868_S57_R1_001.fastq.gz":"mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1.fastq.gz",
"20200302_7_MP7868_S57_R2_001.fastq.gz":"mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R2.fastq.gz",
"20200302_8_MP7868_S58_R1_001.fastq.gz":"mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R1.fastq.gz",
"20200302_8_MP7868_S58_R2_001.fastq.gz":"mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R2.fastq.gz",
"20200302_9_MP7868_S59_R1_001.fastq.gz":"mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3_R1.fastq.gz",
"20200302_9_MP7868_S59_R2_001.fastq.gz":"mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3_R2.fastq.gz", 
"20200302_10_MP7868_S60_R1_001.fastq.gz":"mp855-MV411_IRF2BP_VHL_6h-POLII_total-r3_R1.fastq.gz",
"20200302_10_MP7868_S60_R2_001.fastq.gz":"mp855-MV411_IRF2BP_VHL_6h-POLII_total-r3_R2.fastq.gz",
"20200302_11_MP7868_S61_R1_001.fastq.gz":"mp856-MV411_IRF2BP_VHL_6h-POLII_S2-r3_R1.fastq.gz",
"20200302_11_MP7868_S61_R2_001.fastq.gz":"mp856-MV411_IRF2BP_VHL_6h-POLII_S2-r3_R2.fastq.gz",
"20200302_12_MP7868_S62_R1_001.fastq.gz":"mp857-MV411_IRF2BP_VHL_6h-POLII_S5-r3_R1.fastq.gz",
"20200302_12_MP7868_S62_R2_001.fastq.gz":"mp857-MV411_IRF2BP_VHL_6h-POLII_S5-r3_R2.fastq.gz"
}

In [152]:
for k,v in rename.items():
    ! mv ../../data/bams4/$k ../../data/bams4/$v

In [153]:
ls ../../data/bams4

mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1.fastq.gz
mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R2.fastq.gz
mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1.fastq.gz
mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R2.fastq.gz
mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R1.fastq.gz
mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R2.fastq.gz
mp849-MV411_IRF2BP_DMSO_6h-POLII_total-r3_R1.fastq.gz
mp849-MV411_IRF2BP_DMSO_6h-POLII_total-r3_R2.fastq.gz
mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R1.fastq.gz
mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R2.fastq.gz
mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1.fastq.gz
mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R2.fastq.gz
mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1.fastq.gz
mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R2.fastq.gz
mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R1.fastq.gz
mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R2.fastq.gz
mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3_R1.fastq.gz
mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3_R2.fastq.gz
mp855-MV411_IRF2BP_VHL_6h-POLII_total-r3_R1.fastq.gz
mp855-MV411_IRF2BP_VHL_6h-POLII_total-r3_R2.f

In [154]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
url="https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U"
gsheet = sheets.get(url).sheets[2].to_frame()

In [155]:
a = ! ls ../../data/bams4

In [156]:
gsheet

,id,cell line,replicate,protein,quality,matching input name,processed,name,previous name,bigwig,...,individual QC,unique mapped reads(droso),scaling factor,Total QC,folderNarrow,folderCompensated,folderQC,folderBroad,folder Bigwig,folder diffPeaks
0,mp100,U937,1,INPUT,NaN,NaN,Y,mp100-U937-INPUT-r1,NaN,https://console.cloud.google.com/storage/brows...,...,https://console.cloud.google.com/storage/brows...,NaN,NaN,https://storage.cloud.google.com/amlproject/Ch...,https://console.cloud.google.com/storage/brows...,NaN,https://console.cloud.google.com/storage/brows...,https://console.cloud.google.com/storage/brows...,https://console.cloud.google.com/storage/brows...,NaN
1,mp101,NOMO1,1,INPUT,NaN,NaN,Y,mp101-NOMO1-INPUT-r1,NaN,https://console.cloud.google.com/storage/brows...,...,https://console.cloud.google.com/storage/brows...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,mp102,UT7,1,INPUT,NaN,NaN,Y,mp102-UT7-INPUT-r1,NaN,https://console.cloud.google.com/storage/brows...,...,https://console.cloud.google.com/storage/brows...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mp106,MV411,1,MYB,x,INPUT_MV411,Y,mp106-MV411-MYB-r1,NaN,https://console.cloud.google.com/storage/brows...,...,https://console.cloud.google.com/storage/brows...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mp109,M6,1,CEBPA,x,INPUT_M6,Y,mp109-M6-CEBPA-r1,NaN,https://console.cloud.google.com/storage/brows...,...,https://console.cloud.google.com/storage/brows...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,mp853,MV411_IRF2BP_VHL_6h,5,MED1,NaN,NaN,NaN,mp853-MV411_IRF2BP_VHL_6h-MED1-r5,20200302_8_MP7868_S58_R1_001,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
282,mp854,MV411_IRF2BP_VHL_6h,3,FLAG_IRF2BP2,NaN,NaN,NaN,mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3,20200302_9_MP7868_S59_R1_001,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
283,mp855,MV411_IRF2BP_VHL_6h,3,POLII_total,NaN,NaN,NaN,mp855-MV411_IRF2BP_VHL_6h-POLII_total-r3,20200302_10_MP7868_S60_R1_001,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284,mp856,MV411_IRF2BP_VHL_6h,3,POLII_S2,NaN,NaN,NaN,mp856-MV411_IRF2BP_VHL_6h-POLII_S2-r3,20200302_11_MP7868_S61_R1_001,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [166]:
ls ../../data/bams

mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.fastq.gz
mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R2.fastq.gz
mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1.fastq.gz
mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R2.fastq.gz
mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1.fastq.gz
mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R2.fastq.gz
mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1.fastq.gz
mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R2.fastq.gz
mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1.fastq.gz
mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R2.fastq.gz
mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1.fastq.gz
mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R2.fastq.gz
mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1.fastq.gz
mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R2.fastq.gz
mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1.fastq.gz
mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R2.fastq.gz
mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1.fastq.gz
mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R2.fastq.gz
mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R1.fastq.gz
mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R2.fastq.gz
mp841-MV411_IRF2BP_VHL_6h-SMC1-r1_R1.fastq.gz
mp841-MV411_IRF2BP

In [167]:
df = {
"fastq_1": [],
"fastq_2": [],
"antibody": [],
"group": [],
"replicate": [],
"control": []
}
for val in h.grouped(a,2):
    row = gsheet[gsheet.id==val[0].split('/')[-1].split('-')[0]]
    df['group'].append(row['name'].values[0])
    df['replicate'].append(1)
    df['fastq_1'].append('bams4/'+val[0])
    df['fastq_2'].append('bams4/'+val[1])
    df['antibody'].append(row['protein'].values[0])
    df['control'].append("INPUT")
df['group'].append('INPUT')
df['replicate'].append(1)
df['fastq_1'].append('bams/mp845-MV411_IRF2BP2_-INPUT-r1_R1.fastq.gz')
df['fastq_2'].append('bams/mp845-MV411_IRF2BP2_-INPUT-r1_R2.fastq.gz')
df['antibody'].append("")
df['control'].append("")
df = pd.DataFrame(df)

In [168]:
df

,fastq_1,fastq_2,antibody,group,replicate,control
0,bams4/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1.fa...,bams4/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R2.fa...,MED1,mp846-MV411_IRF2BP_DMSO_6h-MED1-r4,1,INPUT
1,bams4/mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1.fa...,bams4/mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R2.fa...,MED1,mp847-MV411_IRF2BP_DMSO_6h-MED1-r5,1,INPUT
2,bams4/mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-...,bams4/mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-...,FLAG_IRF2BP2,mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3,1,INPUT
3,bams4/mp849-MV411_IRF2BP_DMSO_6h-POLII_total-r...,bams4/mp849-MV411_IRF2BP_DMSO_6h-POLII_total-r...,POLII_total,mp849-MV411_IRF2BP_DMSO_6h-POLII_total-r3,1,INPUT
4,bams4/mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R...,bams4/mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R...,POLII_S2,mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3,1,INPUT
5,bams4/mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R...,bams4/mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R...,POLII_S5,mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3,1,INPUT
6,bams4/mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1.fas...,bams4/mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R2.fas...,MED1,mp852-MV411_IRF2BP_VHL_6h-MED1-r4,1,INPUT
7,bams4/mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R1.fas...,bams4/mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R2.fas...,MED1,mp853-MV411_IRF2BP_VHL_6h-MED1-r5,1,INPUT
8,bams4/mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r...,bams4/mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r...,FLAG_IRF2BP2,mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3,1,INPUT
9,bams4/mp855-MV411_IRF2BP_VHL_6h-POLII_total-r3...,bams4/mp855-MV411_IRF2BP_VHL_6h-POLII_total-r3...,POLII_total,mp855-MV411_IRF2BP_VHL_6h-POLII_total-r3,1,INPUT


In [169]:
df[df.columns[[3,4,0,1,2,5]]].to_csv('../../data/design_rep4.csv',index=False)

In [ ]:
#process chips
! sudo ../nextflow run nf-core/chipseq --paired_end --seq_center 'DFCI' --email \
'jkobject@gmail.com' --narrow_peak --input design_rep4.csv --genome GRCh38 --skip_preseq \
--max_cpus 24 -profile docker -w work

In [238]:
ls ../../data/bams4

mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1.fastq.gz
mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R2.fastq.gz
mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1.fastq.gz
mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R2.fastq.gz
mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R1.fastq.gz
mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R2.fastq.gz
mp849-MV411_IRF2BP_DMSO_6h-POLII_total-r3_R1.fastq.gz
mp849-MV411_IRF2BP_DMSO_6h-POLII_total-r3_R2.fastq.gz
mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R1.fastq.gz
mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R2.fastq.gz
mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1.fastq.gz
mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R2.fastq.gz
mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1.fastq.gz
mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R2.fastq.gz
mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R1.fastq.gz
mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R2.fastq.gz
mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3_R1.fastq.gz
mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3_R2.fastq.gz
mp855-MV411_IRF2BP_VHL_6h-POLII_total-r3_R1.fastq.gz
mp855-MV411_IRF2BP_VHL_6h-POLII_total-r3_R2.f

In [ ]:
!gsutil -m cp -r results/* gs://amlproject/Chip/IRF2BP2_degraded_rep4/ && sudo rm -r work && mv results results4

In [236]:
ls ../../TrimGalore-0.6.5/trim_galore

../../TrimGalore-0.6.5/trim_galore*


In [6]:
# get scaling values
mappedreads, umappedreads_norm = chip.getSpikeInControlScales(refgenome="../../data/ref/reference_droso.fna",
fastQfolder='../../data/res4/',
pairedEnd=True, cores=12,
tofilter=True,
totrim=False,
tomap=True,
results="../../data/res4/",
pathtotrim_galore="../../TrimGalore-0.6.5/trim_galore")
mappedreads, umappedreads_norm

if paired_end, need to be name_*1, name_*2
using all files from folder
[('mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1_val_1.fq.gz', 'mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R2_val_2.fq.gz'), ('mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1_val_1.fq.gz', 'mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R2_val_2.fq.gz'), ('mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R1_val_1.fq.gz', 'mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R2_val_2.fq.gz'), ('mp849-MV411_IRF2BP_DMSO_6h-POLII_total-r3_R1_val_1.fq.gz', 'mp849-MV411_IRF2BP_DMSO_6h-POLII_total-r3_R2_val_2.fq.gz'), ('mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R1_val_1.fq.gz', 'mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R2_val_2.fq.gz'), ('mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1_val_1.fq.gz', 'mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R2_val_2.fq.gz'), ('mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1_val_1.fq.gz', 'mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R2_val_2.fq.gz'), ('mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R1_val_1.fq.gz', 'mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R2_val_2.fq.gz'), ('mp854-MV411_IRF2BP_V

> /home/jeremie/JKBio/epigenetics/ChIP_helper.py(902)getSpikeInControlScales()
    901         print("files need to be named: NAME.sorted.bam")
--> 902         fastqs = [file for file in fastqs if '.sorted.bam' == file[-11:]]
    903     mapped = {}

ipdb> unt
> /home/jeremie/JKBio/epigenetics/ChIP_helper.py(903)getSpikeInControlScales()
    902         fastqs = [file for file in fastqs if '.sorted.bam' == file[-11:]]
--> 903     mapped = {}
    904     norm = {}

ipdb> 
> /home/jeremie/JKBio/epigenetics/ChIP_helper.py(902)getSpikeInControlScales()
    901         print("files need to be named: NAME.sorted.bam")
--> 902         fastqs = [file for file in fastqs if '.sorted.bam' == file[-11:]]
    903     mapped = {}

ipdb> unt
> /home/jeremie/JKBio/epigenetics/ChIP_helper.py(903)getSpikeInControlScales()
    902         fastqs = [file for file in fastqs if '.sorted.bam' == file[-11:]]
--> 903     mapped = {}
    904     norm = {}

ipdb> 
> /home/jeremie/JKBio/epigenetics/ChIP_helper.py

({'mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1_val_1': 0.2990873533246415,
  'mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1_val_1': 0.20899663394552362,
  'mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R1_val_1': 1.0,
  'mp849-MV411_IRF2BP_DMSO_6h-POLII_total-r3_R1_val_1': 0.5203418741380562,
  'mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R1_val_1': 0.6130608143500059,
  'mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1_val_1': 0.4283750877664739,
  'mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1_val_1': 0.07861534637359585,
  'mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R1_val_1': 0.18853988706236116,
  'mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3_R1_val_1': 0.18696534756754876,
  'mp855-MV411_IRF2BP_VHL_6h-POLII_total-r3_R1_val_1': 0.2862047798943014,
  'mp856-MV411_IRF2BP_VHL_6h-POLII_S2-r3_R1_val_1': 0.37066001396034226,
  'mp857-MV411_IRF2BP_VHL_6h-POLII_S5-r3_R1_val_1': 0.6839730465721839},
 {'mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1_val_1': 575250,
  'mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1_val_1': 823219,
  'mp848-MV411_IRF2BP_DMSO_

computing scales from the excel sheet

In [39]:
scales = [[262850,1000000],
[902119,1000000],
[186965,1000000],
[550032,1000000],
[604605,1000000],
[1000000,626304]]

In [24]:
scales = [[1.0,0.2628507876],
[1.0,0.9021192519],
[1.0,0.1869653476],
[1.0,0.5500321887],
[1.0,0.6046056203],
[0.626304048,1.0]]

In [25]:
bams = ! ls ../../data/results4/bwa/mergedLibrary/*.bam
bams

['../../data/results4/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam',
 '../../data/results4/bwa/mergedLibrary/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1.mLb.clN.sorted.bam',
 '../../data/results4/bwa/mergedLibrary/mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1.mLb.clN.sorted.bam',
 '../../data/results4/bwa/mergedLibrary/mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R1.mLb.clN.sorted.bam',
 '../../data/results4/bwa/mergedLibrary/mp849-MV411_IRF2BP_DMSO_6h-POLII_total-r3_R1.mLb.clN.sorted.bam',
 '../../data/results4/bwa/mergedLibrary/mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R1.mLb.clN.sorted.bam',
 '../../data/results4/bwa/mergedLibrary/mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1.mLb.clN.sorted.bam',
 '../../data/results4/bwa/mergedLibrary/mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1.mLb.clN.sorted.bam',
 '../../data/results4/bwa/mergedLibrary/mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R1.mLb.clN.sorted.bam',
 '../../data/results4/bwa/mergedLibrary/mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3_R1.mLb.clN.sorted.bam',
 '../../dat

In [37]:
bams[6]

'mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1'

### on scalled data

In [12]:
mkdir ../../data/IRF2BP2_degraded_4/

In [26]:
bams = [bam1.split('/')[-1].split('.')[0] for bam1 in bams]

In [32]:
# diffPeak on scaled data
size=[ 208, 214, 207, 234, 296, 231]
for i in range(int(len(bams)/2)-1):
    name1 = bams[1+i]
    name2 = bams[7+i]
    print(name1,name2)
    chip.diffPeak(name1, name2, directory= "../../data/IRF2BP2_degraded_4/diffData/", res_directory='../../data/IRF2BP2_degraded_4/diffPeaks/', scaling1=scales[i][1], scaling2=scales[i][0], size=size[i])

mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1 mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1
doing differential peak binding
mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1 mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R1
doing differential peak binding
mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R1 mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3_R1
doing differential peak binding
mp849-MV411_IRF2BP_DMSO_6h-POLII_total-r3_R1 mp855-MV411_IRF2BP_VHL_6h-POLII_total-r3_R1
doing differential peak binding
mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R1 mp856-MV411_IRF2BP_VHL_6h-POLII_S2-r3_R1
doing differential peak binding


In [33]:
# diffPeak on scaled data
for i in range(int(len(bams[1:])/2)):
    name1 = bams[1+i]
    name2 = bams[7+i]
    chip.fullDiffPeak(name1,name2, control1='../../data/results4/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam', directory = "../../data/IRF2BP2_degraded_4/diffData/", res_directory = "../../data/IRF2BP2_degraded_4/diffPeaks/",pairedend=False)

[autoreload of JKBio.Helper failed: Traceback (most recent call last):
  File "/home/jeremie/.local/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/jeremie/.local/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "../../JKBio/Helper.py", line 569
    if len(add) != 0
        

doing diff from ../../data/results4/bwa/mergedLibrary/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1.mLb.clN.sorted.bam and ../../data/results4/bwa/mergedLibrary/mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1.mLb.clN.sorted.bam
computing the fragment avg size
208
computing the scaling values
b'INFO  @ Fri, 13 Mar 2020 18:48:38: \n# Command line: callpeak -B -t ../../data/results4/bwa/mergedLibrary/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1.mLb.clN.sorted.bam -c ../../data/results4/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 208 -n mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1 --outdir ../../data/IRF2BP2_degraded_4/diffData/ -f BAM\n# ARGUMENTS LIST:\n# name = mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1\n# format = BAM\n# ChIP-seq file = [\'../../data/results4/bwa/mergedLibrary/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/results4/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50

doing diff from ../../data/results4/bwa/mergedLibrary/mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1.mLb.clN.sorted.bam and ../../data/results4/bwa/mergedLibrary/mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R1.mLb.clN.sorted.bam
computing the fragment avg size
214
computing the scaling values
b'INFO  @ Fri, 13 Mar 2020 19:53:12: \n# Command line: callpeak -B -t ../../data/results4/bwa/mergedLibrary/mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1.mLb.clN.sorted.bam -c ../../data/results4/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 214 -n mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1 --outdir ../../data/IRF2BP2_degraded_4/diffData/ -f BAM\n# ARGUMENTS LIST:\n# name = mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1\n# format = BAM\n# ChIP-seq file = [\'../../data/results4/bwa/mergedLibrary/mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/results4/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50

doing diff from ../../data/results4/bwa/mergedLibrary/mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R1.mLb.clN.sorted.bam and ../../data/results4/bwa/mergedLibrary/mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3_R1.mLb.clN.sorted.bam
computing the fragment avg size
207
computing the scaling values
b'INFO  @ Fri, 13 Mar 2020 20:53:17: \n# Command line: callpeak -B -t ../../data/results4/bwa/mergedLibrary/mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R1.mLb.clN.sorted.bam -c ../../data/results4/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 207 -n mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R1 --outdir ../../data/IRF2BP2_degraded_4/diffData/ -f BAM\n# ARGUMENTS LIST:\n# name = mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R1\n# format = BAM\n# ChIP-seq file = [\'../../data/results4/bwa/mergedLibrary/mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/results4/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.7

b'INFO  @ Fri, 13 Mar 2020 21:13:03: \n# Command line: callpeak -B -t ../../data/results4/bwa/mergedLibrary/mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3_R1.mLb.clN.sorted.bam -c ../../data/results4/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 207 -n mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3_R1 --outdir ../../data/IRF2BP2_degraded_4/diffData/ -f BAM\n# ARGUMENTS LIST:\n# name = mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3_R1\n# format = BAM\n# ChIP-seq file = [\'../../data/results4/bwa/mergedLibrary/mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/results4/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# Larger dataset will be scaled towards smaller 

b'INFO  @ Fri, 13 Mar 2020 22:15:19: \n# Command line: callpeak -B -t ../../data/results4/bwa/mergedLibrary/mp855-MV411_IRF2BP_VHL_6h-POLII_total-r3_R1.mLb.clN.sorted.bam -c ../../data/results4/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 234 -n mp855-MV411_IRF2BP_VHL_6h-POLII_total-r3_R1 --outdir ../../data/IRF2BP2_degraded_4/diffData/ -f BAM\n# ARGUMENTS LIST:\n# name = mp855-MV411_IRF2BP_VHL_6h-POLII_total-r3_R1\n# format = BAM\n# ChIP-seq file = [\'../../data/results4/bwa/mergedLibrary/mp855-MV411_IRF2BP_VHL_6h-POLII_total-r3_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/results4/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# Larger dataset will be scaled towards smaller data

296
computing the scaling values
b'INFO  @ Fri, 13 Mar 2020 22:52:53: \n# Command line: callpeak -B -t ../../data/results4/bwa/mergedLibrary/mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R1.mLb.clN.sorted.bam -c ../../data/results4/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 296 -n mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R1 --outdir ../../data/IRF2BP2_degraded_4/diffData/ -f BAM\n# ARGUMENTS LIST:\n# name = mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R1\n# format = BAM\n# ChIP-seq file = [\'../../data/results4/bwa/mergedLibrary/mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/results4/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# Larger dataset will be sc

doing diff from ../../data/results4/bwa/mergedLibrary/mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1.mLb.clN.sorted.bam and ../../data/results4/bwa/mergedLibrary/mp857-MV411_IRF2BP_VHL_6h-POLII_S5-r3_R1.mLb.clN.sorted.bam
computing the fragment avg size
231
computing the scaling values
b'INFO  @ Fri, 13 Mar 2020 23:52:47: \n# Command line: callpeak -B -t ../../data/results4/bwa/mergedLibrary/mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1.mLb.clN.sorted.bam -c ../../data/results4/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 231 -n mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1 --outdir ../../data/IRF2BP2_degraded_4/diffData/ -f BAM\n# ARGUMENTS LIST:\n# name = mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1\n# format = BAM\n# ChIP-seq file = [\'../../data/results4/bwa/mergedLibrary/mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/results4/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 30

In [14]:
scales = [1.0,
1.0,
1.0,
1.0,
1.0,
0.626304048,
0.2628507876,
0.9021192519,
0.1869653476,
0.5500321887,
0.6046056203,
1.0]

In [227]:
bams

['../../data/diffBinding_hist/mp811-MV411_IRF2BP2_DMSO_6h-H3K27ac-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp812-MV411_IRF2BP2_DMSO_6h-H3K27ac-r2.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp813-MV411_IRF2BP2_DMSO_6h-H3K27me3-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp814-MV411_IRF2BP2_DMSO_6h-H3K27me3-r2.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp815-MV411_IRF2BP2_DMSO_6h-H3K4me1-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp816-MV411_IRF2BP2_DMSO_6h-H3K4me1-r2.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp817-MV411_IRF2BP2_DMSO_6h-H3K4me3-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp818-MV411_IRF2BP2_DMSO_6h-H3K4me3-r2.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp819-MV411_IRF2BP2_DMSO_6h-H3K79me2-r1.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp820-MV411_IRF2BP2_DMSO_6h-H3K79me2-r2.mLb.clN.sorted.bam',
 '../../data/diffBinding_hist/mp821-MV411_IRF2BP2_VHL_6h-H3K27ac-r1.mLb.clN.sorted.bam',
 '../..

In [18]:
chip.bigWigFrom(bams[1:], 
                genome='GRCh38',scaling=scales,
               numthreads=8)

In [20]:
!mv bigwig ../../data/recalib_bigwig_4 

In [33]:
os.popen('for i in $(ls ../../data/IRF2BP2_degraded_4/diffPeaks/*.bed); \
            do echo $(wc -l $i); \
            done').read().split('\n')

['439 ../../data/IRF2BP2_degraded_4/diffPeaks/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1_vs_mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1_c3.0_common.bed',
 '4403 ../../data/IRF2BP2_degraded_4/diffPeaks/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1_vs_mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1_c3.0_cond1.bed',
 '1 ../../data/IRF2BP2_degraded_4/diffPeaks/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1_vs_mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1_c3.0_cond2.bed',
 '2732 ../../data/IRF2BP2_degraded_4/diffPeaks/mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1_vs_mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R1_c3.0_common.bed',
 '393 ../../data/IRF2BP2_degraded_4/diffPeaks/mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1_vs_mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R1_c3.0_cond1.bed',
 '107 ../../data/IRF2BP2_degraded_4/diffPeaks/mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1_vs_mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R1_c3.0_cond2.bed',
 '48 ../../data/IRF2BP2_degraded_4/diffPeaks/mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R1_vs_mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3_R1_c3.0_common.b

In [27]:
bw = ! ls ../../data/recalib_bigwig_4/*.bw
bw

['../../data/recalib_bigwig_4/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1.bw',
 '../../data/recalib_bigwig_4/mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1.bw',
 '../../data/recalib_bigwig_4/mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R1.bw',
 '../../data/recalib_bigwig_4/mp849-MV411_IRF2BP_DMSO_6h-POLII_total-r3_R1.bw',
 '../../data/recalib_bigwig_4/mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R1.bw',
 '../../data/recalib_bigwig_4/mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1.bw',
 '../../data/recalib_bigwig_4/mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1.bw',
 '../../data/recalib_bigwig_4/mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R1.bw',
 '../../data/recalib_bigwig_4/mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3_R1.bw',
 '../../data/recalib_bigwig_4/mp855-MV411_IRF2BP_VHL_6h-POLII_total-r3_R1.bw',
 '../../data/recalib_bigwig_4/mp856-MV411_IRF2BP_VHL_6h-POLII_S2-r3_R1.bw',
 '../../data/recalib_bigwig_4/mp857-MV411_IRF2BP_VHL_6h-POLII_S5-r3_R1.bw']

In [28]:
cond1peak = ! ls ../../data/IRF2BP2_degraded_4/diffPeaks/*cond1.bed
cond2peak = ! ls ../../data/IRF2BP2_degraded_4/diffPeaks/*cond2.bed
commonpeak = ! ls ../../data/IRF2BP2_degraded_4/diffPeaks/*common.bed
cond1peak

['../../data/IRF2BP2_degraded_4/diffPeaks/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1_vs_mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_4/diffPeaks/mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1_vs_mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R1_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_4/diffPeaks/mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R1_vs_mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3_R1_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_4/diffPeaks/mp849-MV411_IRF2BP_DMSO_6h-POLII_total-r3_R1_vs_mp855-MV411_IRF2BP_VHL_6h-POLII_total-r3_R1_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_4/diffPeaks/mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R1_vs_mp856-MV411_IRF2BP_VHL_6h-POLII_S2-r3_R1_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_4/diffPeaks/mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1_vs_mp857-MV411_IRF2BP_VHL_6h-POLII_S5-r3_R1_c3.0_cond1.bed']

In [29]:
names = ["MED1","MED1_v2","FLAG_IRF2BP2","POLII_total","POLII_S2","POLII_S5"]

In [34]:
for i in range(int(len(bw)/2)):
    if i <0:
        continue
    name1 = bw[i]
    name2 = bw[i+6]
    peak = [cond1peak[i], commonpeak[i], cond2peak[i]]
   # for val in peak:
      #  chip.dropWeirdChromosomes(val)
    name = names[i]
    print(name1,name2)
    chip.computePeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], window=3000, folder="", title=name, numthreads=7, refpoint="center", name='../../data/IRF2BP2_degraded_4/diffPeaks/'+name+'_mat.pdf', withDeeptools=True, torecompute=True)

../../data/recalib_bigwig_4/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1.bw ../../data/recalib_bigwig_4/mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1.bw
CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_4/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1.bw ../../data/recalib_bigwig_4/mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1.bw  --referencePoint center --regionsFileName ../../data/IRF2BP2_degraded_4/diffPeaks/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1_vs_mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1_c3.0_cond1.bed ../../data/IRF2BP2_degraded_4/diffPeaks/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1_vs_mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1_c3.0_common.bed  --missingDataAsZero --outFileName ../../data/IRF2BP2_degraded_4/diffPeaks/MED1_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotHeatmap --matrixFile ../../data/IRF2BP2_degraded_4/diffPeaks/MED1_mat.gz --outFileName ../../data/IRF2BP2_degraded_4/diffPeaks/MED1_mat.pdf --refPointLabel center --regionsLabel DMSO_peaks common  --samplesLab

In [234]:
peaks = ! ls ../../data/results4/bwa/mergedLibrary/macs/narrowPeak/*.narrowPeak
peaks

['../../data/results4/bwa/mergedLibrary/macs/narrowPeak/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1_peaks.narrowPeak',
 '../../data/results4/bwa/mergedLibrary/macs/narrowPeak/mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1_peaks.narrowPeak',
 '../../data/results4/bwa/mergedLibrary/macs/narrowPeak/mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R1_peaks.narrowPeak',
 '../../data/results4/bwa/mergedLibrary/macs/narrowPeak/mp849-MV411_IRF2BP_DMSO_6h-POLII_total-r3_R1_peaks.narrowPeak',
 '../../data/results4/bwa/mergedLibrary/macs/narrowPeak/mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R1_peaks.narrowPeak',
 '../../data/results4/bwa/mergedLibrary/macs/narrowPeak/mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1_peaks.narrowPeak',
 '../../data/results4/bwa/mergedLibrary/macs/narrowPeak/mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1_peaks.narrowPeak',
 '../../data/results4/bwa/mergedLibrary/macs/narrowPeak/mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R1_peaks.narrowPeak',
 '../../data/results4/bwa/mergedLibrary/macs/narrowPeak/mp854-MV411_IRF2BP_

In [232]:
names = ["MED1","MED1_v2","FLAG_IRF2BP2","POLII_total","POLII_S2","POLII_S5"]
names.extend([i+'_VHL' for i in names])
names

['MED1',
 'MED1_v2',
 'FLAG_IRF2BP2',
 'POLII_total',
 'POLII_S2',
 'POLII_S5',
 'MED1_VHL',
 'MED1_v2_VHL',
 'FLAG_IRF2BP2_VHL',
 'POLII_total_VHL',
 'POLII_S2_VHL',
 'POLII_S5_VHL']

In [235]:
for i in range(len(bw)):
    chip.computePeaksAt(peaks[i], bw[i], window=3000, folder="", title=names[i], numthreads=7, torecompute=True, onlyProfile=True, name='../../data/IRF2BP2_degraded_4/diffPeaks/'+names[i]+'_mat_profile.pdf', withDeeptools=True, refpoint="center")
    chip.computePeaksAt(peaks[i], bw[i], window=3000, folder="", title=names[i], numthreads=7, torecompute=True, onlyProfile=True,name='../../data/IRF2BP2_degraded_4/diffPeaks/'+names[i]+'_mat_profile_clust3.pdf', withDeeptools=True, cluster=3, refpoint="center")

CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_4/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1.bw  --referencePoint center --regionsFileName ../../data/results4/bwa/mergedLibrary/macs/narrowPeak/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1_peaks.narrowPeak  --missingDataAsZero --outFileName ../../data/IRF2BP2_degraded_4/diffPeaks/MED1_mat_profile.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotProfile --matrixFile ../../data/IRF2BP2_degraded_4/diffPeaks/MED1_mat_profile.gz --outFileName ../../data/IRF2BP2_degraded_4/diffPeaks/MED1_mat_profile.pdf --refPointLabel center --plotTitle MED1', returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_4/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1.bw  --referencePoint center --regionsFileName ../../data/results4/bwa/mergedLibrary/macs/narrowPeak/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1_peaks.narrowPeak  --missingDataAsZero --outFileName ../../data/IRF2B

CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_4/mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1.bw  --referencePoint center --regionsFileName ../../data/results4/bwa/mergedLibrary/macs/narrowPeak/mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1_peaks.narrowPeak  --missingDataAsZero --outFileName ../../data/IRF2BP2_degraded_4/diffPeaks/MED1_VHL_mat_profile.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotProfile --matrixFile ../../data/IRF2BP2_degraded_4/diffPeaks/MED1_VHL_mat_profile.gz --outFileName ../../data/IRF2BP2_degraded_4/diffPeaks/MED1_VHL_mat_profile.pdf --refPointLabel center --plotTitle MED1_VHL', returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_4/mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1.bw  --referencePoint center --regionsFileName ../../data/results4/bwa/mergedLibrary/macs/narrowPeak/mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1_peaks.narrowPeak  --missingDataAsZero --outFileName ../.

### on unscalled data

In [59]:
bams = !ls ../../data/results4/bwa/mergedLibrary/mp*.bam
bams

['../../data/results4/bwa/mergedLibrary/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1.mLb.clN.sorted.bam',
 '../../data/results4/bwa/mergedLibrary/mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1.mLb.clN.sorted.bam',
 '../../data/results4/bwa/mergedLibrary/mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R1.mLb.clN.sorted.bam',
 '../../data/results4/bwa/mergedLibrary/mp849-MV411_IRF2BP_DMSO_6h-POLII_total-r3_R1.mLb.clN.sorted.bam',
 '../../data/results4/bwa/mergedLibrary/mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R1.mLb.clN.sorted.bam',
 '../../data/results4/bwa/mergedLibrary/mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1.mLb.clN.sorted.bam',
 '../../data/results4/bwa/mergedLibrary/mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1.mLb.clN.sorted.bam',
 '../../data/results4/bwa/mergedLibrary/mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R1.mLb.clN.sorted.bam',
 '../../data/results4/bwa/mergedLibrary/mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3_R1.mLb.clN.sorted.bam',
 '../../data/results4/bwa/mergedLibrary/mp855-MV411_IRF2BP_VHL_6h-POLII_total-r3_

In [57]:
! mkdir ../../data/IRF2BP2_degraded_4/diffPeaks_unscaled

In [76]:
#on unscalled data 
for i in range(int(len(bams)/2)):
    if i < 5:
        continue
    name1 = bams[i]
    name2 = bams[6+i]
    print(name1,name2)
    chip.fullDiffPeak(name1,name2, control1='../../data/results4/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam', directory = "../../data/IRF2BP2_degraded_4/diffData_unscaled/", res_directory = "../../data/IRF2BP2_degraded_4/diffPeaks_unscaled/",pairedend=False)

../../data/results4/bwa/mergedLibrary/mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1.mLb.clN.sorted.bam ../../data/results4/bwa/mergedLibrary/mp857-MV411_IRF2BP_VHL_6h-POLII_S5-r3_R1.mLb.clN.sorted.bam
doing diff from ../../data/results4/bwa/mergedLibrary/mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1.mLb.clN.sorted.bam and ../../data/results4/bwa/mergedLibrary/mp857-MV411_IRF2BP_VHL_6h-POLII_S5-r3_R1.mLb.clN.sorted.bam
computing the fragment avg size
231
computing the scaling values
b'INFO  @ Tue, 17 Mar 2020 13:57:47: \n# Command line: callpeak -B -t ../../data/results4/bwa/mergedLibrary/mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1.mLb.clN.sorted.bam -c ../../data/results4/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 231 -n mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1 --outdir ../../data/IRF2BP2_degraded_4/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1\n# format = BAM\n# ChIP-seq file = [\'../../data/results4/bwa/mergedLibrary/mp

b'INFO  @ Tue, 17 Mar 2020 14:15:18: \n# Command line: callpeak -B -t ../../data/results4/bwa/mergedLibrary/mp857-MV411_IRF2BP_VHL_6h-POLII_S5-r3_R1.mLb.clN.sorted.bam -c ../../data/results4/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 231 -n mp857-MV411_IRF2BP_VHL_6h-POLII_S5-r3_R1 --outdir ../../data/IRF2BP2_degraded_4/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = mp857-MV411_IRF2BP_VHL_6h-POLII_S5-r3_R1\n# format = BAM\n# ChIP-seq file = [\'../../data/results4/bwa/mergedLibrary/mp857-MV411_IRF2BP_VHL_6h-POLII_S5-r3_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/results4/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# Larger dataset will be scaled towards smaller dataset

In [236]:
bw = ! ls ../../data/results4/bwa/mergedLibrary/bigwig/*.bigWig
bw

['../../data/results4/bwa/mergedLibrary/bigwig/INPUT_R1.mLb.clN.bigWig',
 '../../data/results4/bwa/mergedLibrary/bigwig/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1.mLb.clN.bigWig',
 '../../data/results4/bwa/mergedLibrary/bigwig/mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1.mLb.clN.bigWig',
 '../../data/results4/bwa/mergedLibrary/bigwig/mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R1.mLb.clN.bigWig',
 '../../data/results4/bwa/mergedLibrary/bigwig/mp849-MV411_IRF2BP_DMSO_6h-POLII_total-r3_R1.mLb.clN.bigWig',
 '../../data/results4/bwa/mergedLibrary/bigwig/mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R1.mLb.clN.bigWig',
 '../../data/results4/bwa/mergedLibrary/bigwig/mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1.mLb.clN.bigWig',
 '../../data/results4/bwa/mergedLibrary/bigwig/mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1.mLb.clN.bigWig',
 '../../data/results4/bwa/mergedLibrary/bigwig/mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R1.mLb.clN.bigWig',
 '../../data/results4/bwa/mergedLibrary/bigwig/mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3_R

In [237]:
cond1peak = ! ls ../../data/IRF2BP2_degraded_4/diffPeaks_unscaled/*cond1.bed
cond2peak = ! ls ../../data/IRF2BP2_degraded_4/diffPeaks_unscaled/*cond2.bed
commonpeak = ! ls ../../data/IRF2BP2_degraded_4/diffPeaks_unscaled/*common.bed
commonpeak

['../../data/IRF2BP2_degraded_4/diffPeaks_unscaled/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1_vs_mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1_c3.0_common.bed',
 '../../data/IRF2BP2_degraded_4/diffPeaks_unscaled/mp847-MV411_IRF2BP_DMSO_6h-MED1-r5_R1_vs_mp853-MV411_IRF2BP_VHL_6h-MED1-r5_R1_c3.0_common.bed',
 '../../data/IRF2BP2_degraded_4/diffPeaks_unscaled/mp848-MV411_IRF2BP_DMSO_6h-FLAG_IRF2BP2-r3_R1_vs_mp854-MV411_IRF2BP_VHL_6h-FLAG_IRF2BP2-r3_R1_c3.0_common.bed',
 '../../data/IRF2BP2_degraded_4/diffPeaks_unscaled/mp849-MV411_IRF2BP_DMSO_6h-POLII_total-r3_R1_vs_mp855-MV411_IRF2BP_VHL_6h-POLII_total-r3_R1_c3.0_common.bed',
 '../../data/IRF2BP2_degraded_4/diffPeaks_unscaled/mp850-MV411_IRF2BP_DMSO_6h-POLII_S2-r3_R1_vs_mp856-MV411_IRF2BP_VHL_6h-POLII_S2-r3_R1_c3.0_common.bed',
 '../../data/IRF2BP2_degraded_4/diffPeaks_unscaled/mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1_vs_mp857-MV411_IRF2BP_VHL_6h-POLII_S5-r3_R1_c3.0_common.bed']

In [238]:
names = ["MED1","MED1_v2","FLAG_IRF2BP2","POLII_total","POLII_S2","POLII_S5"]

In [239]:
for i in range(int((len(bw)-1)/2)):
    name1 = bw[1+i]
    name2 = bw[7+i]
    peak = [cond1peak[i], commonpeak[i], cond2peak[i]]
    #for val in peak:
     #   chip.dropWeirdChromosomes(val)
    name = names[i]
    print(name1,name2)
    chip.computePeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], window=3000, folder="", title=name, numthreads=7, torecompute=True, refpoint='center', name='../../data/IRF2BP2_degraded_4/diffPeaks_unscaled/'+name+'_mat.pdf', withDeeptools=True)

../../data/results4/bwa/mergedLibrary/bigwig/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1.mLb.clN.bigWig ../../data/results4/bwa/mergedLibrary/bigwig/mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1.mLb.clN.bigWig
CompletedProcess(args='computeMatrix reference-point -S ../../data/results4/bwa/mergedLibrary/bigwig/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1.mLb.clN.bigWig ../../data/results4/bwa/mergedLibrary/bigwig/mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1.mLb.clN.bigWig  --referencePoint center --regionsFileName ../../data/IRF2BP2_degraded_4/diffPeaks_unscaled/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1_vs_mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1_c3.0_cond1.bed ../../data/IRF2BP2_degraded_4/diffPeaks_unscaled/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1_vs_mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1_c3.0_common.bed ../../data/IRF2BP2_degraded_4/diffPeaks_unscaled/mp846-MV411_IRF2BP_DMSO_6h-MED1-r4_R1_vs_mp852-MV411_IRF2BP_VHL_6h-MED1-r4_R1_c3.0_cond2.bed  --missingDataAsZero --outFileName ../../data/IRF2BP2_degraded_4/diffPeaks_unscaled/

CompletedProcess(args='computeMatrix reference-point -S ../../data/results4/bwa/mergedLibrary/bigwig/mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1.mLb.clN.bigWig ../../data/results4/bwa/mergedLibrary/bigwig/mp857-MV411_IRF2BP_VHL_6h-POLII_S5-r3_R1.mLb.clN.bigWig  --referencePoint center --regionsFileName ../../data/IRF2BP2_degraded_4/diffPeaks_unscaled/mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1_vs_mp857-MV411_IRF2BP_VHL_6h-POLII_S5-r3_R1_c3.0_cond1.bed ../../data/IRF2BP2_degraded_4/diffPeaks_unscaled/mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1_vs_mp857-MV411_IRF2BP_VHL_6h-POLII_S5-r3_R1_c3.0_common.bed ../../data/IRF2BP2_degraded_4/diffPeaks_unscaled/mp851-MV411_IRF2BP_DMSO_6h-POLII_S5-r3_R1_vs_mp857-MV411_IRF2BP_VHL_6h-POLII_S5-r3_R1_c3.0_cond2.bed  --missingDataAsZero --outFileName ../../data/IRF2BP2_degraded_4/diffPeaks_unscaled/POLII_S5_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotHeatmap --matrixFile ../../data/IRF2BP2_degraded_4/diffPeaks_unscaled/POLII_S5_mat.

In [ ]:
! gsutil -m cp -r "../../data/recalib_bigwig_3/*" gs://amlproject/Chip/IRF2BP2_degraded_rep3/recalib_bigwig/ &&\
gsutil -m cp -r "../../data/IRF2BP2_degraded_3/diffData/*" gs://amlproject/Chip/IRF2BP2_degraded_rep3/diffData/ &&\
gsutil -m cp -r "../../data/IRF2BP2_degraded_3/diffPeaks/*" gs://amlproject/Chip/IRF2BP2_degraded_rep3/diffPeaks/

## v3

In [ ]:
! gsutil mv gs://transfer-amlproject/*MP7781*  gs://transfer-amlproject/IRF2BP2_v3/

In [ ]:
ls

### analysis

In [ ]:
! mkdir bams && gsutil -m cp gs://transfer-amlproject/IRF2BP2_v3/* bams/

In [ ]:
cd bams

In [ ]:
rename = {
"20200203_1_MP7781_S67_R1_001.fastq.gz":"mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.fastq.gz",
"20200203_1_MP7781_S67_R2_001.fastq.gz":"mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R2.fastq.gz",
"20200203_2_MP7781_S68_R1_001.fastq.gz":"mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1.fastq.gz",
"20200203_2_MP7781_S68_R2_001.fastq.gz":"mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R2.fastq.gz",
"20200203_3_MP7781_S69_R1_001.fastq.gz":"mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1.fastq.gz",
"20200203_3_MP7781_S69_R2_001.fastq.gz":"mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R2.fastq.gz",
"20200203_4_MP7781_S70_R1_001.fastq.gz":"mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1.fastq.gz",
"20200203_4_MP7781_S70_R2_001.fastq.gz":"mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R2.fastq.gz",
"20200203_5_MP7781_S71_R2_001.fastq.gz":"mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R2.fastq.gz",
"20200203_5_MP7781_S71_R1_001.fastq.gz":"mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1.fastq.gz",
"20200203_6_MP7781_S72_R1_001.fastq.gz":"mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1.fastq.gz",
"20200203_6_MP7781_S72_R2_001.fastq.gz":"mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R2.fastq.gz",
"20200203_7_MP7781_S73_R1_001.fastq.gz":"mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1.fastq.gz",
"20200203_7_MP7781_S73_R2_001.fastq.gz":"mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R2.fastq.gz",
"20200203_8_MP7781_S74_R1_001.fastq.gz":"mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1.fastq.gz",
"20200203_8_MP7781_S74_R2_001.fastq.gz":"mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R2.fastq.gz",
"20200203_9_MP7781_S75_R1_001.fastq.gz":"mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1.fastq.gz",
"20200203_9_MP7781_S75_R2_001.fastq.gz":"mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R2.fastq.gz",
"20200203_10_MP7781_S76_R2_001.fastq.gz":"mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R2.fastq.gz",
"20200203_10_MP7781_S76_R1_001.fastq.gz":"mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R1.fastq.gz",
"20200203_11_MP7781_S77_R1_001.fastq.gz":"mp841-MV411_IRF2BP_VHL_6h-SMC1-r1_R1.fastq.gz",
"20200203_11_MP7781_S77_R2_001.fastq.gz":"mp841-MV411_IRF2BP_VHL_6h-SMC1-r1_R2.fastq.gz",
"20200203_12_MP7781_S78_R1_001.fastq.gz":"mp842-MV411_IRF2BP_VHL_6h-MED1-r3_R1.fastq.gz",
"20200203_12_MP7781_S78_R2_001.fastq.gz":"mp842-MV411_IRF2BP_VHL_6h-MED1-r3_R2.fastq.gz",
"20200203_13_MP7781_S79_R1_001.fastq.gz":"mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1_R1.fastq.gz",
"20200203_13_MP7781_S79_R2_001.fastq.gz":"mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1_R2.fastq.gz",
"20200203_14_MP7781_S80_R2_001.fastq.gz":"mp844-MV411_IRF2BP_VHL_6h-CEBPA-r1_R2.fastq.gz",
"20200203_14_MP7781_S80_R1_001.fastq.gz":"mp844-MV411_IRF2BP_VHL_6h-CEBPA-r1_R1.fastq.gz",
"20200203_Input_MP7781_S81_R1_001.fastq.gz":"mp845-MV411_IRF2BP2_-INPUT-r1_R1.fastq.gz",
"20200203_Input_MP7781_S81_R2_001.fastq.gz":"mp845-MV411_IRF2BP2_-INPUT-r1_R2.fastq.gz"
}

In [ ]:
for k,v in rename.items():
    ! mv $k $v

In [ ]:
ls

In [ ]:
cd ..

In [ ]:
! gsutil ls gs://amlproject/Chip_AML/IRF2BP2/

In [ ]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
url="https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U"
gsheet = sheets.get(url).sheets[2].to_frame()

In [ ]:
a = ! ls bams

In [ ]:
gsheet

In [ ]:
df = {
"fastq_1": [],
"fastq_2": [],
"antibody": [],
"group": [],
"replicate": [],
"control": []
}
for val in helper.grouped(a[:-2],2):
    row = gsheet[gsheet.id==val[0].split('/')[-1].split('-')[0]]
    df['group'].append(row['name'].values[0])
    df['replicate'].append(1)
    df['fastq_1'].append('bams/'+val[0])
    df['fastq_2'].append('bams/'+val[1])
    df['antibody'].append(row['protein'].values[0])
    df['control'].append("INPUT")
df['group'].append('INPUT')
df['replicate'].append(1)
df['fastq_1'].append('bams/'+a[-2])
df['fastq_2'].append('bams/'+a[-1])
df['antibody'].append("")
df['control'].append("")
df = pd.DataFrame(df)

In [ ]:
df

In [ ]:
df[df.columns[[3,4,0,1,2,5]]].to_csv('design.csv',index=False)

In [ ]:
#process chips
! sudo ../nextflow run nf-core/chipseq --paired_end --seq_center 'DFCI' --email \
'jkobject@gmail.com' --narrow_peak --input design.csv --genome GRCh38 --skip_preseq \
--max_cpus 24 -profile docker -w work

In [ ]:
ls

In [ ]:
!gsutil -m cp -r results/* gs://amlproject/Chip/IRF2BP2_degraded_rep3/ && sudo rm -r work && mv results results3

In [ ]:
ls ../TrimGalore-0.6.5/trim_galore

In [ ]:
# get scaling values
mappedreads, umappedreads_norm = chip.getSpikeInControlScales(refgenome="ref/reference_droso.fna",
                                                                      fastQfolder='res/',
                                                                      pairedEnd=True, cores=8,
                                                                      tofilter=False,
                                                                      totrim=False,
                                                                      tomap=False,
                                                                      pathtotrim_galore="../TrimGalore-0.6.5/trim_galore")
mappedreads, umappedreads_norm

computing scales from the excel sheet

In [297]:
scales = [[536923,632558],
[601370,681405],
[2402198,1676203],
[417892,216192],
[1544590,1350802],
[1174994,1376726],
[289635,240366]]

In [7]:
scales= [[1.0, 0.8488122828],
[1.0, 0.8825441551],
[0.6977788675, 1.0],
[0.5173394083, 1.0],
[0.8745375796, 1.0],
[1.0, 0.8534697536],
[0.8298927961, 1.0]]

In [9]:
bams = ! ls ../../data/IRF2BP2_degraded_3/*.bam #../../data/results3/bwa/mergedLibrary/*.bam
bams

['../../data/IRF2BP2_degraded_3/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.mLb.clN.sorted.bam',
 '../../data/IRF2BP2_degraded_3/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1.mLb.clN.sorted.bam',
 '../../data/IRF2BP2_degraded_3/mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1.mLb.clN.sorted.bam',
 '../../data/IRF2BP2_degraded_3/mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1.mLb.clN.sorted.bam',
 '../../data/IRF2BP2_degraded_3/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1.mLb.clN.sorted.bam',
 '../../data/IRF2BP2_degraded_3/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1.mLb.clN.sorted.bam',
 '../../data/IRF2BP2_degraded_3/mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1.mLb.clN.sorted.bam',
 '../../data/IRF2BP2_degraded_3/mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1.mLb.clN.sorted.bam',
 '../../data/IRF2BP2_degraded_3/mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1.mLb.clN.sorted.bam',
 '../../data/IRF2BP2_degraded_3/mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R1.mLb.clN.sorted.bam',
 '../../data/IRF2BP2_degraded_3/mp841-MV411_IRF2BP_VHL_6h-SMC1-r1_R1.mLb.clN.sorted.bam',
 '

In [241]:
! gsutil cp gs://amlproject/IRF2BP2_degraded_rep3/bwa/mergedLibrary/*.bam ../../data/IRF2BP2_degraded_3/

CommandException: No URLs matched: gs://amlproject/IRF2BP2_degraded_rep3/bwa/mergedLibrary/*.bam


In [12]:
bams[7]

'mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1'

In [11]:
bams = [bam1.split('/')[-1].split('.')[0] for bam1 in bams]

### on scalled data

In [20]:
# diffPeak on scaled data
size=[206, 218, 189, 194, 217, 217, 176]
for i in range(int(len(bams)/2)):
    if i<0:
        continue
    name1 = bams[i]
    name2 = bams[7+i]
    print(name1,name2)
    chip.diffPeak(name1, name2, directory= "../../data/IRF2BP2_degraded_3/diffData/", res_directory='../../data/IRF2BP2_degraded_3/diffPeaks/', scaling1=scales[i][1], scaling2=scales[i][0], size=size[i])

mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1 mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1
doing differential peak binding
mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1 mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1
doing differential peak binding
mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1 mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R1
doing differential peak binding
mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1 mp841-MV411_IRF2BP_VHL_6h-SMC1-r1_R1
doing differential peak binding
mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1 mp842-MV411_IRF2BP_VHL_6h-MED1-r3_R1
doing differential peak binding
mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1 mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1_R1
doing differential peak binding
mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1 mp844-MV411_IRF2BP_VHL_6h-CEBPA-r1_R1
doing differential peak binding


In [21]:
os.popen('for i in $(ls ../../data/IRF2BP2_degraded_3/diffPeaks/*.bed); \
            do echo $(wc -l $i); \
            done').read().split('\n')

['9562 ../../data/IRF2BP2_degraded_3/diffPeaks/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_vs_mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1_c3.0_common.bed',
 '573 ../../data/IRF2BP2_degraded_3/diffPeaks/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_vs_mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1_c3.0_cond1.bed',
 '380 ../../data/IRF2BP2_degraded_3/diffPeaks/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_vs_mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1_c3.0_cond2.bed',
 '24160 ../../data/IRF2BP2_degraded_3/diffPeaks/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1_vs_mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1_c3.0_common.bed',
 '762 ../../data/IRF2BP2_degraded_3/diffPeaks/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1_vs_mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1_c3.0_cond1.bed',
 '820 ../../data/IRF2BP2_degraded_3/diffPeaks/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1_vs_mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1_c3.0_cond2.bed',
 '48157 ../../data/IRF2BP2_degraded_3/diffPeaks/mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1_vs_mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R1_c3.0_common.bed',
 '9 .

In [250]:
# diffPeak on scaled data
for i in range(int(len(bams)/2)):
    if i <0:
        continue
    name1 = bams[i]
    name2 = bams[7+i]
    print(name1,name2)
    chip.fullDiffPeak(name1,name2, control1='../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam', directory = "../../data/IRF2BP2_degraded_3/diffData/", res_directory = "../../data/IRF2BP2_degraded_3/diffPeaks/", isTF=True, compute_size=True, pairedend=False)

../../data/IRF2BP2_degraded_3/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.mLb.clN.sorted.bam ../../data/IRF2BP2_degraded_3/mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1.mLb.clN.sorted.bam
doing diff from ../../data/IRF2BP2_degraded_3/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.mLb.clN.sorted.bam and ../../data/IRF2BP2_degraded_3/mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1.mLb.clN.sorted.bam
computing the fragment avg size
206
computing the scaling values
b'INFO  @ Thu, 26 Mar 2020 19:49:51: \n# Command line: callpeak -B -t ../../data/IRF2BP2_degraded_3/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.mLb.clN.sorted.bam -c ../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 206 -n mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1 --outdir ../../data/IRF2BP2_degraded_3/diffData/ -f BAM\n# ARGUMENTS LIST:\n# name = mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1\n# format = BAM\n# ChIP-seq file = [\'../../data/IRF2BP2_degraded_3/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffB

../../data/IRF2BP2_degraded_3/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1.mLb.clN.sorted.bam ../../data/IRF2BP2_degraded_3/mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1.mLb.clN.sorted.bam
doing diff from ../../data/IRF2BP2_degraded_3/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1.mLb.clN.sorted.bam and ../../data/IRF2BP2_degraded_3/mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1.mLb.clN.sorted.bam
computing the fragment avg size
218
computing the scaling values
b'INFO  @ Thu, 26 Mar 2020 20:19:37: \n# Command line: callpeak -B -t ../../data/IRF2BP2_degraded_3/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1.mLb.clN.sorted.bam -c ../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 218 -n mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1 --outdir ../../data/IRF2BP2_degraded_3/diffData/ -f BAM\n# ARGUMENTS LIST:\n# name = mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1\n# format = BAM\n# ChIP-seq file = [\'../../data/IRF2BP2_degraded_3/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffB

../../data/IRF2BP2_degraded_3/mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1.mLb.clN.sorted.bam ../../data/IRF2BP2_degraded_3/mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R1.mLb.clN.sorted.bam
doing diff from ../../data/IRF2BP2_degraded_3/mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1.mLb.clN.sorted.bam and ../../data/IRF2BP2_degraded_3/mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R1.mLb.clN.sorted.bam
computing the fragment avg size
189
computing the scaling values
b'INFO  @ Thu, 26 Mar 2020 20:49:23: \n# Command line: callpeak -B -t ../../data/IRF2BP2_degraded_3/mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1.mLb.clN.sorted.bam -c ../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 189 -n mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1 --outdir ../../data/IRF2BP2_degraded_3/diffData/ -f BAM\n# ARGUMENTS LIST:\n# name = mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1\n# format = BAM\n# ChIP-seq file = [\'../../data/IRF2BP2_degraded_3/mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffB

../../data/IRF2BP2_degraded_3/mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1.mLb.clN.sorted.bam ../../data/IRF2BP2_degraded_3/mp841-MV411_IRF2BP_VHL_6h-SMC1-r1_R1.mLb.clN.sorted.bam
doing diff from ../../data/IRF2BP2_degraded_3/mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1.mLb.clN.sorted.bam and ../../data/IRF2BP2_degraded_3/mp841-MV411_IRF2BP_VHL_6h-SMC1-r1_R1.mLb.clN.sorted.bam
computing the fragment avg size
194
computing the scaling values
b'INFO  @ Thu, 26 Mar 2020 21:16:12: \n# Command line: callpeak -B -t ../../data/IRF2BP2_degraded_3/mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1.mLb.clN.sorted.bam -c ../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 194 -n mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1 --outdir ../../data/IRF2BP2_degraded_3/diffData/ -f BAM\n# ARGUMENTS LIST:\n# name = mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1\n# format = BAM\n# ChIP-seq file = [\'../../data/IRF2BP2_degraded_3/mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffB

../../data/IRF2BP2_degraded_3/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1.mLb.clN.sorted.bam ../../data/IRF2BP2_degraded_3/mp842-MV411_IRF2BP_VHL_6h-MED1-r3_R1.mLb.clN.sorted.bam
doing diff from ../../data/IRF2BP2_degraded_3/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1.mLb.clN.sorted.bam and ../../data/IRF2BP2_degraded_3/mp842-MV411_IRF2BP_VHL_6h-MED1-r3_R1.mLb.clN.sorted.bam
computing the fragment avg size
217
computing the scaling values
b'INFO  @ Thu, 26 Mar 2020 21:45:08: \n# Command line: callpeak -B -t ../../data/IRF2BP2_degraded_3/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1.mLb.clN.sorted.bam -c ../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 217 -n mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1 --outdir ../../data/IRF2BP2_degraded_3/diffData/ -f BAM\n# ARGUMENTS LIST:\n# name = mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1\n# format = BAM\n# ChIP-seq file = [\'../../data/IRF2BP2_degraded_3/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffB

../../data/IRF2BP2_degraded_3/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1.mLb.clN.sorted.bam ../../data/IRF2BP2_degraded_3/mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1_R1.mLb.clN.sorted.bam
doing diff from ../../data/IRF2BP2_degraded_3/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1.mLb.clN.sorted.bam and ../../data/IRF2BP2_degraded_3/mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1_R1.mLb.clN.sorted.bam
computing the fragment avg size
217
computing the scaling values
b'INFO  @ Thu, 26 Mar 2020 22:11:18: \n# Command line: callpeak -B -t ../../data/IRF2BP2_degraded_3/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1.mLb.clN.sorted.bam -c ../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 217 -n mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1 --outdir ../../data/IRF2BP2_degraded_3/diffData/ -f BAM\n# ARGUMENTS LIST:\n# name = mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1\n# format = BAM\n# ChIP-seq file = [\'../../data/IRF2BP2_degraded_3/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/diffB

../../data/IRF2BP2_degraded_3/mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1.mLb.clN.sorted.bam ../../data/IRF2BP2_degraded_3/mp844-MV411_IRF2BP_VHL_6h-CEBPA-r1_R1.mLb.clN.sorted.bam
doing diff from ../../data/IRF2BP2_degraded_3/mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1.mLb.clN.sorted.bam and ../../data/IRF2BP2_degraded_3/mp844-MV411_IRF2BP_VHL_6h-CEBPA-r1_R1.mLb.clN.sorted.bam
computing the fragment avg size
176
computing the scaling values
b'INFO  @ Thu, 26 Mar 2020 22:37:13: \n# Command line: callpeak -B -t ../../data/IRF2BP2_degraded_3/mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1.mLb.clN.sorted.bam -c ../../data/diffBinding_hist/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 176 -n mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1 --outdir ../../data/IRF2BP2_degraded_3/diffData/ -f BAM\n# ARGUMENTS LIST:\n# name = mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1\n# format = BAM\n# ChIP-seq file = [\'../../data/IRF2BP2_degraded_3/mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../da

In [ ]:
scales = [1.0,
1.0,
0.6977788675,
0.5173394083,
0.8745375796,
1.0,
0.8298927961,
0.8488122828,
0.8825441551,
1.0,
1.0,
1.0,
0.8534697536,
1.0]

In [ ]:
bams

In [ ]:
chip.bigWigFrom(bams[1:], 
                genome='GRCh38',scaling=scales,
               numthreads=8)

In [ ]:
mkdir ../../data/IRF2BP2_degraded_3/

In [ ]:
! mv diffPeaks ../../data/IRF2BP2_degraded_3
! mv diffData ../../data/IRF2BP2_degraded_3

In [ ]:
!mv bigwig ../../data/recalib_bigwig_3 

In [ ]:
os.popen('for i in $(ls ../../data/IRF2BP2_degraded_3/diffPeaks); \
            do echo $(wc -l "../../data/IRF2BP2_degraded_3/diffPeaks/"$i); \
            done').read().split('\n')

In [18]:
bw = ! ls ../../data/recalib_bigwig_3/*.bw
bw

['../../data/recalib_bigwig_3/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.bw',
 '../../data/recalib_bigwig_3/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1.bw',
 '../../data/recalib_bigwig_3/mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1.bw',
 '../../data/recalib_bigwig_3/mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1.bw',
 '../../data/recalib_bigwig_3/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1.bw',
 '../../data/recalib_bigwig_3/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1.bw',
 '../../data/recalib_bigwig_3/mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1.bw',
 '../../data/recalib_bigwig_3/mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1.bw',
 '../../data/recalib_bigwig_3/mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1.bw',
 '../../data/recalib_bigwig_3/mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R1.bw',
 '../../data/recalib_bigwig_3/mp841-MV411_IRF2BP_VHL_6h-SMC1-r1_R1.bw',
 '../../data/recalib_bigwig_3/mp842-MV411_IRF2BP_VHL_6h-MED1-r3_R1.bw',
 '../../data/recalib_bigwig_3/mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1_R1.bw',
 '../../data/recalib_bigwig_3/mp844-MV411_IRF2BP_VHL_6h-

In [22]:
cond1peak = ! ls ../../data/IRF2BP2_degraded_3/diffPeaks/*cond1.bed
cond2peak = ! ls ../../data/IRF2BP2_degraded_3/diffPeaks/*cond2.bed
commonpeak = ! ls ../../data/IRF2BP2_degraded_3/diffPeaks/*common.bed
cond1peak

['../../data/IRF2BP2_degraded_3/diffPeaks/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_vs_mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_3/diffPeaks/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1_vs_mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_3/diffPeaks/mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1_vs_mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R1_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_3/diffPeaks/mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1_vs_mp841-MV411_IRF2BP_VHL_6h-SMC1-r1_R1_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_3/diffPeaks/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1_vs_mp842-MV411_IRF2BP_VHL_6h-MED1-r3_R1_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_3/diffPeaks/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1_vs_mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1_R1_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_3/diffPeaks/mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1_vs_mp844-MV411_IRF2BP_VHL_6h-CEBPA-r1_R1_c3.0_cond1.bed']

In [16]:
names = ["CDK8","BRD4","IRF8","SMC1","MED1","ZEB2","CEBPA"]

In [23]:
for i in range(int(len(bw)/2)):
    if i<0:
        continue
    name1 = bw[i]
    name2 = bw[7+i]
    peak = [cond1peak[i], commonpeak[i], cond2peak[i]]
    #for val in peak:
        #chip.dropWeirdChromosomes(val)
    name = names[i]
    chip.computePeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], refpoint='center', peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], window=3000, folder="", title=name, numthreads=14, name='../../data/IRF2BP2_degraded_3/diffPeaks/'+name+'_mat.pdf', withDeeptools=True, torecompute=True)

CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_3/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.bw ../../data/recalib_bigwig_3/mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1.bw  --referencePoint center --regionsFileName ../../data/IRF2BP2_degraded_3/diffPeaks/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_vs_mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1_c3.0_cond1.bed ../../data/IRF2BP2_degraded_3/diffPeaks/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_vs_mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1_c3.0_common.bed ../../data/IRF2BP2_degraded_3/diffPeaks/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_vs_mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1_c3.0_cond2.bed  --missingDataAsZero --outFileName ../../data/IRF2BP2_degraded_3/diffPeaks/CDK8_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 14 && plotHeatmap --matrixFile ../../data/IRF2BP2_degraded_3/diffPeaks/CDK8_mat.gz --outFileName ../../data/IRF2BP2_degraded_3/diffPeaks/CDK8_mat.pdf --refPointLabel center --regionsLabel DMSO_peaks common VHL_peaks  --sam

In [256]:
names = ["CDK8","BRD4","IRF8","SMC1","MED1","ZEB2","CEBPA"]
names.extend([i+'_VHL' for i in names])
names

['CDK8',
 'BRD4',
 'IRF8',
 'SMC1',
 'MED1',
 'ZEB2',
 'CEBPA',
 'CDK8_VHL',
 'BRD4_VHL',
 'IRF8_VHL',
 'SMC1_VHL',
 'MED1_VHL',
 'ZEB2_VHL',
 'CEBPA_VHL']

In [155]:
mkdir ../../data/peaks3

In [156]:
! gsutil cp gs://amlproject/Chip/IRF2BP2_degraded_rep3/bwa/mergedLibrary/macs/narrowPeak/*.narrowPeak ../../data/peaks3

Copying gs://amlproject/Chip/IRF2BP2_degraded_rep3/bwa/mergedLibrary/macs/narrowPeak/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_peaks.narrowPeak...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep3/bwa/mergedLibrary/macs/narrowPeak/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1_peaks.narrowPeak...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep3/bwa/mergedLibrary/macs/narrowPeak/mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1_peaks.narrowPeak...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep3/bwa/mergedLibrary/macs/narrowPeak/mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1_peaks.narrowPeak...
- [4 files][ 21.3 MiB/ 21.3 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://amlproject/Chip/IRF2BP2_degraded_rep3/bwa/mergedLibrary/macs/narrowPeak/mp835-MV41

In [257]:
peaks = ! ls ../../data/peaks3/*
peaks

['../../data/peaks3/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_peaks.narrowPeak',
 '../../data/peaks3/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1_peaks.narrowPeak',
 '../../data/peaks3/mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1_peaks.narrowPeak',
 '../../data/peaks3/mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1_peaks.narrowPeak',
 '../../data/peaks3/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1_peaks.narrowPeak',
 '../../data/peaks3/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1_peaks.narrowPeak',
 '../../data/peaks3/mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1_peaks.narrowPeak',
 '../../data/peaks3/mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1_peaks.narrowPeak',
 '../../data/peaks3/mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1_peaks.narrowPeak',
 '../../data/peaks3/mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R1_peaks.narrowPeak',
 '../../data/peaks3/mp841-MV411_IRF2BP_VHL_6h-SMC1-r1_R1_peaks.narrowPeak',
 '../../data/peaks3/mp842-MV411_IRF2BP_VHL_6h-MED1-r3_R1_peaks.narrowPeak',
 '../../data/peaks3/mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1_R1_peaks.narrowPeak',
 '..

In [ ]:
for i in range(len(bw)):
    chip.computePeaksAt(peaks[i], bw[i], window=3000, folder="", title=names[i], numthreads=7, torecompute=True, onlyProfile=True, refpoint='center', name='../../data/IRF2BP2_degraded_3/diffPeaks/'+names[i]+'_mat_profile.pdf', withDeeptools=True)
    chip.computePeaksAt(peaks[i], bw[i], window=3000, folder="", title=names[i], numthreads=7, torecompute=True, refpoint='center', onlyProfile=True,name='../../data/IRF2BP2_degraded_3/diffPeaks/'+names[i]+'_mat_profile_clust3.pdf', withDeeptools=True, cluster=3)

CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_3/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.bw  --referencePoint center --regionsFileName ../../data/peaks3/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_peaks.narrowPeak  --missingDataAsZero --outFileName ../../data/IRF2BP2_degraded_3/diffPeaks/CDK8_mat_profile.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotProfile --matrixFile ../../data/IRF2BP2_degraded_3/diffPeaks/CDK8_mat_profile.gz --outFileName ../../data/IRF2BP2_degraded_3/diffPeaks/CDK8_mat_profile.pdf --refPointLabel center --plotTitle CDK8', returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_3/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.bw  --referencePoint center --regionsFileName ../../data/peaks3/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_peaks.narrowPeak  --missingDataAsZero --outFileName ../../data/IRF2BP2_degraded_3/diffPeaks/CDK8_mat_profile_clust3.gz --upstream 3000 --dow

CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_3/mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1.bw  --referencePoint center --regionsFileName ../../data/peaks3/mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1_peaks.narrowPeak  --missingDataAsZero --outFileName ../../data/IRF2BP2_degraded_3/diffPeaks/CEBPA_mat_profile_clust3.gz --upstream 3000 --downstream 3000 --numberOfProcessors 7 && plotProfile --matrixFile ../../data/IRF2BP2_degraded_3/diffPeaks/CEBPA_mat_profile_clust3.gz --outFileName ../../data/IRF2BP2_degraded_3/diffPeaks/CEBPA_mat_profile_clust3.pdf --refPointLabel center --perGroup --kmeans 3 --plotTitle CEBPA', returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args='computeMatrix reference-point -S ../../data/recalib_bigwig_3/mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1.bw  --referencePoint center --regionsFileName ../../data/peaks3/mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1_peaks.narrowPeak  --missingDataAsZero --outFileName ../../data/IRF2BP2_degraded_3/diffPeaks/C

### on unscalled data

In [185]:
bams = !ls ../../data/IRF2BP2_degraded_3/mp*.bam
bams

['../../data/IRF2BP2_degraded_3/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.mLb.clN.sorted.bam',
 '../../data/IRF2BP2_degraded_3/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1.mLb.clN.sorted.bam',
 '../../data/IRF2BP2_degraded_3/mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1.mLb.clN.sorted.bam',
 '../../data/IRF2BP2_degraded_3/mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1.mLb.clN.sorted.bam',
 '../../data/IRF2BP2_degraded_3/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1.mLb.clN.sorted.bam',
 '../../data/IRF2BP2_degraded_3/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1.mLb.clN.sorted.bam',
 '../../data/IRF2BP2_degraded_3/mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1.mLb.clN.sorted.bam',
 '../../data/IRF2BP2_degraded_3/mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1.mLb.clN.sorted.bam',
 '../../data/IRF2BP2_degraded_3/mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1.mLb.clN.sorted.bam',
 '../../data/IRF2BP2_degraded_3/mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R1.mLb.clN.sorted.bam',
 '../../data/IRF2BP2_degraded_3/mp841-MV411_IRF2BP_VHL_6h-SMC1-r1_R1.mLb.clN.sorted.bam',
 '

In [ ]:
! mkdir ../../data/IRF2BP2_degraded_3/diffPeaks_unscaled

In [186]:
#on unscalled data 
for i in range(int(len(bams)/2)):
    if i < 0:
        continue
    name1 = bams[i]
    name2 = bams[7+i]
    print(name1,name2)
    chip.fullDiffPeak(name1,name2, control1='../../data/results3/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam', directory = "../../data/IRF2BP2_degraded_3/diffData_unscaled/", res_directory = "../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/",pairedend=False)

../../data/IRF2BP2_degraded_3/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1.mLb.clN.sorted.bam ../../data/IRF2BP2_degraded_3/mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1_R1.mLb.clN.sorted.bam
doing diff from ../../data/IRF2BP2_degraded_3/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1.mLb.clN.sorted.bam and ../../data/IRF2BP2_degraded_3/mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1_R1.mLb.clN.sorted.bam
computing the fragment avg size
217
computing the scaling values
16490251 21182237
doing differential peak binding
../../data/IRF2BP2_degraded_3/mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1.mLb.clN.sorted.bam ../../data/IRF2BP2_degraded_3/mp844-MV411_IRF2BP_VHL_6h-CEBPA-r1_R1.mLb.clN.sorted.bam
doing diff from ../../data/IRF2BP2_degraded_3/mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1.mLb.clN.sorted.bam and ../../data/IRF2BP2_degraded_3/mp844-MV411_IRF2BP_VHL_6h-CEBPA-r1_R1.mLb.clN.sorted.bam
computing the fragment avg size
176
computing the scaling values
30015739 29684545
doing differential peak binding


In [311]:
!mkdir ../../data/bigwigrep3unscaled/
!gsutil -m cp gs://amlproject/Chip/IRF2BP2_degraded_rep3/bwa/mergedLibrary/bigwig/*.bigWig ../../data/bigwigrep3unscaled/

Copying gs://amlproject/Chip/IRF2BP2_degraded_rep3/bwa/mergedLibrary/bigwig/INPUT_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep3/bwa/mergedLibrary/bigwig/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep3/bwa/mergedLibrary/bigwig/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep3/bwa/mergedLibrary/bigwig/mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep3/bwa/mergedLibrary/bigwig/mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep3/bwa/mergedLibrary/bigwig/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep3/bwa/mergedLibrary/bigwig/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/IRF2BP2_degraded_rep3/bwa/mergedLibrary/bigwig/mp837-MV411_IRF2BP_DMSO_6h-CE

In [312]:
bw = ! ls ../../data/bigwigrep3unscaled/*.bigWig
bw

['../../data/bigwigrep3unscaled/INPUT_R1.mLb.clN.bigWig',
 '../../data/bigwigrep3unscaled/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.mLb.clN.bigWig',
 '../../data/bigwigrep3unscaled/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1.mLb.clN.bigWig',
 '../../data/bigwigrep3unscaled/mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1.mLb.clN.bigWig',
 '../../data/bigwigrep3unscaled/mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1.mLb.clN.bigWig',
 '../../data/bigwigrep3unscaled/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1.mLb.clN.bigWig',
 '../../data/bigwigrep3unscaled/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1.mLb.clN.bigWig',
 '../../data/bigwigrep3unscaled/mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1.mLb.clN.bigWig',
 '../../data/bigwigrep3unscaled/mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1.mLb.clN.bigWig',
 '../../data/bigwigrep3unscaled/mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1.mLb.clN.bigWig',
 '../../data/bigwigrep3unscaled/mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R1.mLb.clN.bigWig',
 '../../data/bigwigrep3unscaled/mp841-MV411_IRF2BP_VHL_6h-SMC1-r1_R1.mLb.c

In [313]:
cond1peak = ! ls ../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/*cond1.bed
cond2peak = ! ls ../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/*cond2.bed
commonpeak = ! ls ../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/*common.bed
cond1peak

['../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_vs_mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/mp832-MV411_IRF2BP_DMSO_6h-BRD4-r1_R1_vs_mp839-MV411_IRF2BP_VHL_6h-BRD4-r1_R1_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/mp833-MV411_IRF2BP_DMSO_6h-IRF8-r1_R1_vs_mp840-MV411_IRF2BP_VHL_6h-IRF8-r1_R1_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/mp834-MV411_IRF2BP_DMSO_6h-SMC1-r1_R1_vs_mp841-MV411_IRF2BP_VHL_6h-SMC1-r1_R1_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1_vs_mp842-MV411_IRF2BP_VHL_6h-MED1-r3_R1_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1_vs_mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1_R1_c3.0_cond1.bed',
 '../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/mp837-MV411_IRF2BP_DMSO_6h-CEBPA-r1_R1_vs_mp844-MV411_IRF2BP_VHL_6h-CEBPA

In [314]:
names = ["CDK8","BRD4","IRF8","SMC1","MED1","ZEB2","CEBPA"]

In [317]:
for i in range(int((len(bw)-1)/2)):
    if i <0:
        continue
    name1 = bw[1+i]
    name2 = bw[8+i]
    peak = [cond1peak[i], commonpeak[i], cond2peak[i]]
    #for val in peak:
        #chip.dropWeirdChromosomes(val)
    name = names[i]
    print(name1,name2)
    chip.computePeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], window=3000, folder="", title=name, numthreads=14, torecompute=True, refpoint="center", name='../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/'+name+'_mat.pdf', withDeeptools=True)

../../data/bigwigrep3unscaled/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.mLb.clN.bigWig ../../data/bigwigrep3unscaled/mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1.mLb.clN.bigWig
CompletedProcess(args='computeMatrix reference-point -S ../../data/bigwigrep3unscaled/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1.mLb.clN.bigWig ../../data/bigwigrep3unscaled/mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1.mLb.clN.bigWig  --referencePoint center --regionsFileName ../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_vs_mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1_c3.0_cond1.bed ../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_vs_mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1_c3.0_common.bed ../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/mp831-MV411_IRF2BP_DMSO_6h-CDK8-r1_R1_vs_mp838-MV411_IRF2BP_VHL_6h-CDK8-r1_R1_c3.0_cond2.bed  --missingDataAsZero --outFileName ../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/CDK8_mat.gz --upstream 3000 --downstream 3000 --numberOfProc

CompletedProcess(args='computeMatrix reference-point -S ../../data/bigwigrep3unscaled/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1.mLb.clN.bigWig ../../data/bigwigrep3unscaled/mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1_R1.mLb.clN.bigWig  --referencePoint center --regionsFileName ../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1_vs_mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1_R1_c3.0_cond1.bed ../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1_vs_mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1_R1_c3.0_common.bed ../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/mp836-MV411_IRF2BP_DMSO_6h-ZEB2-r1_R1_vs_mp843-MV411_IRF2BP_VHL_6h-ZEB2-r1_R1_c3.0_cond2.bed  --missingDataAsZero --outFileName ../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/ZEB2_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 14 && plotHeatmap --matrixFile ../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/ZEB2_mat.gz --outFileName ../../data/IRF2BP2_degraded_3/diffPeaks_unscaled/ZEB

In [ ]:
! gsutil -m cp -r "../../data/recalib_bigwig_3/*" gs://amlproject/Chip/IRF2BP2_degraded_rep3/recalib_bigwig/ &&\
gsutil -m cp -r "../../data/IRF2BP2_degraded_3/diffData/*" gs://amlproject/Chip/IRF2BP2_degraded_rep3/diffData/ &&\
gsutil -m cp -r "../../data/IRF2BP2_degraded_3/diffPeaks/*" gs://amlproject/Chip/IRF2BP2_degraded_rep3/diffPeaks/

In [ ]:
## on local
! scp runningchip2.us-east1-b.aml-chipseq-rnaseq-259419:"~/data/IRF2BP2_degraded_3/diffPeaks_unscaled/*.pdf" unscaled/v3/
! scp runningchip2.us-east1-b.aml-chipseq-rnaseq-259419:"~/data/IRF2BP2_degraded_3/diffPeaks/*.pdf" scaled/v3/
! scp runningchip2.us-east1-b.aml-chipseq-rnaseq-259419:"~/data/diffBinding_rep1/diffPeaks_unscaled/*.pdf" unscaled/v3/
! scp runningchip2.us-east1-b.aml-chipseq-rnaseq-259419:"~/data/diffBinding_rep1/diffPeaks/*.pdf" scaled/v1/
! scp runningchip2.us-east1-b.aml-chipseq-rnaseq-259419:"~/data/diffBinding_hist/diffPeaks_unscaled/*.pdf" unscaled/v1/
! scp runningchip2.us-east1-b.aml-chipseq-rnaseq-259419:"~/data/diffBinding_hist/diffPeaks/*.pdf" unscaled/v1/
! scp runningchip2.us-east1-b.aml-chipseq-rnaseq-259419:"~/data/diffBinding_rep2/diffPeaks_unscaled/*.pdf" unscaled/v2/
! scp runningchip2.us-east1-b.aml-chipseq-rnaseq-259419:"~/data/diffBinding_rep2/diffPeaks/*.pdf" scaled/v2/

# TODO:

- run full diffPeak v1,v2
- run v1 v2 unscaled plots
- run v1 v2 profiles
- run histones unscaled plots
- run v3 unscaled plots

## RNA

In [ ]:
rename1 = {
'gs://amlproject/slamseq/IRF2BP2/20191212_10_MP7623_S30_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp810_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_10_MP7623_S30_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp810_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_11_MP7623_S31_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp811_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_11_MP7623_S31_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp811_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_12_MP7623_S32_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp812_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_12_MP7623_S32_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp812_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_13_MP7623_S33_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp813_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_13_MP7623_S33_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp813_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_14_MP7623_S34_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp814_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_14_MP7623_S34_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp814_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_15_MP7623_S35_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp815_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_15_MP7623_S35_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp815_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_16_MP7623_S36_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp816_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_16_MP7623_S36_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp816_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_1_MP7623_S21_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp801_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_1_MP7623_S21_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp801_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_2_MP7623_S22_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp802_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_2_MP7623_S22_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp802_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_3_MP7623_S23_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp803_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_3_MP7623_S23_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp803_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_4_MP7623_S24_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp804_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_4_MP7623_S24_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp804_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_5_MP7623_S25_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp805_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_5_MP7623_S25_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp805_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_6_MP7623_S26_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp806_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_6_MP7623_S26_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp806_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_7_MP7623_S27_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp807_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_7_MP7623_S27_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp807_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_8_MP7623_S28_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp808_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_8_MP7623_S28_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp808_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_9_MP7623_S29_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp809_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_9_MP7623_S29_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp809_2.fastq.gz'}

In [ ]:
for k, val in rename1.items():
    os.system('gsutil mv '+k+' '+val)

In [ ]:
a

In [ ]:
terra.list_blobs_with_prefix("amlproject","slamseq/IRF2BP2/")

In [ ]:
terra.uploadFromFolder("amlproject", "slamseq/IRF2BP2/", workspace="broad-firecloud-ccle/hg38_RNAseq", 
                       fformat="fastq12", source="maximP",samplesetname="slamseq_IRF2BP2")

In [ ]:
samplesetname = 'slamseq_IRF2BP2'
refworkspace="broad-firecloud-ccle/hg38_RNAseq"
refwm = dm.WorkspaceManager(refworkspace)

In [ ]:
submission_id = refwm.create_submission("slamdunk", samplesetname,'sample_set',expression='this.samples')
terra.waitForSubmission(refworkspace, submission_id)

In [ ]:
submission_id1 = refwm.create_submission("rsem_v1-0_BETA_cfg", samplesetname,'sample_set',expression='this.samples')
terra.waitForSubmission(refworkspace, submission_id1)

In [ ]:
! pip3 install git+https://github.com/jkobject/slamdunk.git --upgrade

In [ ]:
folder_bams = "../../data/slam/data/"
bams = ! ls $folder_bams*.fastq.gz
bams

In [ ]:
for val in h.grouped(bams[4:],2):
    print(val)
    res = os.system('../../TrimGalore-0.6.5/trim_galore --paired --cores 12 --retain_unpaired -stringency 3 --illumina '+
    val[0] + ' '+val[1]+" -o "+folder_bams)
    if res!=0:
        raise ValueError()
    res = os.system('slamdunk all -r ../../data/slam/ref_genome.fa -b ../../data/slam/GSE100708_hg38_refseq_062016_ensemblv84_3UTR.bed \
    -o ../../data/slam/resIRF2BP2/ -t 12 -c 2  '+'.'.join(val[0].split('.')[:-2])+'_val_1.fq.gz \
    '+'.'.join(val[1].split('.')[:-2])+"_val_2.fq.gz")
    if res!=0:
        raise ValueError()

In [ ]:
for val in h.grouped(bams[:4],2):
    print(val)
    res = os.system('slamdunk all -r ../../data/slam/ref_genome.fa -b ../../data/slam/GSE100708_hg38_refseq_062016_ensemblv84_3UTR.bed \
    -o ../../data/slam/resIRF2BP2/ -t 12 -c 2  '+'.'.join(val[0].split('.')[:-2])+'_val_1.fq.gz \
    '+'.'.join(val[1].split('.')[:-2])+"_val_2.fq.gz")
    if res!=0:
        raise ValueError()